<a href="https://colab.research.google.com/github/manzoniandrea/Geo-reconstruction_-ANN/blob/main/ISPRA/Lithology_Po.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Library installation

In [ ]:
pip install geopandas

     |████████████████████████████████| 1.0 MB 5.5 MB/s 
     |████████████████████████████████| 6.3 MB 31.1 MB/s 
     |████████████████████████████████| 16.7 MB 212 kB/s 


In [ ]:
pip install rasterio

     |████████████████████████████████| 19.3 MB 1.2 MB/s 


In [ ]:
pip install pyvista

     |████████████████████████████████| 1.4 MB 5.4 MB/s 
     |████████████████████████████████| 88.3 MB 96 kB/s 
     |████████████████████████████████| 1.1 MB 32.5 MB/s 
     |████████████████████████████████| 144 kB 40.9 MB/s 
     |████████████████████████████████| 94 kB 1.7 MB/s 
     |████████████████████████████████| 271 kB 44.8 MB/s 


#import libraries and input files

In [ ]:
import pyvista as pv
import pandas as pd
from pyproj import Transformer
import numpy as np
import rasterio

In [ ]:
from bs4 import BeautifulSoup
import requests

stratrigrafie_ISPRA.csv is derived from shape files available on the ISPRA national database.
Data source: https://www.isprambiente.gov.it/it/banche-dati/banche-dati-folder/suolo-e-territorio/dati-geognostici-e-geofisici

In [ ]:
pd.options.mode.chained_assignment = None 
df = pd.read_csv('stratrigrafie_ISPRA.csv')
df.drop('Comune', inplace=True, axis=1)
df.drop('Opera', inplace=True, axis=1)
df.drop('Profondit�', inplace=True, axis=1)
df.drop('Longitudin', inplace=True, axis=1)
df.drop('Latitudine', inplace=True, axis=1)
df.drop('LEGENDA', inplace=True, axis=1)
df['spessore_livello'] = np.zeros(df.shape[0])
df['l1_descr'] = np.zeros(df.shape[0])
well=0
for i in range(0,df.shape[0]):
  link = df['URL'][well]
  source = requests.get(link).text
  soup = BeautifulSoup(source, 'lxml')
  tables = soup.find_all('table')
  if len(tables[-1])<1:
    well+=1
  else:
    tbl = tables[-1]
    data_frame = pd.read_html(str(tbl),header=0,decimal=',',thousands='.')[0]
    for layer in range(0,data_frame.shape[0]):
      if layer == 0:
        df['spessore_livello'][well] = data_frame['Spessore (m)'][0]
        df['l1_descr'][well] = data_frame['Descrizione litologica'][0]
      else:
        line = pd.DataFrame({'URL':df['URL'][well],'E_54012':df['E_54012'][well],'N_54012':df['N_54012'][well],'spessore_livello':data_frame['Spessore (m)'][layer],'l1_descr':data_frame['Descrizione litologica'][layer]}, index=[well+layer+1])
        df = pd.concat([df.iloc[:well+layer], line, df.iloc[well+layer:]]).reset_index(drop=True)
    well=well+layer+1
    if i%100==0:
      print(i)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500


In [ ]:
df.to_excel("output.xlsx")

In [ ]:
df = pd.read_excel("output.xlsx")

#Data preprocessing

Assign the well head level according to the well location using the copernicus Digital elevation model.
DEM_clip.tif is derived from: https://land.copernicus.eu/imagery-in-situ/eu-dem/eu-dem-v1.1

In [ ]:
wellLoc = df
src = rasterio.open('DEM_clip.tif')
coords = [(x,y) for x, y in zip( wellLoc.E_54012, wellLoc.N_54012)]

wellLoc.index = range(len(wellLoc))
wellLoc['quota'] = [x[0] for x in src.sample(coords)]

In [ ]:
wellLoc1 = wellLoc

compute the top and bottom of each layer

In [ ]:
#valuto il top e il bottom layer dei dati
offsetList = []
linSec = []
linVerts = []

i = 0

botLito = np.zeros(wellLoc.spessore_livello.size)
topLito = np.zeros(wellLoc.spessore_livello.size)

for layer in range(0,wellLoc.spessore_livello.size):
  x,y,z = wellLoc.loc[layer][['E_54012','N_54012','quota']]
  if layer == 0:
    topLito[0] = wellLoc.quota[0]
    botLito[0] = topLito[0] - wellLoc.spessore_livello[0]
  elif wellLoc.URL[layer] == wellLoc.URL[layer-1]:
    topLito[layer] =  botLito[layer-1]
    botLito[layer] = topLito[layer] - (float(wellLoc.spessore_livello[layer])-float(wellLoc.spessore_livello[layer-1]))
  else:
    topLito[layer] = wellLoc.quota[layer]
    botLito[layer] = topLito[layer] - wellLoc.spessore_livello[layer]
  cellVerts = [[x,y,topLito[layer]],[x,y,botLito[layer]]]
  offsetList.append(i*3)
  linSec = linSec + [2,2*i,2*i+1]
  linVerts = linVerts + cellVerts
  i+=1
  if topLito[layer] > wellLoc.quota[layer]:
    print(layer)

offsetArray = np.array(offsetList)
linArray = np.array(linSec)
cellType = np.ones([i])*3
vertArray = np.array(linVerts)

wellLoc['top_Layer'] = topLito
wellLoc['bttom_Layer'] =  botLito

26678


#Data chategorization

In [ ]:
#creo e stampo il dizionario
litoDict =  {}

i=1000

litotype = pd.DataFrame(np.zeros(len(wellLoc.l1_descr.unique())))
litotype['name'] = pd.DataFrame(np.zeros(len(wellLoc.l1_descr.unique())))
litotype = litotype.iloc[: , 1:]
litotype['name'] = wellLoc.l1_descr.unique()
litotype.name.astype(str)
litotype.name[742] ='zero'
litotype = litotype.sort_values(by=['name'],ignore_index=1)

for lito in litotype.name.unique():
  litoDict[lito] = i
  i += 1
litoDict

{'" FILTRO" FORSE ARGILLA; PERFORO FINO A 116 MT.': 1000,
 '"COME SOPRA", DI COLORE BRUNO-ROSSICCIO, TALORA DEBOLMENTE ALTERATI': 1001,
 '"DESCRIZIONE LITOLOGICA NON PRESENTE"': 1002,
 '"MARNOSO ARENACEA ROMAGNOLA" ARENARIE QUARZOSO FELDSPATICHE IN GROSSI BANCHI CON INTERCALAZIONI DI SILTITI GRIGIE, MARNE ED ARGILLITICI. TRA 27 E 28 E TRA 36 E 38 M DAL P.C. SI HANNO LIVELLI...': 1003,
 '"MARNOSO-ARENACEE ROMAGNOLA" ALTERNANZE RITMICHE DI ARENARIE E SILTITI RISEDIMENTATE E DI MARNE ED ARGILLITI. GENERALMENTE LE SILTITI SONO PREVALENTI ED HANNO COMPOSIZIONE ANALOGA ALLE ARENARIE': 1004,
 '(CONTINUA) GHIAIA MISTA ARGILLA': 1005,
 '(FALDA LEGGERMENTE ACQUIFERA) ARGILLA SABBIOSA GIALLA': 1006,
 '(INCLINAZIONE SONDA: 45 GRADI, DIREZIONE 240 GRADI) CONGLOMERATO CEMENTIZIO': 1007,
 '(MATERIALE NON RECUPERATO) PROBABILMENTE SABBIA FINE LIMOSA': 1008,
 '+ SD) = STRATI DURI. M. 32': 1009,
 ',,ILLEGGIBILE': 1010,
 '- ALLUVIONI - GHIAIA - SABBIA E CIOTTOLI.': 1011,
 '- DESCRIZIONE LITOLOGICA NON PR

In [ ]:
#uso il dizionario per dare un valore numerico ai litotipi
wellLoc['l1'] = wellLoc.l1_descr
wellLoc = wellLoc.replace({'l1':litoDict})

In [ ]:
#salvo i risultati in un file excel
wellLoc.to_excel('Po.xlsx') 

In [ ]:
print(wellLoc.l1_descr.unique())

['GHIAIA CON CEPPO NOCCIOLA E SABBIA CON GROSSI TROVANTI'
 'ARGILLA CON POCA GHIAIA'
 'GHIAIA CON CEPPO NOCCIOLA E BIANCO MOLTO COMPATTO' ...
 'LIMO OCRACEO MOLTO SABBIOSO'
 'GHIAIA E CIOTTOLI IN MATRICE LIMOSA SABBIOSA PASSANTE A SABBIOSA LIMOSA MARRONE'
 'GHIAIA E CIOTTOLI IN MATRICE SABBIOSA MEDIA POCO LIMOSA MARRONE GRIGIASTRA CON LIVELLO OCRA ROSSICCIO OSSIDATO DA 5.70 A 6.00 M']


In [ ]:
wellLoc = pd.read_excel('Po.xlsx') 

In [ ]:
wellLoc = pd.read_csv('wellLito_Po.csv') 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
wellLoc

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,URL,E_54012,N_54012,spessore_livello,l1_descr,quota,top_Layer,bttom_Layer,l1,category_name,category
0,0,0,0,http://sgi2.isprambiente.it/indagini/scheda_in...,862684,5604045,23.0,GHIAIA CON CEPPO NOCCIOLA E SABBIA CON GROSSI ...,164,164.0,141.0,11118,Ghiaia,300
1,1,1,1,http://sgi2.isprambiente.it/indagini/scheda_in...,862684,5604045,4.0,ARGILLA CON POCA GHIAIA,164,141.0,160.0,2812,Argilla,100
2,2,2,2,http://sgi2.isprambiente.it/indagini/scheda_in...,862684,5604045,38.0,GHIAIA CON CEPPO NOCCIOLA E BIANCO MOLTO COMPATTO,164,160.0,126.0,11117,Ghiaia,300
3,3,3,3,http://sgi2.isprambiente.it/indagini/scheda_in...,862684,5604045,38.0,GHIAIA CON CEPPO GRIGIO E BIANCO,164,126.0,126.0,11102,Ghiaia,300
4,4,4,4,http://sgi2.isprambiente.it/indagini/scheda_in...,862684,5604045,3.0,ARGILLA GRIGIA-NOCCIOLA,164,126.0,161.0,4487,Argilla,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84998,94400,94400,94400,http://sgi2.isprambiente.it/indagini/scheda_in...,674166,5582283,0.2,TERRENO AGRICOLO SABB. LIMOSO,155,155.0,154.8,27762,Materiale organico,602
84999,94401,94401,94401,http://sgi2.isprambiente.it/indagini/scheda_in...,674166,5582283,0.8,LIMO OCRACEO MOLTO SABBIOSO,155,154.8,154.2,19284,Limo,400
85000,94402,94402,94402,http://sgi2.isprambiente.it/indagini/scheda_in...,674166,5582283,1.7,LIMO POCO SABBIOSO MARRONE CON VENATURE GRIGIE...,155,154.2,153.3,19294,Limo,400
85001,94403,94403,94403,http://sgi2.isprambiente.it/indagini/scheda_in...,674166,5582283,1.3,GHIAIA E CIOTTOLI IN MATRICE LIMOSA SABBIOSA P...,155,153.3,153.7,12471,Limo,400


In [ ]:
#creo e stampo il dizionario
litoDict =  {}

i=1000

litotype = pd.DataFrame(np.zeros(len(wellLoc.l1_descr.unique())))
litotype['name'] = pd.DataFrame(np.zeros(len(wellLoc.l1_descr.unique())))
litotype = litotype.iloc[: , 1:]
litotype['name'] = wellLoc.l1_descr.unique()
litotype.name.astype(str)
litotype.name[742] ='zero'
litotype = litotype.sort_values(by=['name'],ignore_index=1)

for lito in litotype.name.unique():
  litoDict[lito] = i
  i += 1
litoDict

{'" FILTRO" FORSE ARGILLA; PERFORO FINO A 116 MT.': 1000,
 '"COME SOPRA", DI COLORE BRUNO-ROSSICCIO, TALORA DEBOLMENTE ALTERATI': 1001,
 '"DESCRIZIONE LITOLOGICA NON PRESENTE"': 1002,
 '"MARNOSO ARENACEA ROMAGNOLA" ARENARIE QUARZOSO FELDSPATICHE IN GROSSI BANCHI CON INTERCALAZIONI DI SILTITI GRIGIE, MARNE ED ARGILLITICI. TRA 27 E 28 E TRA 36 E 38 M DAL P.C. SI HANNO LIVELLI...': 1003,
 '"MARNOSO-ARENACEE ROMAGNOLA" ALTERNANZE RITMICHE DI ARENARIE E SILTITI RISEDIMENTATE E DI MARNE ED ARGILLITI. GENERALMENTE LE SILTITI SONO PREVALENTI ED HANNO COMPOSIZIONE ANALOGA ALLE ARENARIE': 1004,
 '(CONTINUA) GHIAIA MISTA ARGILLA': 1005,
 '(FALDA LEGGERMENTE ACQUIFERA) ARGILLA SABBIOSA GIALLA': 1006,
 '(INCLINAZIONE SONDA: 45 GRADI, DIREZIONE 240 GRADI) CONGLOMERATO CEMENTIZIO': 1007,
 '(MATERIALE NON RECUPERATO) PROBABILMENTE SABBIA FINE LIMOSA': 1008,
 '+ SD) = STRATI DURI. M. 32': 1009,
 ',,ILLEGGIBILE': 1010,
 '- ALLUVIONI - GHIAIA - SABBIA E CIOTTOLI.': 1011,
 '- DESCRIZIONE LITOLOGICA NON PR

In [ ]:
key_list = list(litoDict.keys())
val_list = list(litoDict.values())

In [ ]:
#all values containing a key word
keyW = 'MATRICE ARGILLOSA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'SCARSA MATRICE' in key_list[position] or 'POCA MATRICE' in key_list[position] or 'SUBORDINATA MATRICE' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

ACCUMULO DETRITICO OFIOLITICO DI VERSANTE AD ABBONDANTE MATRICE ARGILLOSA
ALTERNANZA DI LIVELLI CENTIMETRICI DI SABBIA MEDIO-FINE IN MATRICE ARGILLOSA, RICCA IN LAMELLIBRANCHI, E DI LIMO ARGILLOSO TENDENTE A MARNOSO CON VELI DI MATERIALE ORGANICO NERASTRO, IN STRATI INCLINATI DA 3 A 5 GRADI SULL'ORIZZONTALE.
ALTERNANZA DI SABBIE CON SABBIA IN MATRICE ARGILLOSA
ALTERNANZE DI ARGILLA E GHIAIA IN MATRICE ARGILLOSA
ALTERNANZE DI ARGILLE CON CONCREZIONI E GHIAIE IN MATRICE ARGILLOSA SABBIOSA
ALTERNANZE DI CONGLOMERATO BEN CEMENTATO E CIOTTOLI IN MATRICE ARGILLOSA. CLASTI PREVALENTEMENTE GRANITOIDI, SECONDARIAMENTE METAMORFICI. RARI CARBONATI. CEMENTAZIONE MASSICCIA, DI DIFFICILE PERFORABILITA'. MATRICE ARGILLOSA COESIVA DI COLORE BEIGE. ASSENZA DI FRATTURE ACQUIFERE.
ALTERNANZE DI LIVELLI ARGILLOSI E GHIAIOSO-CIOTTOLOSI IN ABBONDANTE MATRICE ARGILLOSA
ALTERNANZE DI SPESSORE METRICO COSTITUITE DA CONGLOMERATI E CIOTTOLI IN MATRICE ARGILLOSA. MATRICE DI COLORE OCRA - BEIGE. CONGLOMERATO A CEM

In [ ]:
#all values containing a key word
keyW = 'MATRICE ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'SCARSA MATRICE' in key_list[position] or 'POCA MATRICE' in key_list[position] or 'SUBORDINATA MATRICE' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

BLOCCHI DI ARENARIA, DI MEDIA E GROSSA PEZZATURA, IN MATRICE ARGILLOSO-SABBIOSA E SABBIOSO LIMOSA
CIOTTOLI E BLOCCHI ADDENSATII IN MATRICE ARGILLOSO-SABBIOSA
CIOTTOLI E BLOCCHI ADDENSATII SERPENTINITICI IN MATRICE ARGILLOSO-SABBIOSA
CIOTTOLI POLIGENICI IMMERSI IN ABBONDANTE MATRICE ARGILLOSO - LIMOSA CON SCARSI ELEMENTI DI GHIAIETTO
CONGLOMERATO CON LIVELLI ARGILLOSI E LOCALE ABBONDANTE MATRICE ARGILLOSO-LIMOSA
CONGLOMERATO E GHIAIE SCIOLTE A SUPPORTO DI MATRICE ARGILLOSO ROSSASTRO
CONGLOMERATO IN MATRICE ARGILLOSO COMPATTO
DEPOSITO ALLUVIONALE CIOTTOLOSO CARATTERIZZATO DA LIVELLI CIOTTOLOSI CENTIMETRICI E LIVELLI CIOTTOLOSI DECIMETRICI, DI NATURA CALCAREO-MARNOSA, IMMERSI IN MATRICE ARGILLOSO-LIMOSA E LIMOSO-SABBIOSA. UMIDITA' CRESCENTE VERSO IL LETTO DEL DEPOSITO
DEPOSITO ALLUVIONALE GHIAIOSO, IN ABBONDANTE MATRICE ARGILLOSO-LIMOSA
DEPOSITO ALLUVIONALE GHIAIOSO, IN MATRICE ARGILLOSO-LIMOSA
DEPOSITO GHIAIOSO CON CIOTTOLI DI NATURA PER LO PIU' CALACAREO-MARNOSA DI DIMENSIONI 4-5 CM E S

In [ ]:
#all values containing a key word
keyW = 'MATRICE LIMOSA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'SCARSA MATRICE' in key_list[position] or 'POCA MATRICE' in key_list[position] or 'SUBORDINATA MATRICE' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

ALLUVIONI CIOTTOLOSE POSTGLACIALI CON MATRICE LIMOSA-ARGILLOSA (FALDA SUP. A CARATTERE STAGIONALE)
ALTERNANZE DI GHIAIE GROSSOLANE E CIOTTOLI SUBARROTONDATI CON LIVELLI SABBIOSI PULITI. LOCALMENTE RISULTANO INTERCALATI DEI TROVANTI. POSSIBILE PRESENZA LOCALMENTE DI LIVELLI LIMOSI E/O DI MATRICE LIMOSA
ARGILLA AZZURRA CON MATRICE LIMOSA
ARGILLA GRIGIA IN MATRICE LIMOSA
ARGILLA GRIGIA IN MATRICE LIMOSA CON POCA GHIAIA
ARGILLA GRIGIA IN MATRICE LIMOSA CON POCO CONGLOMER.
ARGILLA GRIGIA IN MATRICE LIMOSA PLASTICA
ARGILLA IN MATRICE LIMOSA
ARGILLA IN MATRICE LIMOSA E GHIAIA
ARGILLA IN MATRICE LIMOSA MARRONE
ARGILLA IN MATRICE LIMOSA MARRONE CON POCHISSIMA GHIAIA
ARGILLA MARRONE IN MATRICE LIMOSA CON GHIAIA E CIOTTOLI
ARGILLA NOCCIOLA IN MATRICE LIMOSA
ARGILLA ROSSA IN MATRICE LIMOSA
ARGILLA ROSSA IN MATRICE LIMOSA CON POCA GHIAIA
ARGILLA ROSSICCIA IN MATRICE LIMOSA CON GHIAIA
BRECCIA CALCAREA MEDIA IN MATRICE LIMOSA DEB. SABBIOSA GRIGIO CHIARA
BRECCIA MEDIA IN MATRICE LIMOSA DEB. SABBIOSA G

In [ ]:
#all values containing a key word
keyW = 'MATRICE LIMO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'SCARSA MATRICE' in key_list[position] or 'POCA MATRICE' in key_list[position] or 'SUBORDINATA MATRICE' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

ALTERNANZA DI LIVELLI LIMOSI E LIMOSO-ARGILLOSI MOLTO COMPATTI E DI LIVELLI CENTIMETRICI SABBIOSI E SABBIOSO-LIMOSI MENO COMPATTI: COLORE GRIGIO E GRIGIO-VIOLACEO CON PRESENZA DI LIVELLI CENTIMETRICI SABBIOSI MEDIO-FINI / IDEM COME SOPRA CON LIVELLETTI CENTIMETRICI DI GHIAIETTO IN MATRICE LIMOSO-SABBIOSA E RESTI VEGETALI
BRECCIA CALCAREA VARIA PEZZATURA IN MATRICE LIMOSO ARGILLOSA GRIGIASTRA
BRECCIA CALCAREA VARIA PEZZATURA INI DEB. MATRICE LIMOSO SABBIOSA GRIGIA
BRECCIA IN MATRICE LIMOSO SABBIOSA GRIGIA
BRECCIA MEDIO GROSSOLANA ED ELEMENTI CALCAREI IN MATRICE LIMOSO ARGILLOSA GRIGIASTRA
BRECCIA VARIA PEZZATURA IN MATRICE LIMOSO ARGILLOSA NERASTRA
BRECCIA VARIA PEZZATURA IN MATRICE LIMOSO ARGILLOSA NOCCIOLA
BRECCIA VARIA PEZZATURA IN MATRICE LIMOSO SABBIOSA NOCCIOLA
CAPPELLACIO ALTERATO: DETRITI ARENACEI DI MATRICE LIMOSO SABBIOSA
CIOTTOLI (40%) CON GHIAIA (30%) E SABBIA MEDIO FINE IN DEBOLE MATRICE LIMOSO - ARGILLOSA.
CIOTTOLI ALTERATI, GHIAIA E CROSTE DI CONGLOMERATO IN MATRICE LIMOS

In [ ]:
#all values containing a key word
keyW = 'MATRICE LIMOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'SCARSA MATRICE' in key_list[position] or 'POCA MATRICE' in key_list[position] or 'SUBORDINATA MATRICE' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

In [ ]:
#all values containing a key word
keyW = 'MATRICE SABBIOSA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'SCARSA MATRICE' in key_list[position] or 'POCA MATRICE' in key_list[position] or 'SUBORDINATA MATRICE' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

ALLUVIONALE - GHIAIE GROSSOLANE IN MATRICE SABBIOSA
ALLUVIONI ATTUALI: BLOCCHI E CIOTTOLI CALCAREI SUBORDINATAMENTE ARENACEI IN MATRICE SABBIOSA
ALLUVIONI: BLOCCHI E CIOTTOLI PREVALENTEMENTE CALCAREI IN MATRICE SABBIOSA
ALTERNANZE CENTIMETRICHE DI GHIAIE FINI A MATRICE SABBIOSA E SABBIE CON LIVELLI LIMO-ARGILLOSI
ALTERNANZE DI GHIAIE IN MATRICE SABBIOSA E ARGILLE SABBIOSE
ARENARIE MEDIO-GROSSOLANE, ARENARIE CONGLOMERATICHE CON ELEMENTI POLIGENICI ED ETEROMETRICI IN MATRICE SABBIOSA.
ARENARIE MEDIO-GROSSOLANE, MICROCONGLOMERATI E CONGLOMERATI POLIGENICI ED ETEROMETRICI IN MATRICE SABBIOSA
ARGILLA CON MATRICE SABBIOSA
ARGILLA DI MATRICE SABBIOSA
ARGILLA GIALLA CON GHIAIETTO IN ABBONDANTE MATRICE SABBIOSA
ARGILLA IN MATRICE SABBIOSA
ARGILLA MATRICE SABBIOSA
BRECCIA ETEROMETRICA E CIOTTOLI GNEISSICI IN MATRICE SABBIOSA FINE DEB. LIMOSA GRIGIO MARRONE E GRIGIA
BRECCIA MEDIO FINE IN MATRICE SABBIOSA DEB. LIMOSA GRIGIA
CIOTTOLI (DIAMETRO 25 - 30) E GHIAIA ETEROMETRICA IN MATRICE SABBIOSA GIAL

In [ ]:
#all values containing a key word
keyW = 'MATRICE SABBIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'SCARSA MATRICE' in key_list[position] or 'POCA MATRICE' in key_list[position] or 'SUBORDINATA MATRICE' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

ALTERNANZA DI LIMO ARGILLOSO DEBOLMENTE SABBIOSO E GHIAIA IN MATRICE SABBIOSO LIMOSA (MARRONCINI)
ALTERNANZE DI SABBIE MEDIE GHIAIOSE DEBOLMENTE LIMOSE E GHIAIE E CIOTTOLI ARROTONDATI IN MATRICE SABBIOSO LIMOSA - COLORE: NOCCIOLA
BLOCCHI (TROVANTI) IN MATRICE SABBIOSO LIMOSA
BRECCIA ETEROMETRICA (POT. MAX 30 CM) GNEISSICA IN DEB. MATRICE SABBIOSO LIMOSA
BRECCIA ETEROMETRICA E PICCOLI TROVANTI IN DEB. MATRICE SABBIOSO LIMOSA GRIGIA
BRECCIA ETEROMETRICA GNEISSICA IN MATRICE SABBIOSO LIMOSA GRIGIO MARRONE. LOC. PRES. TROVANTI GNEISSICI
BRECCIA GROSSOLANA IN MATRICE SABBIOSO LIMOSA GRIGIA. PRES. TROVANTI DI CALCARENITE GRIGIA (POT. MAX 40 CM)
CIOTTOLI CON ABBONDANTE MATRICE SABBIOSO LIMOSA
CIOTTOLI E BLOCCHI IN MATRICE SABBIOSO - LIMOSA
CIOTTOLI E GHIAIA IN MATRICE SABBIOSO-ARG.
CIOTTOLI E GHIAIA IN POCHISSIMA MATRICE SABBIOSO LIMOSA DI COLOR OCRA ACCESO. PRESENZA DI TROVANTI.
CIOTTOLI E GHIAIE A GRANULOMETRIA DA FINE A GROSSOLANA, CON MATRICE SABBIOSO LIMOSA.
CIOTTOLI GROSSI CON GHIAIA IN

In [ ]:
#all values containing a key word
#all values containing a key word
keyW = 'SABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

Output streaming troncato alle ultime 5000 righe.
SABBIA CON GHIAIA F/M (L MAX = 1.5 CM) COLOR NOCCIOLA-OLIVA.
SABBIA CON GHIAIA FILTRAZIONI
SABBIA CON GHIAIA FINE
SABBIA CON GHIAIA FINE ARGILLOSA
SABBIA CON GHIAIA FINE E CIOTTOLI SPARSI SCIOLTA (CIOTTOLI POLIGENICI CON CRISTALLINI QUARZOSI, PORFIDI E CARBONATICI)
SABBIA CON GHIAIA FINE MOLTO ADDENSATA
SABBIA CON GHIAIA FINE, LOCALI LIVELLI LIMOSI
SABBIA CON GHIAIA G MEDIAMENTE ADDENSATA GRIGIO-OLIVA.
SABBIA CON GHIAIA GHIAIETTO E CIOTTOLI
SABBIA CON GHIAIA GIALLA
SABBIA CON GHIAIA GRIGIA
SABBIA CON GHIAIA GRIGIA.
SABBIA CON GHIAIA GRIGIO CHIARA
SABBIA CON GHIAIA GROSSA
SABBIA CON GHIAIA GROSSA MISTA CON LENTI DI CONGLOMERATO
SABBIA CON GHIAIA GROSSA PARZIALMENTE CEMENTATA
SABBIA CON GHIAIA GROSSOLANA
SABBIA CON GHIAIA LEGATA
SABBIA CON GHIAIA LENTI D'ARGILLA
SABBIA CON GHIAIA LIMOSA DEB. ARGILLOSA;
SABBIA CON GHIAIA LIMOSA MARRONE-GRIGIA, DA SCIOLTA A POCO ADD.; TRA - 2.5 / 3.1 M DEB. ARGILLOSO; RARI CIOTTOLI LMAX = 12 CM;
SABBIA CON 

In [ ]:
#all values containing a key word
keyW = 'LIMO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

LIMO
LIMO (LEDA)
LIMO (PERDITA DI CIRCOLAZIONE)
LIMO (PERDITE DI CIRCOLAZIONE)
LIMO - ARGILLA
LIMO - ARGILLOSO
LIMO - ARGILLOSO CON SABBIA
LIMO - GHIAIA
LIMO - LIMO SABBIOSO CON GHIAIA FINE
LIMO - SABBIOSO
LIMO - SABBIOSO ALTERNATO A STRATI DI ARGILLA COMPATTA
LIMO - SABBIOSO CON STRATI DI SABBIA FINE
LIMO - SABBIOSO E SABBIA FINE FOSSILIFERA
LIMO - TORBA E ARGILLA SABBIOSA
LIMO A LIVELLI DEBOLMENTE SABBIOSO FINE, MODERATAMENTE CONSISTENTE, COLORE DA NOCCIOLA A GRIGIO CHIARO.
LIMO ACQUIFERO
LIMO ALTERNATO A STRATI DI ARENARIA
LIMO ARG. SABB. BRUNO CHIARO, LENTI ARENARIA
LIMO ARG. SABB. GRIGIO VERDE COMP. GHIAIA
LIMO ARG.-SABBIOSO COLOR NOCCIOLA CON NUMEROSI INCLUSI ARENACEI E CALCAREI DA MILLIMETRICI A CENTIMETRICI
LIMO ARG.SO GRIGIO CON POCA GHIAIA
LIMO ARGILL. GIALLASTRO CON CALCINELLI
LIMO ARGILL. MARRONE PASSANTE GR. AZZ. CON LIV. SABB. E PUNTI TORB.
LIMO ARGILL. NERASTRO CON ABB. GHIAIA
LIMO ARGILLA
LIMO ARGILLA CON GHIAIA
LIMO ARGILLO-SABBIOSO
LIMO ARGILLO-SABBIOSO DEB. GHIAIOSO


In [ ]:
#all values containing a key word
keyW = 'ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

ARGILLA
ARGILLA "SABBIA COMPATTA"
ARGILLA (4 MT DI PALEOSUOLO) OCRACEA (WURM) / LIMO ARGILLOSO
ARGILLA (CON SABBIA <5%)
ARGILLA (IMPASTO)
ARGILLA (MOLTO CONSOLIDATA)
ARGILLA (PICCOLI STRATI DI SABBIA)
ARGILLA (PREESISTENTE)
ARGILLA (SACCA)
ARGILLA (SUBSTRATO)
ARGILLA + CIOTOLI
ARGILLA + GHIAIA
ARGILLA + GHIAIETTO
ARGILLA + MARNA
ARGILLA + POCA GHIAIA
ARGILLA + SABBIA
ARGILLA + SABBIA FINE
ARGILLA + SABBIA GROSSOLANA
ARGILLA + SABBIA TORBOSA
ARGILLA + TORBA
ARGILLA +/- LIMOSA GIALLASTRA CON FREQUENTI FRAMMENTI DI LATERIZI E CIOTTOLI DI CONSISTENZA DECRESCENTE (RIPORTO)
ARGILLA - ( SPESS. > 9.0 M)
ARGILLA - ARGILLA LIMOSA
ARGILLA - ARGILLA LIMOSA COLOR BLU
ARGILLA - ARGILLA LIMOSA DI COLORE GRIGIO
ARGILLA - ARGILLA LIMOSA NOCCIOLA.
ARGILLA - ARGILLA LIMOSA.
ARGILLA - COMPATTA
ARGILLA - GHIAIA
ARGILLA - GHIAIA PICCOLA
ARGILLA - LIMO
ARGILLA - LIMO SABBIOSO
ARGILLA - LIMOSA
ARGILLA - LIMOSA CON CIOTTOLI
ARGILLA - SABBIA
ARGILLA - SABBIA - GHIAIA
ARGILLA - SABBIA - GHIAIETTO
ARGILLA - SABBI

In [ ]:
#all values containing a key word
keyW = 'TERRENO DI RIPORTO ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

In [ ]:
#all values containing a key word
keyW = 'SUBSTRATO ALTERATO , ARGILLE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

In [ ]:
#all values containing a key word
keyW = 'ARG.'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

ARG.
ARG. + GHIAIA GIALLA
ARG. AZZURRA C / CONCHIGLIE
ARG. BLU
ARG. BLU COMP. C/ LIMO E LIGNITE
ARG. BLU SABBIOSA
ARG. CENERE, TORBA E FOSS.
ARG. COMP. DEB. SABB. CON RARO GH.TTO SENAPE
ARG. COMPATTA
ARG. COMPATTA VARI COLORI
ARG. CON SOST. ORGANICA
ARG. E SABBIA
ARG. E SABBIA C / CONCHIGLIE
ARG. GIALLA
ARG. GIALLA + CIOTTOLI
ARG. GIALLA CON FRAMMENTI DI ARENAR.
ARG. GIALLA E GHIAIE
ARG. GIALLA MISTA A SABBIA
ARG. GIALLA ROSS., GHIAIA, CIOTT., CONGL., AREN.
ARG. GIALLASTRA CON POCA SABBIA
ARG. GRIGIA
ARG. GRIGIA COMPATTA
ARG. GRIGIA CON LIMO ARGILLOSO GRIGIO
ARG. GRIGIA E GIALLA
ARG. GRIGIA IN MATR. LIMOSA CON GHIAIA
ARG. GRIGIA IN MATR. LIMOSA CON POCA GH.
ARG. GRIGIA LIM. CON GHIAIA COMPRESSA
ARG. GRIGIA PLASTICA
ARG. GRIGIA SABB. CON CONCHIGLIE (GASTEROPODI)
ARG. GRIGIO - AZZURRA
ARG. GRIGIO MARRONE CON POCA GHIAIA
ARG. GRIGIO-SCURA, TRACCE D'AREN.
ARG. LIEVEMENTE SABBIOSA DI CONS. COMPATTISSIMA, BRUNO SCURO, CON ABBONDANTI INCLUSI LAPIDEI DI DIM. DA MILLIMETR. A CENTIMETR. DA 19.1 

In [ ]:
#all values containing a key word
keyW = 'SERPENTINOSCISTO ALTERATO ARGILLIFICATO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

In [ ]:
#all values containing a key word
keyW = 'RIPORTO DI ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

In [ ]:
#all values containing a key word
keyW = 'ALTERNANZE DI ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

ALTERNANZE DI ARGILLA CON PICCOLI STRATI DI GHIAIA COMPATTA LIMOSA
ALTERNANZE DI ARGILLA CON RARA GHIAIA E DI SABBIA LIMOSA BRUNO ROSSICCIO SCURA 5YR 3/3 E GRIGIO OLIVA 54Y 4/2; ABBONDANTI FRUSTOLI VEGETALI
ALTERNANZE DI ARGILLA DEBOLMENTE LIMOSA E LIMO ARGILLOSO, GRIGI, CON PICCOLI NODULI CALCAREI.
ALTERNANZE DI ARGILLA E GHIAIA
ALTERNANZE DI ARGILLA E GHIAIA ARGILLOSA.
ALTERNANZE DI ARGILLA E GHIAIA CON LIMO
ALTERNANZE DI ARGILLA E LIMO
ALTERNANZE DI ARGILLA E LIMO GRIGI
ALTERNANZE DI ARGILLA E LIMO SABBIOSO
ALTERNANZE DI ARGILLA E LIMO SABBIOSO ADDENSATI BRUNO ARANCIO, BRUNO OLIVA E OLIVA
ALTERNANZE DI ARGILLA E SABBIA
ALTERNANZE DI ARGILLA E SABBIA GRIGIA E MARRONE
ALTERNANZE DI ARGILLA E SABBIE
ALTERNANZE DI ARGILLA E SASSI CON GHIAIE ARGILLOSE
ALTERNANZE DI ARGILLA GRIGIA E SABBIA LIMOSA
ALTERNANZE DI ARGILLA GRIGIA PLASTICA E COMPATTA TALORA CON PRESENZA DI SCAGLIETTE DURE LIMOSE
ALTERNANZE DI ARGILLA GRIGIA-NERO-PLASTICA
ALTERNANZE DI ARGILLA GRIGIO SCURA E SABBIA DA FINE A MED

In [ ]:
#all values containing a key word
keyW = 'ALTERNANZA DI ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

ALTERNANZA DI ARGILLA COMPATTA CON GHIAIA E CIOTTOLI
ALTERNANZA DI ARGILLA CON PICCOLE LENTI DI SABBIA MOLTO FINE E LIMO
ALTERNANZA DI ARGILLA E GHIAIA
ALTERNANZA DI ARGILLA E LIMI SABBIOSI
ALTERNANZA DI ARGILLA E LIMO
ALTERNANZA DI ARGILLA E SABBIA
ALTERNANZA DI ARGILLA E SABBIA ED ARGILLA
ALTERNANZA DI ARGILLA E SABBIA-ARGILLA
ALTERNANZA DI ARGILLA GRIGIA CON TORBA, LIVELLI DI TORBA E ARGILLA GRIGIO VERDE
ALTERNANZA DI ARGILLA GRIGIA E GRIGIO SCURA COMPATTA
ALTERNANZA DI ARGILLA LIMOSA DEB. SABBIOSA E SABBIA FINE LIMOSA
ALTERNANZA DI ARGILLA LIMOSA DEBOLMENTE SABBIOSA GRIGIA E MARRONCINA E SABBIA FINE LIMOSA MARRONCINA
ALTERNANZA DI ARGILLA LIMOSA E LIMO ARGILLOSO (PREVALENZA LIMO GIALLASTRO) CONSISTENTE
ALTERNANZA DI ARGILLA LIMOSA E LIMO ARGILLOSO GRIGI-GIALLASTRI CON FRAMMENTI DI LATERIZI E LITOIDI Ø MAX 3 ÷ 4 CM.
ALTERNANZA DI ARGILLA LIMOSA E LIMO LIMO SABBIOSO GRIGI-GIALLASTRI DA MODERATAMENTE CONSISTENTI A CONSISTENTI
ALTERNANZA DI ARGILLA PLASTICA E ARGILLA SABBIOSA COMPATTA 

In [ ]:
#all values containing a key word
keyW = 'LIMI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

LIMI
LIMI - LIMI SABBIOSI DI COLORE DA OCRA A GRIGI. SALTUARI LIVELLI DI SABBIE E GHIAIE
LIMI - SABBIOSI
LIMI ARGILLO-SABBIOSI CON GHIAIA
LIMI ARGILLOSI
LIMI ARGILLOSI BLUASTRI
LIMI ARGILLOSI COMPATTI E POCO PLASTICI
LIMI ARGILLOSI COMPATTI INGLOBANTI GHIAIE E CIOTTOLI
LIMI ARGILLOSI COMPATTI OCRA
LIMI ARGILLOSI CON GHIAIETTO
LIMI ARGILLOSI CON INTERCALATI STRATI LIMOSO SABBIOSI MEDIO FINI RISCONTRABILI TRA -4 E -6 M P.C. E ORIZZONTI ARGILLOSI - ARGILLOSO LIMOSI
LIMI ARGILLOSI CON LIVELLI TORBOSI
LIMI ARGILLOSI DEBOLMENTE GHIAIOSI DEBOLMENTE SABBIOSI
LIMI ARGILLOSI DEBOLMENTE SABBIOSI
LIMI ARGILLOSI DEBOLMENTE SABBIOSI NOCCIOLA
LIMI ARGILLOSI DI COLORE GIALLO
LIMI ARGILLOSI DI COLORE MARRONE - BRUNASTRO
LIMI ARGILLOSI DI COLORE ROSSASTRO
LIMI ARGILLOSI E ARGILLE
LIMI ARGILLOSI E LIMI SABBIOSI
LIMI ARGILLOSI E LIVELLI DI ARGILLA
LIMI ARGILLOSI ED ARGILLE
LIMI ARGILLOSI ED ARGILLE LIMOSE, MOLTO CONSISTENTI E PLASTICI, DI COLORE GIALLASTRO
LIMI ARGILLOSI FOSSILIFERI, DI COLORE GRIGIO SCUR

In [ ]:
#all values containing a key word
keyW = 'LIMO,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

LIMO, A LUOGHI, DEBOLMENTE GHIAIOSO MOLTO CONSISTENTE. ASCIUTTO, DI COLORE GRIGIO. VALORI DI POCKET E TORVANE FUORI SCALA.
LIMO, ARGILLA E GHIAIA
LIMO, ARGILLA FINISSIMA
LIMO, ARGILLA SABBIOSA, CONCHIGLIE
LIMO, ARGILLA SARTUMOSA E SABBIA FINE CON TORBA
LIMO, GHIAIA MEDIA CON SABBIA GROSSA
LIMO, LIMO SABBIO - ARGILLOSO GRIGIO SCURO
LIMO, SABBIA - CONCHIGLIE E GHIAIA
LIMO, SABBIA E GHIAIETTO
LIMO, SABBIA FINE
LIMO, SABBIA ROSSA
LIMO, SABBIA, GHIAIA ARGILLOSA CON CIOTTOLI
LIMO, TRACCE DI ARGILLA, TORBA E GASTEROPODI
LIMO, TRACCE DI TORBA, ARGILLA COMP., GHIAIETTO


In [ ]:
#all values containing a key word
keyW = 'SILT'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

SILT ARGILLOSO
SILT ARGILLOSO CENERE SCURO
SILT ARGILLOSO GIALLO
SILT ARGILLOSO GRIGIO
SILT ARGILLOSO GRIGIO MISTO A GHIAIA
SILT ARGILLOSO GRIGIO SCURO CON RESTI FOSSILIFERI ED ELEMENTI DI GHIAIA FINE
SILT ARGILLOSO SABBIOSO
SILT ARGILLOSO VERDE
SILT ARGILOSO GRIGIO
SILT SABBIOSO
SILT SABBIOSO GIALLO


In [ ]:
#all values containing a key word
keyW = 'ARGILLA,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

ARGILLA,
ARGILLA, A TRATTI LIMO SABBIOSA
ARGILLA, A TRATTI LIMOSA O CON CIOTTOLI
ARGILLA, ALCUNI CIOTTOLI
ARGILLA, ARGILLA LIMOSA
ARGILLA, ARGILLA LIMOSA CON INTERCALAZIONI SABBIOSE E LIVELLI DI LIGNITE
ARGILLA, ARGILLA SABBIOSA E ARGILLA LIMOSA DI COLORE DA GIALLASTRO A NOCCIOLA IN FITTE ALTERNANZE, GENERALMENTE PLASTICHE E COMPATTE, PRIVE DI SCHELETRO CLASTICO GROSSOLANO ECCETTO SPORADICO GHIAIETTO SPARSO DI DIAMETRO INFERIORE AL CM. CONDUCIBILITA' IDRAULICA MOLTO BASSA.
ARGILLA, CALCARE, E FOSSILI
ARGILLA, CIOTTOLI E GHIAIA
ARGILLA, CIOTTOLI, GHIAIA
ARGILLA, CIOTTOLI, SABBIA
ARGILLA, CON ALTERNANZE DA PLASTICA A COMPATTA, GRIGIA
ARGILLA, CON CIOTTOLI
ARGILLA, CON CIOTTOLI, E GHIAIETTO
ARGILLA, CON GHIAIETTO
ARGILLA, CON POCO GHIAIETTO
ARGILLA, CON RARO GHIAIETTO
ARGILLA, DI CONSISTENZA COMPATTISSIMA, GRIGIA, CON RESIDUI DI ARGILLA SCAGLIOSA E NUMEROSI INCLUSI CALCAREI E IN SUBORDINE ARENACEI, DI DIMENSIONI DA MILLIMETRICHE A CENTIMETRICHE
ARGILLA, E, GHIAIETTO
ARGILLA, E, RARO GHIAI

In [ ]:
#all values containing a key word
keyW = 'ARGILLE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

ARGILLE
ARGILLE - ARGILLE LIMOSE A DISCRETO GRADO DI CONSISTENZA.
ARGILLE - ARGILLE LIMOSE CONTENENTI SUBORDINATE E DISCONTINUE SOTTILI LENTI SABBIOSE LIMOSE
ARGILLE ADDENSATE, COMPATTE ED OSSIDATE, COLORE BRUNO-OCRA;
ARGILLE ALTERNATE GRIGIO CHIARE GRIGIO SCURE
ARGILLE ARGILLE-MARNOSE
ARGILLE AZZURRE
ARGILLE BIANCHE
ARGILLE BLU
ARGILLE BLUASTRE
ARGILLE BLUASTRE CON CALCINELLI CALCAREI BIANCASTRI
ARGILLE BRUNE E GIALLASTRE COMPATTE E PLASTICHE
ARGILLE BRUNE E GIALLASTRE MEDIAMENTE COMPATTE E PLASTICHE
ARGILLE BRUNO GIALLASTRE
ARGILLE CHIARE CON TORBA
ARGILLE CINEREE
ARGILLE CINEREE COMPATTE
ARGILLE COERENTI OCRACEE CON CIOTTOLI ALTERATI E GHIAIA
ARGILLE COLOR NOCCIOLA
ARGILLE COLORE GRIGIO COMPATTE
ARGILLE COLORI DIVERSI
ARGILLE COLORI VARI CON TROVANTI TORBA
ARGILLE COMPATTE
ARGILLE COMPATTE BLU
ARGILLE COMPATTE CENERINO
ARGILLE COMPATTE CON CIOTTOLI
ARGILLE COMPATTE CON SABBIE FINI
ARGILLE COMPATTE CON SOTTILI LIVELLI CONGLOMERATI
ARGILLE COMPATTE CON SOTTILI LIVELLI CONGLOMERATICI
A

In [ ]:
#all values containing a key word
keyW = 'ARGILLITE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

ARGILLITE
ARGILLITE (CON ACQUA)
ARGILLITE BRUNA. PRES. INTERLIVELLETTI DI CALCARE
ARGILLITE CHIARA
ARGILLITE COMPATTA GRIGIASTRA CON VENE DI CALCITE. PRES. LIVELLETTI DI BRECCIA FINE E INTERCALAZIONI DI CALCARE GRIGIO CHIARO FRATTURATO CON PIANI DI FRATTURA INCL. 40° ÷ 45° LOC. RIEMPITI DA CALCITE SECONDARIA
ARGILLITE CON SASSI
ARGILLITE FESSURATA
ARGILLITE GRIGIA
ARGILLITE GRIGIA E GRIGIO SCURA COMPATTA
ARGILLITE GRIGIA E GRIGIO SCURA COMPATTA CONTENENTE OCCASIONALI CLASTI CALCAREI
ARGILLITE GRIGIA SABBIOSA
ARGILLITE GRIGIASTRA CON BRECCIA VARIA PEZZATURA
ARGILLITE GRIGIASTRA CON INTERCALAZIONI DI CALCARE (POT. MAX 20 CM). LOC. PRESENZA LIVELLO DI BRECCIA CALCAREA
ARGILLITE GRIGIASTRA CON INTERLIVELLI DI CALCARE E CALCARE MARNOSO. PIANI DI STRATIFICAZIONE DA DEB. INCLINATI A SUB-VERTICALI. PRES. RARI RIEMPIMENTI DI CALCITE SECONDARIA LUNGO FRATTURE
ARGILLITE GRIGIASTRA CON VENE DI CALCITE SECONDARIA. PIANI DI FRATTURA INCL. 45°
ARGILLITE GRIGIASTRA DEB. FRATTURATA CON PIANI DI FRATTUR

In [ ]:
#all values containing a key word
keyW = 'ARGILLITI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

ARGILLITI ALTERNATE A CALCARI MARNOSI DI COLORE NERO - ARGILLITI DI RIVA DI SOLTO. PICCOLE VENATURE DI ACQUA OLIGOMINERALE NEI LIVELLI CALCAREI (NON LOCALIZZABILI)
ARGILLITI COMPATTE CON POCA ROCCIA
ARGILLITI CON ROCCIA
ARGILLITI DI COLORE GRIGIO ALTERNATI A LIVELLI DI SPESSORE MOLTO VARIO DI ARENARIE E CALCARI DI COLORE NOCCIOLA E GRIGIO CHIARO.
ARGILLITI DI COLORE GRIGIO CON INTERCALAZIONI LIVELLI ARENACEI DI COLORE GRIGIO.
ARGILLITI E MARNE
ARGILLITI E SILTITI
ARGILLITI GRIGIE CON INTERCALAZIONI DI SOTTILI LIVELLI DI ARENARIA E CALCARE GRIGIO CHIARO.
ARGILLITI GRIGIE CON LIVELLI DI ROCCIA
ARGILLITI NERE CON LIVELLI CALCAREI. FORMAZIONE ARGILLITI DI RIVA DI SOLTO


In [ ]:
#all values containing a key word
keyW = 'ARENARIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

ARENARIA
ARENARIA (ACQUIFERO PRODUTTIVO)
ARENARIA (FORMAZIONE DELLE ARENARIE DI SERRAVALLE)
ARENARIA (FORMAZIONE DELLE MARNE DI S.AGATA FOSSILI)
ARENARIA (FORMAZIONE DELLE MARNE SI S.AGATA FOSSILI)
ARENARIA (PERDITE)
ARENARIA A GRANA FINA COLOR GRIGIO E GRIGIO CHIARO, FRATTURATA.
ARENARIA A GRANA FINA, SILTOSO MARNOSA COLOR GRIGIO SCURO, GIUNTI LISCI CON INCLINAZIONE I = 34°. GIUNTO DI STRATO LISCIO I = 30°.
ARENARIA A GRANA FINE
ARENARIA A GRANA FINE BEN CEMENTATA
ARENARIA A GRANA FINE BEN CEMENTATA.
ARENARIA A GRANA MEDIA DI COLORE GRIGIO-NOCCIOLA IN FRAMMENTI
ARENARIA A GRANA MEDIA DI COLORE GRIGIO-NOCCIOLA, COMPATTA, IN FRAMMENTI (L>10 CM)
ARENARIA A GRANA MEDIA E MEDIO FINA COLOR GRIGIO VERDASTRO; VENE CALCITICHE.
ARENARIA A GRANA MEDIA FINA COLOR GRIGIO VERDASTRO, BEN CEMENTATA, CON INCLUSI ELEMENTI GHIAIOSI MEDIO FINI.
ARENARIA A GRANA MEDIA, COMPATTA, DI COLORE GRIGIO-NOCCIOLA CON ALCUNE SUPERFICI DI ALTERAZIONE
ARENARIA A GRANA MEDIA, COMPATTA, DI COLORE GRIGIO-NOCCIOLA, IN FR

In [ ]:
#all values containing a key word
keyW = 'GNEISS'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

GNEISS ALTERATO E DEGRADATO IN SABBIA ARGILLOSA GIALLASTRA. RICONOSCIBILE LA STRUTTURA ORIGINARIA
GNEISS BIOTITICI CON FRATTURAZIONE AVENTE SPAZIATURA METRICA O LOCALMENTE CENTIMETRICA. / GNEISS BIOTITICI IDEM C.S. / I GIUNTI DI SCISTOSITA' SONO SUBORDINATI CON SPAZIATURA PLURIMETRICA, INCLINATI 45 - 50 GRADI, CON SUPERFICI PIANE-ONDULATE, FRESCHE, APERTE (1 - 2 MM), CON RIEMPIMENTO MILONITICO E SONO OSSERVABILI ALLE PROFONDITA': 87.85; 96.55; 104.20; 122.80 E 122.90. / NEGLI INTERVALLI: 79.65 - 113.80; 115.15 - 116.50; 117.00 - 121.70, I GIUNTI DI FRATTURA HANNO SPAZIATURA METRICA O LOCALM. DECIMETRICA, INCL. PIU' FREQUENTE 60-70 GRADI, ONDULATI - IRREGOLARI, LISCI O SCABRI, SERRATI O APERTI, NON OSSIDATI E CON RIEMPIMENTO COSTITUITO DA MILONITI E RARAMENTE MINERALI SECONDARI (QUARZO); LE PROFONDITA' DELLE FRATTURE SONO: 80.40, 83.70, 84.00, 84.45, 91.10, 91.12, 91.22, 91.75, 94.65, 97.60, 99.50, 99.55, 101.80, 102.15, 102.30, 102.90, 103.40, 105.00, 109.30 E 111.70. / NEGLI INTERVALL

In [ ]:
#all values containing a key word
keyW = 'GRANITO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

GRANITO COMPATTO


In [ ]:
#all values containing a key word
keyW = 'KINZIGITE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

In [ ]:
#all values containing a key word
keyW = 'CIOTTOLI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

CIOTTOLI
CIOTTOLI (40%) E SABBIA GROSSOLANA. ALCUNI TROVANTI DI NATURA METAMORFICA CON DIAMETRO < 40 CM
CIOTTOLI (50%) CON GHIAIA MEDIO FINE (30%) E SABBIA GROSSOLANA (20%). RARI BLOCCHI DI ORTOGNEISS CON DIAMETRO < 50 CM. POCHISSIMO SUPPORTO LIMOSO.
CIOTTOLI (60%) CON GHIAIE MEDIO FINI (30%) E SABBIA MEDIO GROSSOLANA. RARI TROVANTI DI ORTOGNEISS DI DIAMETRO < DI 50 CM
CIOTTOLI (80%)
CIOTTOLI (> 10 CM) E POCA SABBIA GROSSA
CIOTTOLI (ASSORBIMENTO)
CIOTTOLI (DIAM. 12 - 15 CM) E GHIAIA SABBIOSO-LIMOSA, ADDENSATA, DISCRETA OSSIDAZIONE, ALTERAZIONE QUASI ASSENTE, COLORE BRUNO SCURO - GIALLASTRO.
CIOTTOLI (DIAMETRO < 10 CM) E GHIAIA GROSSOLANA, GENERALMENTE SABBIOSA, LOCALMENTE DEBOLMENTE LIMOSA, SCIOLTA. COLORE MARRONCINO CHIARO.
CIOTTOLI (DIAMETRO >80/90) E GHIAIA GROSSA, POCO SABBIOSA LOC. LIMOSA
CIOTTOLI (DIAMETRO CA 20 CM) E GHIAIA GROSSOLANA DEBOLMENTE SABBIOSA, CON ISOLATI PICCOLI TROVANTI (DIAMETRO > 18 CM) RODINGITICI ED ECLOGIDICI.
CIOTTOLI (QUARZO E MICASCISTI) - COLORE: GRIGIO
CI

In [ ]:
keyW = 'SABBIE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIE
SABBIE (FALDA CAPTATA)
SABBIE - ARGILLE MISTE
SABBIE - SABBIE LIMOSE GRIGIE CON POCA GHIAIA
SABBIE A GRANULOMETRIA FINE
SABBIE A GRANULOMETRIA FINE, LIMOSE
SABBIE A GRANULOMETRIA MEDIA
SABBIE A GRANULOMETRIA MEDIA E FINE
SABBIE A VARIA GRANULOMETRIA
SABBIE ACQUIFERE
SABBIE ADDENSATE
SABBIE ARGILLOSE
SABBIE ARGILLOSE CON GHIAIE
SABBIE ARGILLOSE CON GHIAIETTO
SABBIE ARGILLOSE CON LIVELLI GHIAIOSI, INTERCALATE AD ARGILLE.
SABBIE ARGILLOSE GRIGIE
SABBIE ARGILLOSE LIMOSE
SABBIE BUONE GRANE
SABBIE CEMENTATE ("ARENARIE")
SABBIE COMPATTE ACQUIFERE
SABBIE COMPATTE GRIGIE CON LIMI, ARGILLE E ARGILLE PLASTICHE
SABBIE CON FALDA ACQUIFERA
SABBIE CON FRAMMENTI DI FOSSILI
SABBIE CON GHIAIA
SABBIE CON GHIAIA A GRANULOMETRIA GROSSOLANA
SABBIE CON GHIAIA A GRANULOMETRIA MEDIA
SABBIE CON GHIAIA E CIOTTOLI
SABBIE CON GHIAIA, MARRONE, UMIDO
SABBIE CON GHIAIE
SABBIE CON GHIAIE ETEROMETRICHE, DEBOLMENTE LIMOSE; CIOTTOLI FINO A DECIMETRICI; COLORE GRIGIO
SABBIE CON GHIAIE GROSSE
SABBIE CON GHIAIETTO CO

In [ ]:
keyW = 'CLASTI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

CLASTI CIOTTOLI POCA SABBIA MEDIA DEB. ARGILLOSA, AL TETTO TERRENO VEGETALE
CLASTI E INCLUSI DI CALCARE COMPATTO GRIGIO FRAMMISTO A ARGILLA GRIGIA
CLASTI E INCLUSI DI CALCARE GRIGIO COMPATTO FRAMMISTO AD ARGILLA GRIGIA
CLASTI GHIAIOSI ALTERATI IN LIMO SABBIOSO ARGILLOSO COLOR MARRONE
CLASTI MOLTO ALTERATI INGLOBATI IN LIMI SABBIOSI DEBOLMENTE ARGILLOSI COLORE MARRONE GIALLASTRO CON QUALCHE CIOTTOLO PURE ALTERATO / CONSISTENZA DA MEDIA A MODERATA


In [ ]:
keyW = 'FRAMMENTI CLASTI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

In [ ]:
#all values containing a key word
keyW = 'DOLOMIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

DOLOMIA BIANCA FRATTURATA
DOLOMIA BIANCASTRA COMPATTA. DEBOLMENTE ALTERATA DA 27,25 A 30 M DAL PIANO CAMPAGNA
DOLOMIA BIANCASTRA CON ALTERAZIONE ARGILLOSA
DOLOMIA BIANCASTRA CON PRESENZA DI ARGILLA NELLE FESSURE
DOLOMIA BIANCASTRA FRATTURATA
DOLOMIA BIANCASTRA FRATTURATA DEBOLMENTE
DOLOMIA BIANCASTRA NON FRATTURATA
DOLOMIA BIANCO ROSATA DEBOLMENTE ALTERATA
DOLOMIA CHIARA, TENERA, CON STRUTTURE DI DISSOLUZIONE VACUOLARI. VENATURE DI ARGILLE ROSSE RESIDUALI
DOLOMIA COMPATTA NON FRATTURATA
DOLOMIA GRIGIA CHIARA, MASSICCIA
DOLOMIA GRIGIA MASSICCIA CON FRATTURE CHIUSE, DISCONTINUE
DOLOMIA PIU' O MENO CALCAREA BIANCASTRA, CON ALTERAZIONE ARGILLOSA, MOLTO FRATTURATA
DOLOMIA PIU' O MENO CALCAREA BIANCASTRA, DEBOLMENTE FRATTURATA
DOLOMIA PIU' O MENO CALCAREA BIANCASTRA, FRATTURATA, ALTERATA
DOLOMIA PIU' O MENO CALCAREA, BIANCASTRA, MOLTO FRATTURATA CON PRESENZA DI ARGILLA NELLE FESSURE
DOLOMIA PIU' O MENO CALCAREA, GRIGIO CHIARA, DEBOLMENTE FRATTURATA
DOLOMIA PRINCIPALE


In [ ]:
#all values containing a key word
keyW = 'MARNA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

MARNA
MARNA (CALCAREA)
MARNA (GONFOLITE DI FONDO)
MARNA ALTERATA CON VENATURE E FRAMMENTI DI CALCITE.
MARNA ALTERATA LIMOSO-SABBIOSA NOCCIOLA CON LOCALI TRACCE DI CALCITE E FRAMMENTI ARENACEI.
MARNA ARENACEA
MARNA ARGILLOSA
MARNA ARGILLOSA A TRATTI SILTOSA GRIGIA DURA. LA PERCENTUALE DI CAROTAGGIO VARIA DA 0-10 CON CIRCOLAZIONE D'ACQUA, A 90-100 CON PERFORAZIONE A SECCO
MARNA ARGILLOSA COMPATTA CON VELI DI MATERIALE ORGANICO DI DIMENSIONI MILLIMETRICHE.
MARNA ARGILLOSA COMPATTA CON VELI DI MATERIALE ORGANICO MILLIMETRICI, CHE DANNO LAMINAZIONE PIANO-PARALLELA. PRESENTE UN LIVELLO MILLIMETRICO DI TORBA A -44.0 MT. (LIVELLI INCLINATI DI 15 GRADI).
MARNA ARGILLOSA CON PASSAGGI DI ARENARIA E PRESENZA DI FRATTURE GEN. CHIUSE
MARNA ARGILLOSA DI COLORE GRIGIO, CON INTERCALAZIONI DI SOTTILI LIVELLI (CENTIMETRICI) DI SABBIA FINE LIMOSA DI COLORE GRIGIO SCURO.
MARNA ARGILLOSA DI COLORE GRIGIO-BLUASTRO
MARNA ARGILLOSA DURA, FORMAZIONALE, CON PASSAGGI DI LIVELLI CENTIMETRICI DI SABBIA FINE, GRIGIO

In [ ]:
#all values containing a key word
keyW = 'MARNE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

MARNE
MARNE - CALCARI
MARNE / ARGILLITI
MARNE A STRATI CON CALCARE FESSURATO
MARNE ALTERATE
MARNE ARENACEE FOGLIETTATE - NOCCIOLA.
MARNE ARGILLOSE
MARNE ARGILLOSE CON ALTERNANZE DI ARENARIE SABBIOSE POCO CEMENTATE (FORMAZIONE DI CREMOLINO)
MARNE ARGILLOSE GRIGIE, COMPATTE APPARTENENTI ALLA F. DI M. LUMELLO
MARNE ARGILLOSE GRIGIO-AZZURRE
MARNE ARGILLOSE GRIGIO-CENERE
MARNE ARGILLOSE MARNE SABBIOSE CON LIVELLI ARENACEI E CONGLOMERATICI
MARNE CALCAREE
MARNE CALCAREE E MARNE ARGILLOSE
MARNE CALCAREE LOC. FRATTURATE
MARNE COMPATTE
MARNE COMPATTE CON LIVELLETTI CALCARENITICI COLOR NOCCIOLA
MARNE COMPATTE DI COLORE GRIGIO-AZZURRO; RARE INTERCALAZIONI SOTTILI MARNOSO-ARENACEE.
MARNE COMPATTE GRIGIO-AZZURRE. PRESENZA DI ORIZZONTI SCIOLTI, SABBIOSO-GHIAIOSI, DI SPESSORE MASSIMO DECIMETRICO.
MARNE COMPATTE GRIGIO-CENERE
MARNE CON ALTERNATI LIVELLI ARENACEI
MARNE CON ARENARIE IN LIVELLI CENTIMETRICI, FRATTURATI - ACQUIFERO
MARNE CON ARENARIE IN LIVELLI CENTIMETRICI, STERILI
MARNE CON CALCARE
MARNE

In [ ]:
#all values containing a key word
keyW = 'GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIA
GHIAIA (1° FALDA)
GHIAIA (2° FALDA)
GHIAIA (3° FALDA)
GHIAIA (ACQUA SULFUREA)
GHIAIA (ACQUA)
GHIAIA (ACQUIFERA)
GHIAIA (ACQUIFERO INTERCETTATO)
GHIAIA (ACQUIFERO)
GHIAIA (APPROFONDIMENTO PERFORAZIONE ANNO 1998)
GHIAIA (ASCIUTTA)
GHIAIA (ASSORBIMENTO)
GHIAIA (CAMPIONE 5)
GHIAIA (CONGLOMERATO)
GHIAIA (CONTINUA OLTRE 72)
GHIAIA (FALDA ACQUIFERA CAPTATA)
GHIAIA (FALDA ACQUIFERA)
GHIAIA (FALDA CAPTATA)
GHIAIA (FALDA)
GHIAIA (I FALDA)
GHIAIA (IV FALDA ACQ.)
GHIAIA (MARCIA) CON GHIAIETTO E ARGILLA GIALLA COMPATTA
GHIAIA (MAX 3 CM), GHIAIETTO E SABBIA LIMOSA OCRA
GHIAIA (NESSUNA FALDA)
GHIAIA (PERDITA DI CIRCOLAZIONE)
GHIAIA (PERDITE)
GHIAIA (PRIMA FALDA CEMENTATA)
GHIAIA (PRIMO LIVELLO)
GHIAIA (SANA) ROTONDA ARGILLOSA COMPATTA
GHIAIA (SECONDA FALDA)
GHIAIA (TALORA CEMENTATA)
GHIAIA + ACQUA
GHIAIA + ACQUA + SABBIA
GHIAIA + ARGILLA
GHIAIA + ARGILLA CON LENTI SABBIOSE
GHIAIA + CONGLOMERATO
GHIAIA + CONGLOMERATO + POCA ARGILLA
GHIAIA + CONGLOMERATO FESSURATO
GHIAIA + FINE
GHIAIA + FOSSILI


In [ ]:
#all values containing a key word
keyW = 'ALLUVIONE GHIAIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

In [ ]:
#all values containing a key word
keyW = 'ALTERNANZA DI GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

ALTERNANZA DI GHIAIA CON CEPPO BIANCO E PICCOLE LENTI DI CONGLOMERATO
ALTERNANZA DI GHIAIA E DI ARGILLA GRIGIA COMPATTA
ALTERNANZA DI GHIAIA E SABBIA CON STRATI DI CONGLOMERATO
ALTERNANZA DI GHIAIA SABBIOSA E LIMI SABBIOSI DEBOLMENTE ARGILLOSI


In [ ]:
#all values containing a key word
keyW = 'ALTERNANZA DI LIVELLI DI GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

ALTERNANZA DI LIVELLI DI GHIAIA CON SABBIA ARGILLOSA E ARGILLA GHIAIOSO-SABBIOSA MARRONE-GRIGIASTRI CON CIOTTOLI
ALTERNANZA DI LIVELLI DI GHIAIA CON SABBIA ARGILLOSA MARRONE E ARGILLA GHIAIOSO-SABBIOSA MARRONE CON CIOTTOLI
ALTERNANZA DI LIVELLI DI GHIAIA E SABBIA ARGILLOSA CON CIOTTOLI E ARGILLA CON LIMO GHIAIOSO-SABBIOSO MARROEN-GRIGIASTRI
ALTERNANZA DI LIVELLI DI GHIAIA E SABBIA CON CIOTTOLI E STRATERELLI DI CONGLOMERATO E ARGILLA GHIAIOSO-SABBIOSA GRIGIASTRI
ALTERNANZA DI LIVELLI DI GHIAIA E SABBIA DEBOLMENTE ARGILLOSA CON CIOTTOLI E ARGILLA GHIAIOSO-SABBIOSA MARRONE-GRIGIASTRI


In [ ]:
#all values containing a key word
keyW = 'ALTERNANZE DI GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

ALTERNANZE DI GHIAIA ARGILLOSA SABBIA LIMOSA, ARGILLA, TROVANTI (DISTRUZIONE DI NUCLEO)
ALTERNANZE DI GHIAIA CON ARGILLA E CONGLOMERATO
ALTERNANZE DI GHIAIA CON CEPPO NOCCIOLA - BIANCO E GRIGIO COMPATTO
ALTERNANZE DI GHIAIA CON LENTI ARGILLOSE GRIGIE
ALTERNANZE DI GHIAIA CON SABBIA E LIMO
ALTERNANZE DI GHIAIA E ARGILLA
ALTERNANZE DI GHIAIA E ARGILLA LIMOSA GRIGIA
ALTERNANZE DI GHIAIA E CEPPO BIANCO CON LENTI ARGILLOSE
ALTERNANZE DI GHIAIA E CIOTTOLI
ALTERNANZE DI GHIAIA E CONGLOMERATO
ALTERNANZE DI GHIAIA E CONGLOMERATO FANGOSOSTENUTO
ALTERNANZE DI GHIAIA E CONGLOMERATO FANGOSOSTENUTO BEN CEMENTATO
ALTERNANZE DI GHIAIA E SABBIA
ALTERNANZE DI GHIAIA E SABBIA (FLUVIO-GLACIALE RISSIANO)
ALTERNANZE DI GHIAIA E SABBIA ARGILLOSA
ALTERNANZE DI GHIAIA FINE E DI LIMO
ALTERNANZE DI GHIAIA FINE E SABBIA CON COMPONENTE ARGILLOSA (FLUVIO-GLACIALE RISSIANO),
ALTERNANZE DI GHIAIA GROSSA E GHIAIA FINE
ALTERNANZE DI GHIAIA SABBIOSA E ARGILLA GHIAIOSA CON RESTI VEGETALI SPARSI
ALTERNANZE DI GHIAIA SABBI

In [ ]:
#all values containing a key word
keyW = 'DEPOSITO ALLUVIONALE COSTITUITO DA GHIAIE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

DEPOSITO ALLUVIONALE COSTITUITO DA GHIAIE A GRANULOMETRIA GROSSOLANA SABBIOSE INGLOBANTI GROSSI BLOCCHI (TROVANTI)
DEPOSITO ALLUVIONALE COSTITUITO DA GHIAIE A GRANULOMETRIA GROSSOLANA, CIOTTOLOSE
DEPOSITO ALLUVIONALE COSTITUITO DA GHIAIE A GRANULOMETRIA GROSSOLANA, CIOTTOLOSE E DEBOLMENTE SABBIOSE
DEPOSITO ALLUVIONALE COSTITUITO DA GHIAIE A GRANULOMETRIA GROSSOLANA, CIOTTOLOSE, DA 1,0 - 6,0 M TALORA INGLOBANTI GROSSI BLOCCHI (TROVANTI)
DEPOSITO ALLUVIONALE COSTITUITO DA GHIAIE A GRANULOMETRIA GROSSOLANA, CIOTTOLOSE, TALORA INGLOBANTI BLOCCHI
DEPOSITO ALLUVIONALE COSTITUITO DA GHIAIE A GRANULOMETRIA MEDIA E FINE, SABBIOSE, ADDENSATE
DEPOSITO ALLUVIONALE COSTITUITO DA GHIAIE A GRANULOMETRIA MEDIA E GROSSOLANA, CIOTTOLOSE
DEPOSITO ALLUVIONALE COSTITUITO DA GHIAIE A GRANULOMETRIA MEDIA E GROSSOLANA, CIOTTOLOSE E DEBOLMENTE SABBIOSE
DEPOSITO ALLUVIONALE COSTITUITO DA GHIAIE A GRANULOMETRIA MEDIA E GROSSOLANA, SABBIOSE
DEPOSITO ALLUVIONALE COSTITUITO DA GHIAIE A GRANULOMETRIA MEDIA E GROSSOL

In [ ]:
#all values containing a key word
keyW = 'DEPOSITO ALLUVIONALE GHIAIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

DEPOSITO ALLUVIONALE GHIAIOSO
DEPOSITO ALLUVIONALE GHIAIOSO - CIOTTOLOSO
DEPOSITO ALLUVIONALE GHIAIOSO - SABBIOSO
DEPOSITO ALLUVIONALE GHIAIOSO - SABBIOSO CON PRESENZA DI ELEMENTI MOLTO GROSSOLANI
DEPOSITO ALLUVIONALE GHIAIOSO ADDENSATO INGLOBANTE TALORA GROSSI BLOCCHI (TROVANTI)
DEPOSITO ALLUVIONALE GHIAIOSO CIOTOLOSO CON OCCASIONALI INTERCALAZIONI LIMOSO SABBIOSE
DEPOSITO ALLUVIONALE GHIAIOSO CIOTTOLOSO ADDENSATO
DEPOSITO ALLUVIONALE GHIAIOSO CIOTTOLOSO INGLOBANTE BLOCCHI (TROVANTI)
DEPOSITO ALLUVIONALE GHIAIOSO CIOTTOLOSO MOLTO ADDENSATO, INGLOBANTE INTERCALAZIONI SABBIOSO LIMOSE
DEPOSITO ALLUVIONALE GHIAIOSO CIOTTOLOSO SABBIOSO INGLOBANTE BLOCCHI (FREQUENTI ELEMENTI CON DIMENSIONI > 30 CM)
DEPOSITO ALLUVIONALE GHIAIOSO CIOTTOLOSO, INGLOBANTE FREQUENTI INTERCALAZIONI SABBIOSO LIMOSE
DEPOSITO ALLUVIONALE GHIAIOSO CON ABBONDANTE MATRICE FINE LIMOSA;
DEPOSITO ALLUVIONALE GHIAIOSO CON CIOTTOLI ETEROMETRICI E POLIGENICI CON MATRICE MOLTO ABBONDANTE DI TIPO ARGILLOSO INTASANTE.
DEPOSITO A

In [ ]:
#all values containing a key word
keyW = 'DEPOSITO GHIAIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

DEPOSITO GHIAIOSO
DEPOSITO GHIAIOSO / SABBIOSO
DEPOSITO GHIAIOSO ALTERNATO A LIVELLI SABBIOSI DECIMETRICI
DEPOSITO GHIAIOSO ALTERNATO AD ARGILLA
DEPOSITO GHIAIOSO BEN CLASSATO CON INTERCALAZIONI SABBIOSE E LIVELLI LENTIFORMI LIMOSO-ARGILLOSI CON SOLUZIONI DI CONTINUITA'. PRESENZA DEL SECONDO LIVELLO ACQUIFERO.
DEPOSITO GHIAIOSO CIOTTOLOSO A GRANULOMETRIA GROSSOLANA INGLOBANTE FREQUENTI BLOCCHI (TROVANTI)
DEPOSITO GHIAIOSO CIOTTOLOSO INGLOBANTE BLOCCHI (TROVANTI)
DEPOSITO GHIAIOSO CIOTTOLOSO INGLOBANTE CON FREQUENZA GROSSI BLOCCHI (TROVANTI)
DEPOSITO GHIAIOSO CIOTTOLOSO INGLOBANTE INTERCALAZIONI SABBIOSO LIMOSE
DEPOSITO GHIAIOSO CIOTTOLOSO PARZ. CEMENTATO
DEPOSITO GHIAIOSO CIOTTOLOSO PARZIALMENTE CEMENTATO
DEPOSITO GHIAIOSO CON CIOTTOLI BEN CLASSATI CON PRESENZA DI ACQUIFERO
DEPOSITO GHIAIOSO CON CIOTTOLI BEN CLASSATI CON PRESENZA DI ACQUIFERO PRODUTTIVO
DEPOSITO GHIAIOSO CON CIOTTOLI CALCAREI DI DIMENSIONI CENTIMETRICHE (PREVALENZA CLASSE 2-4 CM CON RARI ELEMNTI 7-8 CM). PRESENZA QUART

In [ ]:
#all values containing a key word
keyW = 'LIVELLO GHIAIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

LIVELLO GHIAIOSO
LIVELLO GHIAIOSO CIOTTOLOSO PARZIALMENTE CEMENTATO
LIVELLO GHIAIOSO CIOTTOLOSO PARZIALMENTE CEMENTATO (CONGLOMERATO)
LIVELLO GHIAIOSO CON SCARSA MATRICE LIMOSO-SABBIOSA
LIVELLO GHIAIOSO IN SCARSA MATRICE LIMOSO-SABBIOSA
LIVELLO GHIAIOSO MEDIAMENTE GROSSOLANO, PER LO PIU' CONGLOMERATICO, CON ELEMENTI ARROTONDATI A DIAMETRO MAX 20-25 CM, DI COLORE GRIGIASTRO . BRUNO MARRONCINO
LIVELLO GHIAIOSO PIUTTOSTO CEMENTATO / CONGLOMERATO COMPATTO
LIVELLO GHIAIOSO PRODUTTIVO CON SABBIA CON PASSATE FRANCAMENTE ARGILLOSE


In [ ]:
#all values containing a key word
keyW = 'TERRENO DI RIPORTO PREVALENTEMENTE GHIAIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

TERRENO DI RIPORTO PREVALENTEMENTE GHIAIOSO - SABBIOSO CON CIOTTOLI


In [ ]:
#all values containing a key word
keyW = 'TERRENO DI RIPORTO GHIAIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

TERRENO DI RIPORTO GHIAIOSO - SABBIOSO - LIMOSO BRUNO
TERRENO DI RIPORTO GHIAIOSO - SABBIOSO.


In [ ]:
#all values containing a key word
keyW = 'TERRENO GHIAIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

TERRENO GHIAIOSO - SABBIOSO DEBOLMENTE LIMOSO
TERRENO GHIAIOSO - SABBIOSO DEBOLMENTE LIMOSO CON LOCALI LIVELLI A CONGLOMERATO
TERRENO GHIAIOSO ARGILLOSO GIALLOGNOLO.
TERRENO GHIAIOSO DURISSIMO
TERRENO GHIAIOSO E COMPATTO
TERRENO GHIAIOSO GRASSO MOLTO DURO


In [ ]:
#all values containing a key word
keyW = 'CIOTTOLI,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

CIOTTOLI, ARGILLA, CONGLOMERATO
CIOTTOLI, BLOCCHI E GHIAIA (ASSORBIMENTO DI FANGO)
CIOTTOLI, BLOCCHI E GHIAIA CON SABBIA
CIOTTOLI, CHIAIA E GHIAIETTO
CIOTTOLI, CIOTTOLONI CON GHIAIA E SABBIA
CIOTTOLI, CIOTTOLONI IN GHIAIA E SABBIA
CIOTTOLI, CIOTTOLONI IN MATRICE GRIGIO SABBIOSA ARGILLOSA COMPATTA
CIOTTOLI, CONGLOMERATO
CIOTTOLI, GHIAIA CON POCA SABBIA
CIOTTOLI, GHIAIA CON RARE LENTI DI ARGILLA GIALLA
CIOTTOLI, GHIAIA CON SABBIA FINE GIALLA
CIOTTOLI, GHIAIA CON SABBIA GROSSA
CIOTTOLI, GHIAIA CON SABBIA LIMOSA
CIOTTOLI, GHIAIA CON SABBIA ROSSASTRA (TRACCE D'ARGILLA GIALLASTRA NEI PRIMI DUE METRI)
CIOTTOLI, GHIAIA E ARENARIA (PRIMI 60 CM. LENTI D'ARGILLA SABBIOSA FINE)
CIOTTOLI, GHIAIA E ARGILLA
CIOTTOLI, GHIAIA E ARGILLA LIMOSA.
CIOTTOLI, GHIAIA E ARGILLA SABBIOSA GRIGIA
CIOTTOLI, GHIAIA E CONGLOMERATI CON INTERCALAZIONI ARGILLOSE-ACQUIFEO
CIOTTOLI, GHIAIA E GHIAIETTO
CIOTTOLI, GHIAIA E GHIAIETTO CON SABBIA
CIOTTOLI, GHIAIA E LIMO SABBIOSO
CIOTTOLI, GHIAIA E POCHISSIMA SABBIA
CIOTTOLI, G

In [ ]:
#all values containing a key word
keyW = 'CIOTTOLO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

CIOTTOLO CON GHIAIA E SABBIA
CIOTTOLO CON GHIAIA LEGEGRMENTE ARGILLOSA


In [ ]:
#all values containing a key word
keyW = 'GHIAIA,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

GHIAIA, ARGILLA CON CONCHIGLIE
GHIAIA, ARGILLA E CIOTTOLI DI PICCOLE DIMENSIONI;
GHIAIA, ARGILLA E CIOTTOLI DI PICCOLE DIMENSIONI; COLORE GRAY-BROWN; SPESSORE M 2,50.
GHIAIA, ARGILLA E CONGLOMERATI
GHIAIA, ARGILLA E SABBIA
GHIAIA, ARGILLA E SABBIA.
GHIAIA, ARGILLA GRIGIA COMPATTA E GHIAIETTO
GHIAIA, ARGILLA SABBIOSA GIALLA CON STRATI DI ARENARIA
GHIAIA, ARGILLA, CONGLOMERATO
GHIAIA, CIOTOLAME ED ACQUA
GHIAIA, CIOTT., LENTI DI ARG. E LIVELLI DI CONGL.
GHIAIA, CIOTTOLI
GHIAIA, CIOTTOLI > 10 CM. MATTONI ROSSASTRI TALVOLTA LEGATI CON CEMENTO
GHIAIA, CIOTTOLI ARENARIA E SABBIA
GHIAIA, CIOTTOLI CON ARGILLA
GHIAIA, CIOTTOLI CON LIVELLI CEMENTATI
GHIAIA, CIOTTOLI CON POCA SABBIA (ACQUIFERA)
GHIAIA, CIOTTOLI CON SABBIA E GHIAIETTO
GHIAIA, CIOTTOLI CON SABBIA GROSSA
GHIAIA, CIOTTOLI E ARGILLA COMPATTA
GHIAIA, CIOTTOLI E ARGILLA GRIGIA
GHIAIA, CIOTTOLI E ARGILLA MARRONE
GHIAIA, CIOTTOLI E ARGILLA ROSSASTRA
GHIAIA, CIOTTOLI E BLOCCHI CON GHIAIETTO E SABBIA MEDIO-GROSSA
GHIAIA, CIOTTOLI E BLOCCHI D

In [ ]:
#all values containing a key word
keyW = 'GHIAIE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIE
GHIAIE (FALDA ACQUIFERA)
GHIAIE + ARGILLA
GHIAIE - CIOTTOLI E TROVANTI
GHIAIE - GHIAIONE E POCA SABBIA
GHIAIE - LIMO E SABBIE
GHIAIE A GRANULOMETRIA GROSSOLANA CIOTTOLOSE, ADDENSATE, CON ABBONDANTE FRAZIONE SABBIOSA, A COLORAZIONE GIALLOGNOLA
GHIAIE A GRANULOMETRIA GROSSOLANA E MOLTO GROSSOLANA, CIOTTOLOSE E SABBIOSE, ADDENSATE
GHIAIE A GRANULOMETRIA GROSSOLANA, ADDENSATE, CON ABBONDANTE FRAZIONE SABBIOSA
GHIAIE A GRANULOMETRIA GROSSOLANA, CIOTTOLOSE
GHIAIE A GRANULOMETRIA GROSSOLANA, CIOTTOLOSE, CON ABBONDANTE FRAZIONE SABBIOSA
GHIAIE A GRANULOMETRIA GROSSOLANA, CIOTTOLOSE, CON FRAZIONE SABBIOSO LIMOSA LOCALMENTE MOLTO ABBONDANTE (18,0 - 20,0 M)
GHIAIE A GRANULOMETRIA GROSSOLANA, CIOTTOLOSE, CON SUBORDINATA FRAZIONE SABBIOSA, GIALLOGNOLA
GHIAIE A GRANULOMETRIA GROSSOLANA, DEBOLMENTE SABBIOSE, ADDENSATE
GHIAIE A GRANULOMETRIA GROSSOLANA, INGLOBANTI FREQUENTI BLOCCHI (TROVANTI)
GHIAIE A GRANULOMETRIA GROSSOLANA, SABBIOSE
GHIAIE A GRANULOMETRIA GROSSOLANA, SABBIOSE, ADDENSATE
GHIA

In [ ]:
#all values containing a key word
keyW = 'GHIAIE,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIE, CIOTTOLI DECIMETRICI BEN ARROTONDATI, SABBIE GROSSOLANE
GHIAIE, E CIOTTOLI CON SABBIE
GHIAIE, E CIOTTOLI CON SABBIE CON LENTI DI CONGLOMERATO
GHIAIE, ETEROMETRICHE, IN SCARSA MATRICE
GHIAIE, GHIAIETTI E CIOTTOLI DECIMETRICI ETEROGENEI ED ETEROMETRICI
GHIAIE, SABBIA E ARGILLA
GHIAIE, SABBIE E LIMI
GHIAIE, SABBIE GIALLASTRE
GHIAIE, SABBIE MEDIE LIMOSE CON CIOTTOLI PLURICENTIMETRICI


In [ ]:
#all values containing a key word
keyW = 'GHIAIETTO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIETTO
GHIAIETTO (0,5-1,0 CM)
GHIAIETTO (COLORE ARANCIO) SABBIA LIMO ARGILLOSI
GHIAIETTO (DIAMETRO 0,2-0,5 CM) CON SABBIA GROSSOLANA
GHIAIETTO (FALDA ACQUA)
GHIAIETTO (I FALDA ACQ.)
GHIAIETTO (PREESISTENTE)
GHIAIETTO + SABBIA
GHIAIETTO - ARGILLA
GHIAIETTO - GHIAIA
GHIAIETTO - SABBIA MEDIO GROSSA
GHIAIETTO - SABBIONE E MATERIALE VEGETALE
GHIAIETTO / ACQUA
GHIAIETTO ACQUIFERO
GHIAIETTO ADDENSATO CON STRATI DI ARGILLA
GHIAIETTO ARGILLA SABBIOSO
GHIAIETTO ARGILLOSO
GHIAIETTO ARGILLOSO COMPATTO
GHIAIETTO ARGILLOSO CON CIOTTOLI E TROVANTI
GHIAIETTO ARGILLOSO CON POCA SABBIA
GHIAIETTO ARGILLOSO SABBIOSO
GHIAIETTO ARROTONDATO E A SPIGOLO VIVO POLIGENICO DIAMETRO MAX 2 - 3 CM IN MATRICE DI SABBIA MEDIA GROSSOLANA LIMOSA.
GHIAIETTO ASCIUTTO
GHIAIETTO AZZURRO
GHIAIETTO AZZURRO CON SABBIA E ARGILLA AZZURRA
GHIAIETTO AZZURRO E SABBIA
GHIAIETTO BELLO
GHIAIETTO BIANCO
GHIAIETTO BIANCO E GIALLO
GHIAIETTO BIANCO QUARZO
GHIAIETTO BLU
GHIAIETTO CALCAREO MINUTO
GHIAIETTO CEMENTATO
GHIAIETTO CEMENTATO (

In [ ]:
#all values containing a key word
keyW = 'GHIAIONE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIONE
GHIAIONE (POCA ACQUA)
GHIAIONE (STR. D. M. 1)
GHIAIONE - GHIAIA CON POCA SABBIA
GHIAIONE - GHIAIA E SABBIA
GHIAIONE - GHIAIA E SABBIA CON ACQUA
GHIAIONE ACQUIFERO
GHIAIONE ACQUIFERO CON LENTI DI CONGLOMERATO
GHIAIONE ARGILLOSO
GHIAIONE ARGILLOSO CON TROVANTI
GHIAIONE ARGILLOSO, TROVANTI
GHIAIONE ASCIUTTO
GHIAIONE ASCIUTTO COMPATTO
GHIAIONE ASCIUTTO CON SABBIA GROSSOLANA
GHIAIONE ASCIUTTO CON TROVANTI
GHIAIONE COMPATTO
GHIAIONE COMPATTO CON TROVANTI
GHIAIONE COMPATTO CON TROVANTI DIAMETRO 500 MM
GHIAIONE CON ACQUA
GHIAIONE CON ACQUA FALDA 2°
GHIAIONE CON ACQUA FALDA 3°
GHIAIONE CON ARGILLA
GHIAIONE CON CIOTOLI
GHIAIONE CON CIOTOLI GROSSI
GHIAIONE CON CIOTTOLI
GHIAIONE CON CIOTTOLI E SABBIA
GHIAIONE CON CIOTTOLI GROSSI
GHIAIONE CON CIOTTOLI TERROSO
GHIAIONE CON CONGLOMERATO
GHIAIONE CON CROSTE DI CONGLOMERATO
GHIAIONE CON FALDA 1° (ACQUA)
GHIAIONE CON GROSSE PIETRE
GHIAIONE CON GROSSE PIETRE, TERRENO DURISSIMO
GHIAIONE CON GROSSI CIOTTOLI
GHIAIONE CON GROSSI CIOTTOLI (TROVANTI)


In [ ]:
#all values containing a key word
keyW = 'GESSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=202
    key_list[position]='Gesso, anidrite'

GESSO BIANCASTRO TENERO MISTO A MARNA ARGILLOSA GRIGIA
GESSO COMPATTO MICROCRISTALLINO IN VARIETA' DI SELENITE
GESSO GRIGIO-BIANCASTRO MOLTO DURO CON LIVELLETTI DI MARNA GRIGIA E ARENARIA
GESSO LAMELLARE GRIGIO, SEDE DI FALDA ACQUIFERA AD ALTO CONTENUTO SALINO
GESSO MICROCRISTALLINO
GESSO MICROCRISTALLINO CON LIVELLI DI CALCARI CARIATI


In [ ]:
#all values containing a key word
keyW = 'GESSI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=202
    key_list[position]='Gesso, anidrite'

GESSI ALTERNATI CON CALCARI DOLOMITICI NERI


In [ ]:
#all values containing a key word
keyW = 'RIPORTO GHIAIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

RIPORTO GHIAIOSO


In [ ]:
#all values containing a key word
keyW = 'RIPORTO DI GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

RIPORTO DI GHIAIA SCIOLTA


In [ ]:
#all values containing a key word
keyW = 'STRATI DI GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

STRATI DI GHIAIA
STRATI DI GHIAIA COMPATTA CON STRATI DI ARGILLA GIALLA COMPATTA
STRATI DI GHIAIA CON ALTERNANZA DI ARGILLA COMPATTA
STRATI DI GHIAIA CON ALTERNANZA DI ARGILLA SABBIOSA
STRATI DI GHIAIA CON ARGILLA
STRATI DI GHIAIA CON ARGILLA CON ALTERNANZE DI ARGILLA COMPATTA
STRATI DI GHIAIA CON ARGILLE CHIARE
STRATI DI GHIAIA CONGLOMERATA CON ACQUA.
STRATI DI GHIAIA E ARGILLA
STRATI DI GHIAIA E SABBIA
STRATI DI GHIAIA MISTA
STRATI DI GHIAIA MISTA TERRA + CONGLOMERATO
STRATI DI GHIAIA MISTI A STRATI DI ARGILLA
STRATI DI GHIAIA SCIOLTA, SABB. E STRATI ARG.


In [ ]:
#all values containing a key word
keyW = 'RIPORTO IN GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

RIPORTO IN GHIAIA


In [ ]:
#all values containing a key word
keyW = 'RIPORTO CON GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

RIPORTO CON GHIAIA E CEMENTO


In [ ]:
#all values containing a key word
keyW = 'TERRENO DI RIPORTO CON CIOTTOLI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',5)[0]+' '+key_list[position].split(' ',5)[1]+' '+key_list[position].split(' ',5)[2]+' '+key_list[position].split(' ',5)[3]+' '+key_list[position].split(' ',5)[4]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

TERRENO DI RIPORTO CON CIOTTOLI


In [ ]:
#all values containing a key word
keyW = 'TERRENO DI RIPORTO COSTITUITO DA GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',6)[0]+' '+key_list[position].split(' ',6)[1]+' '+key_list[position].split(' ',6)[2]+' '+key_list[position].split(' ',6)[3]+' '+key_list[position].split(' ',6)[4]+' '+key_list[position].split(' ',6)[5]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

TERRENO DI RIPORTO COSTITUITO DA GHIAIA ETEROGENEA DA SUB-ARROTONDATA A SUB-ANGOLARE SFEROIDALE/DISCOIDALE CON SABBIA G/M E CIOTTOLI (L MAX = 18 CM). NEI PRIMI 20 CM E' PRESENTE DEL CONGLOMERATO BITUMINOSO SEGUITO DA SOTTOFONDO IN "MAGRONE".
TERRENO DI RIPORTO COSTITUITO DA GHIAIA ETEROGENEA SUB-ARROTONDATA DISCOIDALE/SFEROIDALE E SABBIA MOLTO ADDENSATE CON FREQUENTI CIOTTOLI (L MAX = 30 CM) GRIGIO-BRUNE; DA 3.60 M A 4.00 M E DA 3.90 M A 4.50 M E' PRESENTE DEL LIMO IN TRACCE; NEI PRIMI 20 CM E' PRESENTE DEL CONGLOMERATO BITUMINOSO SEGUITO DA "MAGRONE".
TERRENO DI RIPORTO COSTITUITO DA GHIAIA ETEROGENEA SUBARROTONDATA DISCOIDALE/SFEROIDALE E SABBIA MOLTO ADDENSATE CON FREQUENTI CIOTTOLI (L MAX = 30 CM) GRIGIO-BRUNA; NEI PRIMI 20 CM E' PRESENTE DEL CONGLOMERATO BITUMINOSO CON SOTTOFONDO IN "MAGRONE".
TERRENO DI RIPORTO COSTITUITO DA GHIAIA ETEROGENEA SUBARROTONDATA/SUBANGOLARE SFEROIDALE/DISCOIDALE CON SABBIA E CIOTTOLI (L MAX 25 CM) MOLTO ADDENSATE, DI COLORE GRIGIO-NOCCIOLA. NEI PRIMI 

In [ ]:
keyW = 'TERRA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

TERRA
TERRA + ARGILLA
TERRA + GHIAIA
TERRA - ARGILLA
TERRA - GHIAIA
TERRA - SABBIA
TERRA / ARGILLA
TERRA / ARGILLA GIALLA
TERRA / ARGILLA GRIGIA
TERRA AGRICOLA
TERRA ARGILLA
TERRA ARGILLA ROSSA
TERRA ARGILLOSA
TERRA ARGILLOSA E GHIAIA
TERRA ARGILLOSA GIALLA
TERRA ARGILLOSA MARRONE - GRIGIA
TERRA ARGILLOSA OCRA
TERRA ARGILOSA DI RIPORTO
TERRA BLU
TERRA BLU - SABBIA - FONDO MARINO
TERRA CIOTTOLI E TROVANTI
TERRA COLTIVA
TERRA COLTIVO
TERRA COLTURA
TERRA COMPATTA
TERRA CON ARGILLA AZZ.
TERRA CON CIOTTOLI
TERRA CON GHIAIA
TERRA CON GHIAIA E CIOTTOLI
TERRA CON LEDA
TERRA CON PIETRE
TERRA CON PIETRE GRANDI
TERRA CON PIETRE.
TERRA CON SASSI
TERRA CON STRATI DI SABBIA
TERRA CON TROVANTI
TERRA COTTURA
TERRA DA COLTIVO
TERRA DI CAMPAGNA
TERRA DI COLTURA
TERRA DI COLTURA ARGILLOSA
TERRA DI PIANO CAMPAGNA
TERRA DI RIPORTO
TERRA DI RIPORTO (DEPOSITI ALLUVIONI)
TERRA DI SUPERFICIE
TERRA E ARGILLA
TERRA E ARGILLA ROSSA (ELUVIALE COPERTURA LOESSICA)
TERRA E CIOTOLI
TERRA E CIOTOLI MOLTO COMPATTO
TERRA

In [ ]:
keyW = 'TORBA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

TORBA
TORBA - ARGILLA
TORBA ALTERNANZE DI ARGILLE COMPATTE E SABBIOSE
TORBA AMORFA MEDIAMENTE COMPATTA DI COLORE BRUNO.
TORBA ARGILLA
TORBA ARGILLOSA
TORBA COLOR CENERE SCURA
TORBA CON ARGILLA
TORBA CON ARGILLA GRIGIA E CONCHIGLIE
TORBA CON ARGILLA GRIGIO SCURA
TORBA CON ARGILLA LIMOSA
TORBA CON ARGILLA NERA
TORBA CON ARGILLA SABBIOSA
TORBA CON ARGILLA SCURA
TORBA CON CIOTTOLI DIAMETRO 4 CM AL TETTO.
TORBA CON LEGNO
TORBA CON LEGNO E GHIAIA, ARGILLA GRIGIA
TORBA CON LENTI DI ARGILLA CENERE
TORBA CON LENTI DI ARGILLA GRIGIA
TORBA CON LENTI DI ARGILLA GRIGIA SABBIOSA
TORBA CON LIGNITE
TORBA CON PEZZI DI LEGNA
TORBA CON POCA GHIAIA
TORBA CON SABBIA
TORBA CON TRACCE DI ARGILLA
TORBA CON TRACCE DI ARGILLA BLU CENERE
TORBA CON TRACCE DI GHIAIA
TORBA CON TRACCE SABBIA E GHIAIETTO
TORBA CON TRACCIA DI GHIAIA
TORBA DA M. 8.5 A M. 10.
TORBA DEBOLMENTE ARGILLOSA, POCO CONSISTENTE, UMIDA, DI COLORE MARRONE SCURO-NERASTRO.
TORBA E ARGILLA
TORBA E ARGILLA CENERE SABBIOSA
TORBA E ARGILLA GRIGIA
TORBA

In [ ]:
#all values containing a key word
keyW = 'TERRENO VEGETALE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

TERRENO VEGETALE
TERRENO VEGETALE (ARGILLA SABBIOSA)
TERRENO VEGETALE (ORIZZONTE PRESUNTO) / APPROFONDIMENTO POZZO PREESISTENTE
TERRENO VEGETALE (SUOLO AGRARIO)
TERRENO VEGETALE -
TERRENO VEGETALE - ARGILLA
TERRENO VEGETALE - GHIAIA
TERRENO VEGETALE - TERRENO AGRARIO E LIMI ARGILLOSI
TERRENO VEGETALE / ARG. SABBIOSA
TERRENO VEGETALE / ARGILLOSO / RIPORTO.
TERRENO VEGETALE / GHIAIA E SABBIA
TERRENO VEGETALE / LIMO ARGILLOSO
TERRENO VEGETALE / RIPORTO
TERRENO VEGETALE AGRARIO. COLORE MARRONE.
TERRENO VEGETALE AGRICOLO
TERRENO VEGETALE ARGILLA
TERRENO VEGETALE ARGILLO-LIMOSO CON RESTI VEGETALI E FRAMMENTI DI LATERIZI
TERRENO VEGETALE ARGILLO-LIMOSO NOCCIOLA CHIARO POCO PLASTICO MOLTO COMPATTO
TERRENO VEGETALE ARGILLOSO
TERRENO VEGETALE ARGILLOSO E SABBIOSO
TERRENO VEGETALE ARGILLOSO GIALLASTRO
TERRENO VEGETALE ARGILLOSO GIALLO-OCRA
TERRENO VEGETALE ARGILLOSO SABBIOSO GIALLASTRO
TERRENO VEGETALE BRUNO
TERRENO VEGETALE C/PRESENZA DI POCA GHIAIA
TERRENO VEGETALE CIOTTOLI E TROVANTI
TERRENO V

In [ ]:
#all values containing a key word
keyW = 'TERRENO SABBIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

TERRENO SABBIOSO
TERRENO SABBIOSO BLUASTRO CON LENTI DI ARGILLA.
TERRENO SABBIOSO DI CAMPAGNA
TERRENO SABBIOSO DI COLORE BLUASTRO
TERRENO SABBIOSO GIALLASTRO CON ALCUNI PICCOLI CIOTTOLI.


In [ ]:
#all values containing a key word
keyW = 'TERRENO ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

TERRENO ARGILLOSO
TERRENO ARGILLOSO - SUOLO AGRARIO
TERRENO ARGILLOSO / ARGILLA ROSSA
TERRENO ARGILLOSO BLUASTRO E NON DI UNA CERTA CONSISTENZA
TERRENO ARGILLOSO COLTIVO
TERRENO ARGILLOSO COMPATTO
TERRENO ARGILLOSO COMPATTO CON VENATURE NERASTRE
TERRENO ARGILLOSO COMPATTO.
TERRENO ARGILLOSO CON CIOTTOLI E GHIAIA
TERRENO ARGILLOSO CON GHIAIA
TERRENO ARGILLOSO CON PIETRE
TERRENO ARGILLOSO CON RESTI VEGETALI
TERRENO ARGILLOSO CON SABBIA
TERRENO ARGILLOSO CON TROVANTI
TERRENO ARGILLOSO DI COLORE CHIARO
TERRENO ARGILLOSO DI MEDIA CONSISTENZA.
TERRENO ARGILLOSO DI RIPORTO
TERRENO ARGILLOSO E ARGILLA
TERRENO ARGILLOSO GHIAIA
TERRENO ARGILLOSO GIALLASTRO
TERRENO ARGILLOSO GIALLO
TERRENO ARGILLOSO GRIGIO
TERRENO ARGILLOSO MARRONE
TERRENO ARGILLOSO NERASTRO MOLTO DURO
TERRENO ARGILLOSO NON COMPATTO
TERRENO ARGILLOSO ORGANICO CON RESTI VEGETALI
TERRENO ARGILLOSO PLASTICO
TERRENO ARGILLOSO ROSSASTRO
TERRENO ARGILLOSO ROSSICCIO
TERRENO ARGILLOSO SABBIOSO
TERRENO ARGILLOSO VEGETALE


In [ ]:
#all values containing a key word
keyW = 'TERRENO ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

In [ ]:
#all values containing a key word
keyW = 'TERRENO LIMOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

TERRENO LIMOSO
TERRENO LIMOSO - ARGILLOSO GIALLASTRO
TERRENO LIMOSO / ARGILLOSO CON GHIAIETTO
TERRENO LIMOSO ARGILLOSO - ROSSASTRO
TERRENO LIMOSO ARGILLOSO GIALLASTRO
TERRENO LIMOSO ARGILLOSO ROSSASTRO


In [ ]:
#all values containing a key word
keyW = 'SUBSTRATO ROCCIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

SUBSTRATO ROCCIOSO
SUBSTRATO ROCCIOSO (GNEISS E MICASCISTI) MOLTO ALTERATO E FRATTURATO
SUBSTRATO ROCCIOSO (GNEISS)
SUBSTRATO ROCCIOSO (GONFOLITE)
SUBSTRATO ROCCIOSO (MICASCISTI) ALTERATO E FRATTURATO
SUBSTRATO ROCCIOSO (MICASCISTO) MOLTO FRATTURATO
SUBSTRATO ROCCIOSO / BASAMENTO ROCCIOSO GRANITICO ALTERATO
SUBSTRATO ROCCIOSO AD ARENARIE GRIGIE COMPATTE
SUBSTRATO ROCCIOSO ALTERATO FRATTURATO
SUBSTRATO ROCCIOSO COMPATTO
SUBSTRATO ROCCIOSO COSTITUITO DA QUARZITI E SCISTI
SUBSTRATO ROCCIOSO DELLA FORMAZIONE DI CREMOLINO, DI ETA' MIOCENICA INFERIORE, IN FACIES DI ARENARIA MARRONASTRA ROSSICCIA, MOLTO ALTERATA ED ASSAI POCO COMPATTA
SUBSTRATO ROCCIOSO DI COLORE GRIGIO NERASTRO.
SUBSTRATO ROCCIOSO FILLADICO COMPATTO
SUBSTRATO ROCCIOSO FILLADICO COMPATTO, SENZA ACQUA E FRATTURE
SUBSTRATO ROCCIOSO FILLADICO FRATTURATO ED ALTERATO, CON FRATTURE SATURE E/O CON RIEMPIMENTO DI MATERIALE FINE
SUBSTRATO ROCCIOSO FRATTURATO
SUBSTRATO ROCCIOSO FRATTURATO (MICASCISTI)
SUBSTRATO ROCCIOSO FRATTURATO / BA

In [ ]:
#all values containing a key word
keyW = 'SABBIA,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIA, ARENARIA DURA CON CONCHIGLIE
SABBIA, ARENARIA E FOSSILI
SABBIA, ARGILLA E GHIAIA
SABBIA, ARGILLA E GHIAIA;
SABBIA, ARGILLA GIALLA (SABBIA FINE)
SABBIA, ARGILLA SABBIOSA
SABBIA, CIOTTOLI CON STRATI DI CONGLOMERATO
SABBIA, CONCHIGLIE E PICCOLI STRATI GHIAIETTO
SABBIA, FONDO POZZO A 85 METRI
SABBIA, GHIAIA ASCIUTTA
SABBIA, GHIAIA CIOTTOLI CON STRATI DI CONGLOMERATO
SABBIA, GHIAIA CON ACQUA
SABBIA, GHIAIA CON CIOTTOLI E BLOCCHI
SABBIA, GHIAIA CON GHIAIETTO E ARG. MARRONE
SABBIA, GHIAIA CON QUALCHE CIOTTOLO
SABBIA, GHIAIA DI SEZIONE PICCOLA E FALDA ACQUIFERA
SABBIA, GHIAIA E ARGILLA
SABBIA, GHIAIA E ARGILLA GIALLA
SABBIA, GHIAIA E CIOTTOLI
SABBIA, GHIAIA E CIOTTOLI - GRIGIO
SABBIA, GHIAIA E CIOTTOLI COLOR NOCCIOLA
SABBIA, GHIAIA E CIOTTOLI CON ACQUA
SABBIA, GHIAIA E CIOTTOLI CON ARGILLA
SABBIA, GHIAIA E CIOTTOLI LEGGERMENTE ARGILLOSI
SABBIA, GHIAIA E CIOTTOLI SPORCHI (FIL.)
SABBIA, GHIAIA E CIOTTOLI.
SABBIA, GHIAIA E GHIAIETTO
SABBIA, GHIAIA E GHIAIETTO CON CIOTTOLI
SABBIA, GHIAIA E

In [ ]:
#all values containing a key word
keyW = 'SABBIE,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIE, INGLOBANTI GHIAIE MINUTE
SABBIE, SABBIE CON INTERCALAZIONI DI LENTI LIMOSE E/O LENTI DI SABBIE GROSSOLANE
SABBIE, SABBIE GROSSOLANE
SABBIE, SABBIE LIMOSE


In [ ]:
#all values containing a key word
keyW = 'SABBIONE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIONE
SABBIONE - ARGILLA
SABBIONE - GHIAIA
SABBIONE - ZONA FILTRI
SABBIONE ALTERNATO CON GHIAIETTO E SABBIA FINE
SABBIONE ARGILLOSO
SABBIONE ARGILLOSO COMPATTO
SABBIONE ARGILLOSO CON CIOTTOLI
SABBIONE ARGILLOSO ROSSASTRO
SABBIONE AZZURRO
SABBIONE AZZURRO CON GHIAIETTO E GHIAIA
SABBIONE BLU
SABBIONE BLU (FALTRAMOSO?) LIMOSO
SABBIONE BLU CON CONCHIGLIE
SABBIONE CHIARO
SABBIONE COMPATTO
SABBIONE COMPATTO CON GHIAIETTO
SABBIONE COMPATTO CON PIETRA
SABBIONE COMPATTO CON QUALCHE GHIAIETTO
SABBIONE COMPATTO GIALLASTRO
SABBIONE COMPATTO VERDASTRO
SABBIONE CON ARGILLA
SABBIONE CON ARGILLA E LEGNITE
SABBIONE CON CONCHIGLIE
SABBIONE CON GHIAIA
SABBIONE CON GHIAIETTO
SABBIONE CON GHIAIETTO IDRICO (SECONDA FALDA)
SABBIONE CON LIMO GRIGIO SCURO
SABBIONE CON POCO GHIAIETTO
SABBIONE CON POCO GHIAIETTO ARGILLOSO
SABBIONE CON QUALCHE CIOTTOLINO
SABBIONE CON QUALCHE GHIAIA
SABBIONE CON QUALCHE GRANULO DI GHIAIETTO
SABBIONE CON STRATI DI SABBIA FINE
SABBIONE CON SUBORDINATO GHIAIETTO
SABBIONE CON TRACC

In [ ]:
#all values containing a key word
keyW = 'TROVANTE DI CALCARE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',2)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

TROVANTE DI CALCARE ARENACEO COLOR NOCCIOLA LEGGERMENTE FRATTURATO


In [ ]:
#all values containing a key word
keyW = 'TROVANTE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

TROVANTE
TROVANTE (ARENARIA MEDIO-GROSSOLANA LAMINATA) E ARGILLA ROSSA
TROVANTE ARENACEO
TROVANTE CON ARGILLA
TROVANTE CON GROSSO CIOTTOLO SERPENTINITICO.
TROVANTE DI ARENARIA
TROVANTE DI ARENARIA GRIGIA A GRANA MEDIO GROSSA E A CEMENTO CALCAREO (40.00 - 40.70 MT.)
TROVANTE DI ARENARIA GRIGIA A GRANA MEDIO GROSSOLANA
TROVANTE DI ARENARIA GRIGIA, A GRANA MEDIA, PASSANTE A SILTITE NELLA PARTE ALTA
TROVANTE DI ARENARIA GRIGIO VERDE A GRANA GROSSA E CEMENTO CALCAREO
TROVANTE DI GNEISS
TROVANTE DI MEDIA GRANDEZZA
TROVANTE DI ROCCIA METAMORFICA
TROVANTE DI SERPENTINITE COLOR GRIGIO VERDE.
TROVANTE DI TONALITE COMPATTO
TROVANTE DIAMETRO 700 MM
TROVANTE DURISSIMO GRIGIO
TROVANTE GNEISSICO
TROVANTE IN CALCESTRUZZO
TROVANTE LEGNOSO
TROVANTE METAMORFITICO.
TROVANTE ROCCIOSO METAMORFICO


In [ ]:
#all values containing a key word
keyW = 'TROVANTI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

TROVANTI
TROVANTI (S.D.)
TROVANTI - GHIAIA E SABBIA
TROVANTI - GHIAIA E SABBIA ACQUIFERA
TROVANTI - GHIAIA E SABBIA CON ACQUA
TROVANTI - GHIAIA E SABBIA CON LIMO
TROVANTI - GHIAIONE E SABBIA
TROVANTI - GHIAIONE GHIAIA E SABBIA
TROVANTI ADDENSATI CON TRACCE DI ARGILLA
TROVANTI AMMASSATI
TROVANTI ANCHE MOLTO GROSSI CON GHIAIA E POCA SABBIA.
TROVANTI ARENACEI GRIGI A GRANA MEDIO GROSSOLANA DI DIMENSIONI PREVALENTEMENTE DECIMETRICHE
TROVANTI ARENACEI GRIGI A GRANA MEDIO-FINE E MEDIA, DI DIMENSIONI DA CENTIMETRICHE A DECIMETRICHE
TROVANTI CALCAREI
TROVANTI CALCAREI CON CIOTTOLI ED INTERCALAZIONI ARGILLOSE
TROVANTI CALCAREI FRIABILI, SABBIA E GHIAIETTA. (A METRI 45 INIZIA LA 2° FALDA E A M. 49 FINISCE LA 2° FALDA)
TROVANTI CIOTTOLI E GHIAIA GROSSOLANA CEMENTATI
TROVANTI CIOTTOLI GHIAIA GROSSA SABBIA GROSSA
TROVANTI CIOTTOLI GHIAIA SABBIA E LIMO
TROVANTI CON ALTERNANZE DI LIMO E ARGILLA MARRONE
TROVANTI CON ARGILLA
TROVANTI CON ARGILLA COMPATTA
TROVANTI CON ARGILLA GIALLA COMPATTA
TROVANTI CO

In [ ]:
#all values containing a key word
keyW = 'SILTITE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

SILTITE ARENACEA COLOR GRIGIO SCURA, CON SOTTILI VENATURE CALCITICHE, MEDIAMENTE FRATTURATA. FRATTURE APERTE IRREGOLARI I = 60° CON RIEMPIMENTO ARGILLOSI. GIUNTI DI STRATO AL TETTO E AL LETTO INCLINATI I = 30° CIRCA.
SILTITE ARENACEA MARNOSA COLOR GRIGIO SCURO, FRATTURATA (FRATTURE INCLINATE DI 45° E 70° CIRCA, LISCE ED IRREGOLARI, CON RIEMPIMENTO ARGILLOSO), GIUNTI DI STRATO I = 30° DA SERRATI A POCO APERTI, CON LOCALI SOTTILI LIVELLI DI BRECCIA COLOR GRIGIO VERDASTRO AD ELEMENTI MEDIO FINI SPIGOLOSI E SUBARROTONDATI.
SILTITE MARNOSO ARENACEA COLOR GRIGIO SCURO, COMPATTA, CON SOTTILI VENATURE CALCITICHE, A STRATIFICAZIONE EVIDENTE I = 30°; GIUNTI DI STRATIFICAZIONE SERRATI A SPAZIATURA CENTI DECIMETRICA. PRESENTI LIVELLI ARENACEI COLOR GRIGIO VERDASTRO. 36.2 METRI: FRATTURA APERTA MOLTO IRREGOLARE CON RIEMPIMENTO ARGILLOSO.
SILTITE MARNOSO ARENACEA COLOR GRIGIO SCURO, CON VENATURE CALCITICHE, A STRATIFICAZIONE EVIDENTE I = 30°; GIUNTI DI STRATIFICAZIONE LISCI, SERRATI E MOLTO SERRATI,

In [ ]:
#all values containing a key word
keyW = 'TUFO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

TUFO
TUFO - MARNA
TUFO / ROCCIA CALCAREA
TUFO A STRATTI SABBIOSI E COMPATTI
TUFO ARGILLA
TUFO ARGILLOSO
TUFO ARGILLOSO NERO
TUFO AZZURRO
TUFO BLU
TUFO BLU CON STRATI DI ARGILLA GRIGIA
TUFO BLU-VERDE
TUFO CALCAREO STRATIFICATO
TUFO CALCAREO STRATIFICATO - FALDA ACQUIFERA
TUFO COMPATTO
TUFO CON PIETRA
TUFO CON PIETRE
TUFO CON STRATI DI SABBIA E CONCHIGLIE
TUFO E ARGILLA
TUFO E PIETRA
TUFO E SABBIA GIALLA
TUFO E SABBIA GROSSA
TUFO E STRATI DI ARENARIA E SABBIA
TUFO GHIAIOSO
TUFO GIALLO
TUFO GIALLO (ARGILLA)
TUFO GRIGIO
TUFO GRIGIO MORBIDO
TUFO MELMOSO
TUFO MISTO TERRA
TUFO MOLLE CON STRATI DI ARENARIA
TUFO MOLLE E LEGNO
TUFO O TERRENO ARGILLOSO
TUFO ROSSO
TUFO ROSSO COMPATTO
TUFO ROSSO E PIETRA
TUFO ROSSO MISTO PIETRA
TUFO ROSSO MISTO PIETRE
TUFO ROSSO TIPO ARGILLOSO CON STRATI DI DEPOSITO ARENARIE
TUFO SABBIOSO
TUFO STRATIFICATO
TUFO STRATIFICATO DURO CON SABBIA DI ARENARIA
TUFO STRATIFICATO DURO CON SABBIA E ARENARIA.
TUFO TURCHINO


In [ ]:
#all values containing a key word
keyW = 'GHAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHAIA
GHAIA E ARGILLA
GHAIA E SABBIA
GHAIA ETEROMETRICA
GHAIA GIALLA SPORCA
GHAIA LIMOSA
GHAIA SABBIOSO-ARGILLOSA COMPATTA
GHAIA SECCA


In [ ]:
#all values containing a key word
keyW = 'GHIAI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAI
GHIAI (PRIMA FALDA) CEMENTATA
GHIAI CON ACQUIFERO SUPERF.
GHIAI CON ARGILLA
GHIAI CON PICCOLI STRATI COMPATTI DI CONGLOMERATO
GHIAI CON SABBIA
GHIAI CON SCARSA MATRICE CONTENENTE LA FALDA OGETTO DI EMUNGIMENTO
GHIAI DA GROSSOLANA A FINE CON ARGILLA E LIMO
GHIAI E SABBIA
GHIAI E SABBIA ROSSA
GHIAI ETEROMETRICA
GHIAI GROSSA COMPATTA
GHIAI MISTA ARGILLA


In [ ]:
#all values containing a key word
keyW = 'GHIAIETTA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIETTA
GHIAIETTA E SABBIA
GHIAIETTA E TERRICCIO
GHIAIETTA FINE
GHIAIETTA GROSSA BELLA


In [ ]:
#all values containing a key word
keyW = 'GHIAIETTO,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIETTO, CIOTTOLI MEDI E SABBIA SCURA FINE
GHIAIETTO, COMPATTO CON ARGILLA
GHIAIETTO, CON ARGILLA
GHIAIETTO, CON POCA ARGILLA
GHIAIETTO, E ARGILLA
GHIAIETTO, E, ARGILLA
GHIAIETTO, GH., SABBIA FINE GRIGIA, STR. ARG. GRIG.
GHIAIETTO, GHIAIA CON CIOTTOLI
GHIAIETTO, GHIAIA CON POCA SABBIA E QUALCHE CIOTTOLO
GHIAIETTO, GHIAIA CON SABBIA GROSSA
GHIAIETTO, GHIAIA CON SABBIA MEDIA
GHIAIETTO, GHIAIA CON SABBIA, DI COLORE GRIGIO
GHIAIETTO, GHIAIA E CIOTTOLI CON SABBIA GROSSA
GHIAIETTO, GHIAIA E SABBIA
GHIAIETTO, GHIAIA E SABBIA GRIGIA
GHIAIETTO, GHIAIA E SABBIA GROSSA
GHIAIETTO, GHIAIA E SABBIA LEGEGRMENTE ARGILLOSA
GHIAIETTO, GHIAIA E SABBIA MEDIA
GHIAIETTO, GHIAIA E SABBIONE
GHIAIETTO, GHIAIA GROSSA CON SABBIA
GHIAIETTO, GHIAIA GROSSA, SABBIA
GHIAIETTO, GHIAIA PULITA
GHIAIETTO, GHIAIA, ARGILLA GIALLA LIMOSA
GHIAIETTO, GHIAIA, CIOTTOLI E POCA SABBIA
GHIAIETTO, GHIAIA, CIOTTOLI E SABBIA
GHIAIETTO, GHIAIA, LENTI ARGILLOSE GIALLASTRE
GHIAIETTO, GHIAIA, SABBIA COMPATTA
GHIAIETTO, GHIAIA, SABBIA E

In [ ]:
#all values containing a key word
keyW = 'GHIIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIIA CON CEPPO BIANCO E LENTI DI COMGLOMERATO


In [ ]:
#all values containing a key word
keyW = 'GROSSI CIOTTOLI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GROSSI CIOTTOLI (DIAMETRO MAX < 14 CM) GABBRICI E PARASINITICI, CON SCARSA MATRICE FINE.
GROSSI CIOTTOLI (DN 35 MM) IN MATRICE GHIAIOSA
GROSSI CIOTTOLI CON GHIAIA
GROSSI CIOTTOLI CON GHIAIA COMPATTA
GROSSI CIOTTOLI CON GHIAIA MOLTA SABBIA E LENTI DI ARGILLA
GROSSI CIOTTOLI CON LIMO COMPATTO
GROSSI CIOTTOLI CON SABBIA
GROSSI CIOTTOLI CON SABBIA E GHIAIA COMPATTA CON FILTRAZIONI
GROSSI CIOTTOLI CON TROVANTE
GROSSI CIOTTOLI CON TROVANTI -
GROSSI CIOTTOLI CON UN PO' DI SABBIA E CONGLOMERATO
GROSSI CIOTTOLI DURI CON GHIAIA
GROSSI CIOTTOLI DURI CON GHIAIA E SABBIA
GROSSI CIOTTOLI DURI, GHIAIA, SAB.
GROSSI CIOTTOLI E GHIAIA
GROSSI CIOTTOLI E PICCOLI TROVANTI (DIAMETRO 15 CM) OFIOLITICI CON SCARSA MATRICE FINE.
GROSSI CIOTTOLI O TROVANTI CALCAREI
GROSSI CIOTTOLI PLURIDECIMETRICI (20 - 30 CM MAX) CON GHIAIA E ARGILLA VARIEGATA GRIGIO-GIALLASTRO AD ELEVATA PLASTICITA'.
GROSSI CIOTTOLI PLURIDECIMETRICI (BMAX= 30 - 35 CM) CON GHIAIA, ARGILLA E SUBORDINATA SABBIA GROSSOLANA.
GROSSI CIOTTOLI SCIOLTI

In [ ]:
#all values containing a key word
keyW = 'PICCOLI CIOTTOLI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

PICCOLI CIOTTOLI OFIOLITICI E GRANULITICI (DIAMETRO < 7 CM) E GHIAIA GROSSOLANA, DEBOLMENTE SABBIOSA.


In [ ]:
#all values containing a key word
keyW = 'GROSSI TROVANTI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

GROSSI TROVANTI
GROSSI TROVANTI (DIAMETRO > 20 CM) E GHIAIE GROSSOLANE
GROSSI TROVANTI (DIAMETRO >300 MM) E GHIAIE GROSSOLANE
GROSSI TROVANTI A TRATTI CEMENTATI CON ARGILLA
GROSSI TROVANTI CON GHIAIA
GROSSI TROVANTI DI GNEISS E MICASCISTI IMMERSI IN UNA MATRICE GHIAIOSO SABBIOSA
GROSSI TROVANTI DIAMETRO 600 / 1000 MM
GROSSI TROVANTI E GHIAIA
GROSSI TROVANTI E GHIAIE GROSSOLANE
GROSSI TROVANTI GNEISSICI IMMERSI IN SABBIE MEDIE DEBOLMENTE LIMOSE MARRONCINE
GROSSI TROVANTI IN GHIAIA


In [ ]:
#all values containing a key word
keyW = 'GROSSO TROVANTE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

GROSSO TROVANTE
GROSSO TROVANTE E GHIAIA


In [ ]:
#all values containing a key word
keyW = 'GROSSO BLOCCO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

GROSSO BLOCCO (TROVANTE)


In [ ]:
#all values containing a key word
keyW = 'MASSI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

MASSI DI MEDIA GRANDEZZA ALTERNATI A STRATI DI GHIAIA SISTOSA A CONTINUE FALDE SINO A FONDO POZZO
MASSI E CIOTTOLI MOLTO GROSSI CON TERRA SPORCA


In [ ]:
#all values containing a key word
keyW = 'LEGNO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

LEGNO
LEGNO CON SABBIA
LEGNO CON SABBIA E ARGILLA
LEGNO CON STRATI DI GHIAIA E CROSTE DI ARENARIA
LEGNO E STRATI DI ARGILLA
LEGNO FOSSILE E GHIAIETTO
LEGNO MOLTO COMPATTO
LEGNO POCO DETERIORATO


In [ ]:
#all values containing a key word
keyW = 'MARMI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

MARMI
MARMI BIANCASTRI
MARMI GRIGI LISTATI A FLOGOPITE INTENSAMENTE FRATTURATI LUNGO PIANI SUB-VERTICALI. PRES. ABB. CALCITE E LIVELLI DI PICCOLI SOLFURI LUNGO PIANI DI SCISTOSITA'. DA 211.20 A 211.30 M BRECCIA IN PARASCISTI A GRANATO
MARMI LISTATI GRIGIO CHIARI A FLOGOPITE
MARMI MICACEI
MARMI MICACEI GRIGIO CHIARI MOLTO FRATTURATI CON LIVELLI DI CALCESCISTI MOLTO ALTERATI (245.90 - 246.00) E ZONE DI TAGLIO


In [ ]:
#all values containing a key word
keyW = 'MARMO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

MARMO A FLOGOPITE INTENSAMENTE FRATTURATO LUNGO PIANI SUB-VERTICALI
MARMO BIANCO FRATTURATO LOC. INTENSAMENTE ALTERATO. PRES. FORTE ODORE DI ZOLFO
MARMO DOLOMITICO INTENSAMENTE FRATTURATO MA NON ALTERATO
MARMO GRIGIO BIANCASTRO NON FRATTURATO


In [ ]:
#all values containing a key word
keyW = 'MICASCISTI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

MICASCISTI A MEDIA COMPATTEZZA
MICASCISTI A PIGMENTO GRAFITACO (?) PASSANTI TALORA A SCISTI GRAFITICI DI COLORE NERO.
MICASCISTI A PIGMENTO GRAFITICO DI COLORE GRIGIO SCURO.
MICASCISTI BIOTITICI CON PREVALENTI GIUNTI DI SCISTOSITA' E CON GIUNTI OSSIDATI. / MICASCISTI BIOTITICI, GRIGIO-CHIARO, STRUTTURA ORIENTATA SCISTOSA BEN EVIDENTE, TESSITURA A GRANI MEDIO-FINI, DURI, CON GIUNTI LOCALMENTE OSSIDATI. I GIUNTI DI SCISTOSITA' SONO INCL. 48 - 55 GRADI, SPAZIATURA DECIMETRICA (MAX 55 CM), ONDULATI-PIANI, LISCI-LAMINATI, FRESCHI O OSSIDATI, APERTI CON RIEMPIMENTO DI MILONITI PREVALENTI O LOCALM. CLORITI E PIRITE. LE FRATTURE SONO INCL. 45 - 80 GRADI, IRREG.-ONDULATE, LISCE-SCABRE, FRESCHE O OSSIDATE, SERRATE O APERTE CON MILONITI E OSSIDI. PARTICOLARM. SIGNIFICATIVE: SCISTOSITA' A 124.90 (CON CLORITI) A 126.55 (CON ARGILLA) E LA FRATTURA 127.45 (MOLTO OSSIDATA). / LIVELLO DI CATACLASITI: 125.50 - 125.80.
MICASCISTI BIOTITICI E MUSCOVITICI ALTERATI E FRATTURATI SINO A 50 M DI PROFONDITA'
MI

In [ ]:
#all values containing a key word
keyW = 'MICASCISTO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

MICASCISTO A GRANATO (DIAMETRO 1 - 2 MM - GRANATI 2 - 5%) GRIGIO DA DEB. FRATTURATO A FRATTURATO LUNGO PIANI DI SCISTOSITA' SUB-ORIZZONTALI. LOC. PRES. LIVELLETTI VERDASTRI CON CLORITE
MICASCISTO A GRANATO (DIAMETRO 1 - 2 MM - GRANATI 5%) GRIGIO DA DEB. FRATTURATO A FRATTURATO LUNGO PIANI DI SCISTOSITA' SUB-ORIZZONTALI. PRES. RIEMP. DI QUARZO E VENE DI CALCITE E CLORITE
MICASCISTO A GRANATO (DIAMETRO 1 - 4 MM - GRANATI 15%) GRIGIO CHIARO DA DEB. FRATTURATO A FRATTURATO LUNGO PIANI DI SCISTOSITA' SUBORIZZONTALI. PRES. RIEMPIMENTI DI QUARZO E RARE VENE DI CALCITE E CLORITE
MICASCISTO A GRANATO (DIAMETRO 1 - 4 MM - GRANATI 15%) GRIGIO CHIARO FRATTURATO LUNGO PIANI DI SCISTOSITA' SUB-ORIZZONTALI. FREQUENTI RIEMPIMENTI DI QUARZO
MICASCISTO A GRANATO (DIAMETRO 1 2 MM - GRANATI 5%) GRIGIO DA DEB. FRATTURATO A FRATTURATO LUNGO PIANI DI SCISTOSITA' SUB-ORIZZONTALI. LOC. PRES. LIVELLETTI DI QUARZO
MICASCISTO GRIGIO DA DEB. FRATTURATO A FRATTURATO LUNGO PIANI DI SCISTOSITA' SUB-ORIZZONTALI. LOC. 

In [ ]:
#all values containing a key word
keyW = 'PIETRA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

PIETRA
PIETRA (CON ARGILLA!) (A 185)
PIETRA COMPATTA
PIETRA DURA
PIETRA E ARGILLA
PIETRA FRIABILE
PIETRA NERA
PIETRA NERA FESSURATA A STRATI
PIETRA ROCCIOSA


In [ ]:
#all values containing a key word
keyW = 'PIETRE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

PIETRE
PIETRE CON ARGILLA
PIETRE CON GHIAIA
PIETRE CON GHIAIA GROSSOLANA
PIETRE E ARGILLA
PIETRE E GHIAIA
PIETRE E GHIAIETTO GRIGIO-VERDE CON SABBIA ARGILLOSA
PIETRE E SABBIA
PIETRE E SABBIONE
PIETRE GRANDI
PIETRE GROSSE
PIETRE GROSSE CON GHIAIA
PIETRE GROSSE E GHIAIA
PIETRE GROSSE E GHIAIA CON ARGILLA
PIETRE GROSSE E TROVANTI DURI
PIETRE GROSSE MISTE TERRA
PIETRE GROSSOLANE


In [ ]:
#all values containing a key word
keyW = 'PRASINITI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

PRASINITI


In [ ]:
#all values containing a key word
keyW = 'QUARZITE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

QUARZITE BIANCASTRA DURA SCISTOSA CON LENTI DI QUARZO E ROCCIA FRESCA NON OSSIDATA, CATACLASITI DI ORIGINE MECCANICA
QUARZITE BIOTITICA CON FRATTURE AVENTI SPAZIATURA DA DECIMETRICA A METRICA E TALVOLTA OSSIDATE. QUARZITE BIOTITICA, BIANCASTRA, MOLTO DURA, STRUTTURA ORIENTATA SCISTOSA POCO EVIDENTE, TESSITURA MEDIO-GROSSA. SCISTOSITA' INCL. 50 GRADI, SPAZIATURA PLURIMETRICA O CENTIMETRICA, PIANE, LISCE, APERTE: 138.00 CON CATACLASITI. SABBIA E ARGILLA: 140.15 - 140.35 CON MILONITI, CLORITI E TALCO; 142.45 OSSIDATA; 142.60 CON MILONITI E PIRITE. I GIUNTI DI FRATTURA INCL. 50 - 80 GRADI, IRREGOLARI, SCABRI, SERRATI E FRESCHI. / PARTICOLARMENTE IMPORTANTI LE FRATTURE: 138.45 CON MINERALI VERDI; 139.95 MOLTO OSSIDATA, MODER. LARGA CON MINERALI VERDI, OSSIDI E QUARZO; 142.15 MODERAT. LARGA CON QUARZO; 142.60 MOLTO OSSIDATA.


In [ ]:
#all values containing a key word
keyW = 'PUDDINGA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

PUDDINGA
PUDDINGA - ACQUIFERO
PUDDINGA COMPATTA CON MATRICE CALCAREA CON RARE LENTI PIU' FESSURATE
PUDDINGA CON GHIAIA
PUDDINGA DURISSIMA
PUDDINGA E STRATI GHIAIOSI
PUDDINGA FESSURATA
PUDDINGA MOLTO DURA


In [ ]:
#all values containing a key word
keyW = 'GABBRO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

GABBRO RODINGITICO RODINGITE CALCITE E GRANATO COLOR ROSSO (GRANATITE)


In [ ]:
#all values containing a key word
keyW = 'MATERIALE SABBIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

MATERIALE SABBIOSO FINE
MATERIALE SABBIOSO GHIAIOSO DI RIPORTO.
MATERIALE SABBIOSO GROSSOLANO


In [ ]:
#all values containing a key word
keyW = 'LIVELLO SABBIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

LIVELLO SABBIOSO CON PICCOLI CIOTTOLI.
LIVELLO SABBIOSO DEBOLMENTE GHIAIOSO, COSTITUITO DA SABBIA MEDIO-FINE CON ALL'INTERNO ALCUNI ELEMENTI A DIMENSIONI MAGGIORI (DIAMETRO DI 2 - 4 CM, DA ARROTONDATI A LIEVEMENTE SPIGOLOSI), DI COLORE DA GRIGIO A MARRONCINO


In [ ]:
#all values containing a key word
keyW = 'LIVELLI SABBIOSI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

LIVELLI SABBIOSI E SABBIOSO LIMOSI


In [ ]:
#all values containing a key word
keyW = 'LIVELLI DI SABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

LIVELLI DI SABBIA E ARGILLA
LIVELLI DI SABBIA E CONGLOMERATO


In [ ]:
#all values containing a key word
keyW = 'DEPOSITO SABBIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

DEPOSITO SABBIOSO
DEPOSITO SABBIOSO CON CIOTTOLI
DEPOSITO SABBIOSO LIMOSO CON INTERCALAZIONI GHIAIOSO SABBIOSE CIOTTOLOSE
DEPOSITO SABBIOSO LIMOSO, CON PASSATE GHIAIOSO SABBIOSE
DEPOSITO SABBIOSO LIMOSO, CON SUBORDINATI LIVELLI SABBIOSO GHIAIOSI, A PREVALENTE COLORAZIONE GRIGIASTRA


In [ ]:
#all values containing a key word
keyW = 'LIVELLO DI SABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

LIVELLO DI SABBIA DEB. LIMOSA NOCCIOLA.
LIVELLO DI SABBIA F LIMOSA BRUNA CON TORBA FIBROSA.
LIVELLO DI SABBIA FINE CON LIMO E DEBOLE PRESENZA DI GHIAIA FINE (DIAMETRO MAX EL. 2-3 CM), A MODERATA PERMEABILITA', MARRONCINO NOCCIOLA - GIALLOGNOLO
LIVELLO DI SABBIA LIMOSA DEBOLMENTE GHIAIOSA FINE (ELEMENTI CON DIAMETRO MAX 1 - 2 CM), BEN COMPATTATA E/O PARZIALMENTE PSEUDOCEMENTATA, NO PLASTICA AL TATTO, DI COLORE DA NOCCIOLA A GIALLOGNOLO
LIVELLO DI SABBIA M/F DI COLORE BRUNO.


In [ ]:
#all values containing a key word
keyW = 'STRATI DI SABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

STRATI DI SABBIA ALTERNATI AD ARGILLA
STRATI DI SABBIA CON ARGILLA SABBIOSA
STRATI DI SABBIA CON LEGNI
STRATI DI SABBIA CON VENATURE DI ACQUA.
STRATI DI SABBIA DEBOLM. CEM. E STRATI LIMOSI
STRATI DI SABBIA E ARGILLA
STRATI DI SABBIA E ARGILLA ALTERNI
STRATI DI SABBIA E ARGILLA GIALLA
STRATI DI SABBIA E GHIAIA


In [ ]:
#all values containing a key word
keyW = 'ALTERNANZA DI LIVELLI SABBIOSI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

ALTERNANZA DI LIVELLI SABBIOSI A VARIA GRANULOM. CON GHIAIETTO
ALTERNANZA DI LIVELLI SABBIOSI E ARGILLOSI
ALTERNANZA DI LIVELLI SABBIOSI LIMOSI


In [ ]:
#all values containing a key word
keyW = 'ALTERNANZA DI SABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

ALTERNANZA DI SABBIA DEBOLMENTE GHIAIOSA GRIGIA E ARGILLA LIMOSA GRIGIA O MARRONCINA
ALTERNANZA DI SABBIA E ARGILLA
ALTERNANZA DI SABBIA E GHIAIA ARGILLOSA COMPATTA
ALTERNANZA DI SABBIA E LIMO GRIGIO-GIALLASTRO
ALTERNANZA DI SABBIA FINE LIMOSA GIALLASTRA, LENTI DI POCHI CM. DI LIMO DELLO STESSO COLORE PIU' QUALCHE LIVELLETTO ARGILLOSO (PREVALENZA DI SABBIA)
ALTERNANZA DI SABBIA FINE LIMOSA GRIGIA E ARGILLA GRIGIA E MARRONCINA (LIVELLI CON SPESSORI DA POCHI MM AD ALCUNI CM)
ALTERNANZA DI SABBIA GHIAIA E ARGILLA
ALTERNANZA DI SABBIA MEDIA E SABBIA FINE MOLTO ADDENSATA
ALTERNANZA DI SABBIA MEDIA LIMOSA SATURA E LIMO ARGILLOSO MOLTO UMIDO IL TUTTO DI COLORE GRIGIO


In [ ]:
#all values containing a key word
keyW = 'ALTERNANZE DI SABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

ALTERNANZE DI SABBIA DEBOLMENTE LIMOSA NOCCIOLA E CONGLOMERATO MEDIAMENTE CEMENTATO
ALTERNANZE DI SABBIA E ARGILLA
ALTERNANZE DI SABBIA E ARGILLA GRIGIA
ALTERNANZE DI SABBIA E LIMO
ALTERNANZE DI SABBIA E LIMO GRIGI
ALTERNANZE DI SABBIA FINE E ARGILLA
ALTERNANZE DI SABBIA FINE E ARGILLA LIMOSA
ALTERNANZE DI SABBIA FINE NOCCIOLA E SABBIA MEDIA-FINE GRIGIASTRA LOCALMENTE LIMOSA CON POCA GHIAIA ETEROMETRICA IN PREVALENZA ALTERATA, PRESENZA DI RARI CIOTTOLI
ALTERNANZE DI SABBIA GHIAIA E ARGILLA
ALTERNANZE DI SABBIA LIMOSA E ARGILLA SABBIOSA
ALTERNANZE DI SABBIA MEDIO - FINE GRIGIA E ARGILLA GRIGIA
ALTERNANZE DI SABBIA MEDIO FINE E ARGILLA GRIGIA (PREVALENZA DI ARGILLA)


In [ ]:
#all values containing a key word
keyW = 'ALTERNANZA DI SABBIE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

ALTERNANZA DI SABBIE E ARENARIE
ALTERNANZA DI SABBIE E ARGILLE
ALTERNANZA DI SABBIE E GHIAIE
ALTERNANZA DI SABBIE FINI E ARGILLE LIMOSE
ALTERNANZA DI SABBIE LIMOSE ADDENSATE E LIMI SABBIOSI GRIGIASTRI CONSISTENTI.


In [ ]:
#all values containing a key word
keyW = 'ALTERNANZE DI SABBIE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

ALTERNANZE DI SABBIE E ARENARIE PSEUDOCOERENTI GIALLASTRE
ALTERNANZE DI SABBIE E ARG.
ALTERNANZE DI SABBIE E GHIAIE
ALTERNANZE DI SABBIE E GHIAIE CON LOCALI ALTERNANZE ARGILLOSE
ALTERNANZE DI SABBIE E GHIAIE DI COLRE GRIGIO
ALTERNANZE DI SABBIE E GHIAIE SPORCHE, DEBOLMENTE LIMOSE, CON CIOTTOLI INGLOBATI
ALTERNANZE DI SABBIE ED ARGILLA
ALTERNANZE DI SABBIE FINI E ARGILLE LIMOSE
ALTERNANZE DI SABBIE FINI E GROSSOLANE CON LIVELLI DI GHIAIETTO
ALTERNANZE DI SABBIE FINI LIMOSE E ARGILLE
ALTERNANZE DI SABBIE FINI, LIMI, LIMI ARGILLOSI ED ARGILLE LIMOSE
ALTERNANZE DI SABBIE GROSSOLANE ARENARIE E LENTI LIMOSE DI COLORE SENAPE
ALTERNANZE DI SABBIE LIMOSE CONSOLIDATE ED ARGILLE SABBIOSE DI COLORE SENAPE
ALTERNANZE DI SABBIE LIMOSE E LIMI SABBIOSI


In [ ]:
#all values containing a key word
keyW = 'CONGLOMERATO DI SABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

CONGLOMERATO DI SABBIA
CONGLOMERATO DI SABBIA CEM. CON GHIAIETTO
CONGLOMERATO DI SABBIA E GHIAIA
CONGLOMERATO DI SABBIA PIETRE


In [ ]:
#all values containing a key word
keyW = 'CONGLOMERATO SABBIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

CONGLOMERATO SABBIOSO
CONGLOMERATO SABBIOSO CON LENTI DI ARGILLA
CONGLOMERATO SABBIOSO DEB. LIMOSO


In [ ]:
#all values containing a key word
keyW = 'RIPORTO SABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

RIPORTO SABBIA E STABILIZZATO


In [ ]:
#all values containing a key word
keyW = 'STRATO SABBIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

STRATO SABBIOSO CHIARO


In [ ]:
#all values containing a key word
keyW = 'SUOLO SABBIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SUOLO SABBIOSO


In [ ]:
#all values containing a key word
keyW = 'CONGLOMERATO ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

CONGLOMERATO ARGILLOSO
CONGLOMERATO ARGILLOSO COLOR SENAPE
CONGLOMERATO ARGILLOSO COMPATTO
CONGLOMERATO ARGILLOSO CON CIOTTOLI
CONGLOMERATO ARGILLOSO CON GHIAIA
CONGLOMERATO ARGILLOSO CON POCA GHIAIA
CONGLOMERATO ARGILLOSO DURO DA PERFORARE.
CONGLOMERATO ARGILLOSO E GHIAIA
CONGLOMERATO ARGILLOSO GRIGIO
CONGLOMERATO ARGILLOSO MOLTO COMPATTO


In [ ]:
#all values containing a key word
keyW = 'DA ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

DA ARGILLA AD ARGILLA LIMOSA ADDENSATA CON RARI CIOTTOLI. COLORE PREVALENTEMENTE ROSSASTRO / BEIJE O VERDASTRO. PLASTICITA' MODERATA. UMIDITA' SCARSA. CIOTTOLI DA CENTIMETRICI A DECIMETRICI, DA SUB ARROTONDATI AD ARROTONDATI DI NATURA POLIGENICA. SI OSSERVANO LOCALI LENTI CENTIMETRICHE SABBIOSE GHIAIOSE. (FERRETTO)
DA ARGILLA AD ARGILLA LIMOSA CONSISTENTE CON SABBIA E CIOTTOLI E RARI CIOTTOLI. COLORE GRIGIO / MARRONE. PLASTICITA' MODERATA. UMIDITA' SCARSA. CIOTTOLI DA CENTIMETRICI A DECIMETRICI, DA SUB ARROTONDATI AD ARROTONDATI DI NATURA POLIGENICA. SI OSSERVANO LOCALI LENTI CENTIMETRICHE SABBIOSE GHIAIOSE. (FERRETTO)


In [ ]:
#all values containing a key word
keyW = 'DEPOSITO ALLUVIONALE ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

DEPOSITO ALLUVIONALE ARGILLOSO E ARGILLOSO-LIMOSO
DEPOSITO ALLUVIONALE ARGILLOSO ED ARGILLOSO-LIMOSO
DEPOSITO ALLUVIONALE ARGILLOSO ED ARGILLOSO-LIMOSO CON CIOTTOLI


In [ ]:
#all values containing a key word
keyW = 'DEPOSITO ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

DEPOSITO ARGILLOSO
DEPOSITO ARGILLOSO / ARGILLA MARRONE
DEPOSITO ARGILLOSO ASCIUTTO, CON CIOTTOLI
DEPOSITO ARGILLOSO COMPATTO
DEPOSITO ARGILLOSO COMPATTO / ARGILLA COMPATTA MARRONE
DEPOSITO ARGILLOSO COMPATTO CON SPORADICA PRESENZA DI ELEMENTI CIOTTOLOSI DELL'ORDINE MASSIMO DEL CENTIMETRO, PRATICAMENTE IMPERMEABILE.
DEPOSITO ARGILLOSO COMPATTO, MARRONE
DEPOSITO ARGILLOSO CON CARATTERISTICHE DI BUONA PLASTICITA'; PRESENZA DI INTERCALAZIONI SABBIOSO-LIMOSE CON SPORADICI CLASTI DI DIMENSIONI CENTIMETRICHE.
DEPOSITO ARGILLOSO CON CIOTTOLI A SCARSA FREQUENZA
DEPOSITO ARGILLOSO CON CIOTTOLI NELLA PARTE SUPERIORE
DEPOSITO ARGILLOSO DI COLORE 7.5YR 3/3 NERO OLIVA. TRA 7 E 8 M. SI RISCONTRA FRAZIONE SABBIOSA.
DEPOSITO ARGILLOSO DI COLORE BRUNO OLIVA CON SPORADICI CIOTTOLI DI NATURA CALCAREO-MARNOSA DI DIMENSIONI CENTIMETRICHE.
DEPOSITO ARGILLOSO E ARGILLOSO - LIMOSO, DI COLORE VARIABILE DA MARRONE A GRIGIASTRO
DEPOSITO ARGILLOSO ED ARGILLOSO-LIMOSO
DEPOSITO ARGILLOSO ED ARGILLOSO-LIMOSO CON CIO

In [ ]:
#all values containing a key word
keyW = 'LIVELLO ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

LIVELLO ARGILLOSO
LIVELLO ARGILLOSO DI COLORE GRIGIO-AZZURRO COMPATTO.
LIVELLO ARGILLOSO DI COLORE GRIGIO-AZZURRO CON CARATTERISTICHE DI PLASTICITA'.
LIVELLO ARGILLOSO E ARGILLOSO-LIMOSO
LIVELLO ARGILLOSO FRANCO
LIVELLO ARGILLOSO FRANCO COLOR GRIGIASTRO
LIVELLO ARGILLOSO LIMOSO CON GHIAIA
LIVELLO ARGILLOSO MOLLE


In [ ]:
#all values containing a key word
keyW = 'LIVELLO DI ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

LIVELLO DI ARGILLA DEBOLMENTE LIMOSA NOCCIOLA


In [ ]:
#all values containing a key word
keyW = 'MATERIALE ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

MATERIALE ARGILLOSO
MATERIALE ARGILLOSO BRUNO ROSSICCIO
MATERIALE ARGILLOSO CON CIOTTOLI


In [ ]:
#all values containing a key word
keyW = 'RIPORTO ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

RIPORTO ARGILLA GHIAIOSA CON RESTI DI DEMOLIZIONE.


In [ ]:
#all values containing a key word
keyW = 'STRATO ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

STRATO ARGILLOSO


In [ ]:
#all values containing a key word
keyW = 'STRATO DI ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

STRATO DI ARGILLA
STRATO DI ARGILLA DURA
STRATO DI ARGILLA E GHIAIA
STRATO DI ARGILLA GIALLA
STRATO DI ARGILLA GIALLA DURISSIMO DA PERFORARE.
STRATO DI ARGILLA MISTA A GHIAIA


In [ ]:
#all values containing a key word
keyW = 'SUBSTRATO ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

SUBSTRATO ARGILLOSO PLIOCENICO


In [ ]:
#all values containing a key word
keyW = 'SUOLO ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

SUOLO ARGILLOSO
SUOLO ARGILLOSO RUBEFATTO


In [ ]:
#all values containing a key word
keyW = 'ALTERNANZA DI MARNA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

ALTERNANZA DI MARNA ARGILLOSA GRIGIO, DA MOLTO CONSISTENTE A DURA E SABBIA FINE-MEDIA, GRIGIO, CON TRITUME ORGANOGENO IN STRATI CENTIMETRICI.
ALTERNANZA DI MARNA DI COLORE GRIGIO E SABBIA FINE-FINISSIMA, GRIGIO IN STRATI DECIMETRICI. PRESENZA DI LAMINE MILLIMETRICHE TORBOSE, NERASTRE, INCLINATE DI 8-10 GRADI.
ALTERNANZA DI MARNA E SABBIA FINE-FINISSIMA, GRIGIO, CON PRESENZA DI RESTI ORGANOGENI. LAMINE MILLIMETRICHE DI TORBA INCLINATE DI CIRCA 10 GRADI SULL'ORIZZONTALE.
ALTERNANZA DI MARNA E SABBIA IN SEQUENZA CON LAMINE TORBOSE NERASTRE PIANO-PARALLELE, SUBORIZZONTALI (INCLINAZIONE MEDIA DELLE LAMINE: 8-10 GRADI).
ALTERNANZA DI MARNA GRIGIO CON LAMINE MILLIMETRICHE DI MATERIALE TORBOSO E STRATI DECIMETRICI DI SABBIA FINE-FINISSIMA, MOLTO ADDENSATA.
ALTERNANZA DI MARNA GRIGIO E DI SABBIA FINE-FINISSIMA IN STRATI CENTIMETRICI. PRESENZA DI LAMINE MILLIMETRICHE TORBOSE BRUNO-NERASTRE E DI STRATI ORGANOGENI.
ALTERNANZA DI MARNA GRIGIO MOLTO COMPATTA E LIVELLI SABBIOSI FINI DA MILLIMETRICI A

In [ ]:
#all values containing a key word
keyW = 'CALCESCISTI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

CALCESCISTI
CALCESCISTI E CALCEMICASCISTI
CALCESCISTI E CALCEMICASCISTI ESTREMAMENTE ALTERATI;
CALCESCISTI FILLADICI
CALCESCISTI MARMOREI
CALCESCISTI PIU' O MENO FILLADICI
CALCESCISTI PIU' O MENO MARMOREI
CALCESCISTI SFATTI


In [ ]:
#all values containing a key word
keyW = 'CALCESCISTO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

CALCESCISTO GRIGIO BIANCASTRO INTENSAMENTE FRATTURATO LUNGO PIANI SUB-VERTICALI. PRES. ABB. CALCITE E RARI LIVELLI DI GRANATI ROSSI DI PICCOLE DIMENSIONI. PRES. DI SOLFURI LUNGO PIANI DI SCISTOSITA'
CALCESCISTO GRIGIO INTENSAMENTE FRATTURATO CON PIANI DI SCISTOSITA' SUB-ORIZZONTALI. PRES. INTERLIVELLI DI MARMO
CALCESCISTO GRIGIO MOLTO FRATTURATO LUNGO PIANI SUB-VERTICALI CON RIEMPIMENTO CALCITICO. PRES. RARI PICCOLI GRANATI
CALCESCISTO GRIGIO QUARZOSO FRATTURATO LUNGO PIANI SUB-VERTICALI CON RIEMPIMENTO CALCITICO. PIANI DI SCISTOSITA' SUB-ORIZZONTALI. PRES. OMBRE DI GRANATO E RARI GRANATI SANI
CALCESCISTO GRIGIO SCURO MOLTO FRATTURATO LUNGO PIANI SUB-VERTICALI. PIANI DI SCISTOSITA' SUB-ORIZZONTALI. PRES. RARI PICCOLI GRANATI E FREQUENTI LIVELLETTI QUARZO E CALCITE


In [ ]:
#all values containing a key word
keyW = 'CALCARE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

CALCARE
CALCARE (INIZIO 3° FALDA)
CALCARE (MOLTO SIMILE AL DEPOSITO DEL CARBONATO DI CALCIO)
CALCARE ALLOCTONO DEBOLMENTE FESSURATO
CALCARE ALLOCTONO DURO
CALCARE ARGILLOSO
CALCARE BIANCASTRO COMPATTO CON SOTTILI INTERCALAZIONI DI CALCARI SCAGLIOSI
CALCARE BIANCASTRO CON ALTERAZIONE ARGILLOSA INTENSA E FRATTURATA
CALCARE BIANCO
CALCARE BIANCO AVORIO CON ALTERNANZE DI LIVELLI DI ARGILLA SCURA
CALCARE BIANCO AVORIO CON NODULI DI SELCE.
CALCARE BIANCO COMPATTO
CALCARE BIANCO COMPATTO (MAIOLICA)
CALCARE BIANCO COMPATTO CON INTERCALAZIONI DI MARNA BIANCASTRA
CALCARE COLORE CHIARO A TRATTI FRATTURATO
CALCARE COLORE SCURO FRATTURATO
CALCARE COMPATTO
CALCARE COMPATTO AVORIO
CALCARE COMPATTO CON INTERSTRATI ARGILLOSI GRIGIO-VERDI (MEDOLO)
CALCARE COMPATTO CON LIVELLI DI CALCARE SCURO
CALCARE CON ALCUNI STRATI DI SABBIA E TROVANTI
CALCARE CON ARGILLA GRIGIA
CALCARE CON INTERCALAZIONI MARNOSE
CALCARE CON TROVANTI E STRATI DI SABBIA
CALCARE DEBOLMENTE FRATTURATO
CALCARE DI VARI COLORI (GRIGIO - VE

In [ ]:
#all values containing a key word
keyW = 'BLOCCHI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

BLOCCHI (TROVANTI) CON GHIAIE SABBIOSO LIMOSE
BLOCCHI CIOTTOLI CON SABBIA IN TRACCIA
BLOCCHI CON FREQUENTI LIVELLI SABBIOSO LIMOSI
BLOCCHI CON GHIAIA E CIOTTOLI
BLOCCHI CON GHIAIA E SABBIA
BLOCCHI CON LEGANTE CALCAREO SCURO
BLOCCHI DI ARENARIA GRIGIA
BLOCCHI DI GRANITO, GNEISS QUARZITI
BLOCCHI DI ROCCE CALCAREE E DOLOMITICHE
BLOCCHI DI ROCCIA
BLOCCHI E CIOTTOLI
BLOCCHI E CIOTTOLI (DIAM. MAX. 40-50 CM) CON LIMO SABBIOSO, COLORE GRIGIO-VERDOGNOLO.
BLOCCHI E CIOTTOLI ARROTONDATI DI ROCCE DOLOMITICHE E CIOTTOLI DI ROCCE METAMORFICHE
BLOCCHI E CLASTI CALCAREI
BLOCCHI E STRATI CALCAREO MARNOSI PER LO PIU' IN ASSETTO CAOTICO INTERVALLATI DA ORIZZONTI ARGILLOSI GRIGI E GRIGIO SCURI ALTERATI
BLOCCHI MARNOSI
BLOCCHI MARNOSI IN ARGILLA


In [ ]:
#all values containing a key word
keyW = 'BLOCCO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

BLOCCO ARENACEO
BLOCCO DI CALCESTRUZZO
BLOCCO DI CONGLOMERATO A CEMENTO QUARZOSO (VERUCANO)
BLOCCO LITOIDE DI COMPOSIZIONE GNEISSICA (TROVANTE)
BLOCCO PIETRA


In [ ]:
#all values containing a key word
keyW = 'ARENARIE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

ARENARIE
ARENARIE A TRAMA GROSSA
ARENARIE ALTERATE/FRATTURATE
ARENARIE CALCAREE CON LIVELLETTI SABBIOSI
ARENARIE CEMENTATE, COLORE GRIGIO;
ARENARIE COLOR GRIGIO CHIARO CON LIVELLI SILTITICI COMPATTI (GONFOLITE)
ARENARIE COMPATTE
ARENARIE COMPATTE CON ALTERNATI LIVELLI DI MARNE
ARENARIE CON PICCOLI LIVELLI MARNOSI, ARENARIE STERILI - ARENARIA FRATTURATA ACQUIFERA
ARENARIE CON STRATI DI SABBIA E GHIAIA IDRICA
ARENARIE DEBOLMENTE CEMENTATE INTERCALATE DA SOTTILI ORIZZONTI BRECCIATI
ARENARIE E LIVELLI CONGLOMERATICI
ARENARIE E SABBIE PSEUDOCOERENTI ANIDRE
ARENARIE E SILTITI - FORMAZIONE "ARENARIE DI M. MODINO"
ARENARIE E SILTITI ARGILLOSI
ARENARIE E SILTITI FRATTURATE
ARENARIE E SILTITI POCO FRATTURATE
ARENARIE FINI QUARZOSO FELDSPATICHE GRIGIE
ARENARIE FORTEMENTE CEMENTATE - FACIES ARENACEA DELLA F. CASSANO SPINOLA
ARENARIE GRIGIE COMPATTE
ARENARIE GRIGIE COMPATTE INDIFFERENZIATE
ARENARIE INTENSAMENTE FRATTURATE CON VENE DI CALCITE - PRESENZA DI ACQUA
ARENARIE MEDIO-GROSSOLANE, ARENARIE C

In [ ]:
#all values containing a key word
keyW = 'SUOLO LIMOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

SUOLO LIMOSO - ARGILLOSO
SUOLO LIMOSO ARGILLOSO
SUOLO LIMOSO ROSSASTRO CON GHIAIA FINE


In [ ]:
#all values containing a key word
keyW = 'MATERIALE LIMOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

MATERIALE LIMOSO NON RICUPERATO


In [ ]:
#all values containing a key word
keyW = 'LIVELLO LIMOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

LIVELLO LIMOSO
LIVELLO LIMOSO / ARGILLOSO
LIVELLO LIMOSO E ARGILLOSO-LIMOSO


In [ ]:
#all values containing a key word
keyW = 'LIVELLO DI LIMO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

LIVELLO DI LIMO - SABBIOSO GRIGIO-AZZURRO.
LIVELLO DI LIMO CON SABBIA F, PLASTICO, GRIGIO.
LIVELLO DI LIMO GRIGIO CON ALLA BASE SABBIA OCRA.
LIVELLO DI LIMO SABBIOSO BRUNO.


In [ ]:
#all values containing a key word
keyW = 'DEPOSITO LIMOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

DEPOSITO LIMOSO
DEPOSITO LIMOSO - ARGILLOSO CON RARI CIOTTOLI
DEPOSITO LIMOSO ARGILLOSO CON SCARSO SCHELETRO SILTOSO
DEPOSITO LIMOSO SABBIOSO INGLOBANTE CORPI E LIVELLI CIOTTOLOSI E GHIAIOSI


In [ ]:
#all values containing a key word
keyW = 'ALTERNANZE DI LIMO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

ALTERNANZE DI LIMO ARGILLOSO E DI GHIAIA CON LIMO
ALTERNANZE DI LIMO ARGILLOSO GRIGIO SABBIOSO E LIMO ARGILLOSO NOCCIOLA, ARGILLA LIMOSA TORBOSA GRIGIA, SABBIA MEDIO FINE LIMOSA GRIGIA
ALTERNANZE DI LIMO COMPATTO POCO SABBIOSO E SABBIA FINISSIMA LIMOSA O POCO LIMOSA OCRA COLORAZIONE MARRONE SCURO A 23.85 M
ALTERNANZE DI LIMO CON ARGILLA E LIMO ARGILLOSO
ALTERNANZE DI LIMO CON SABBIA INGLOBANTI SCARSA SABBIA
ALTERNANZE DI LIMO E ARGILLA LIMOSA
ALTERNANZE DI LIMO E DI GHIAIA
ALTERNANZE DI LIMO E LIMO SABBIOSO
ALTERNANZE DI LIMO E SABBIA LIMOSA
ALTERNANZE DI LIMO GRIGIO CON PICCOLI STRATI DI GHIAIA SPORCA E LENTI ARGILLOSE GRIGIE
ALTERNANZE DI LIMO GRIGIO E GIALLASTRO
ALTERNANZE DI LIMO SABBIOSO CON ARGILLE LIMOSE GRIGIE
ALTERNANZE DI LIMO SABBIOSO DEB. GHIAIOSO NOCCIOLA E DI LIMO ARGILLOSO DEB. GHIAIOSO NOCCIOLA; GHIAIA IDEM C.S.
ALTERNANZE DI LIMO SABBIOSO DEBOLMENTE ARGILLOSO CON GHIAIA E CIOTTOLI
ALTERNANZE DI LIMO SABBIOSO E ARGILLA LIMOSA
ALTERNANZE DI LIMO SABBIOSO E SABBIA DEBOLME

In [ ]:
#all values containing a key word
keyW = 'ALTERNANZA DI LIMO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

ALTERNANZA DI LIMO ARGILLOSO E LIMO SABBIOSO BRUNO CON POCA GHIAIA ETEROMETRICA IN PREVALENZA ALTERATA
ALTERNANZA DI LIMO ARGILLOSO E LIMO SABBIOSO GRIGIO MOLTO UMIDO CON INTERLIVELLI DECIMETRICI DI TORBA E DI SABBIA LIMOSA SATURA
ALTERNANZA DI LIMO ARGILLOSO NOCCIOLA-AVANA, CONSISTENTE, E SABBIA FINE-FINISSIMA, LIMOSA, NOCCIOLA-GIALLASTRA. SONO PRESENTI CALCINELLI MILLIMETRICI. VELI DI OSSIDAZIONE OCRACEI ED INCLUSI DI MATERIALE ORGANICO NERASTRO, MILLIMETRICO.
ALTERNANZA DI LIMO E ARGILLA
ALTERNANZA DI LIMO E ARGILLA ORGANICA
ALTERNANZA DI LIMO E LENTI ARGILLOSE
ALTERNANZA DI LIMO E SABBIA FINE
ALTERNANZA DI LIMO SABBIOSO DEBOLMENTE ARGILLOSO A GHIAIA SABBIOSA
ALTERNANZA DI LIMO SABBIOSO E ARGILLOSO NOCCIOLA INGLOBANTE RARA GHIAIA ETEROGENEA E RARI CIOTTOLI (DIAM. MAX 15 CM)


In [ ]:
#all values containing a key word
keyW = 'CALCARI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

CALCARI - CALCARI MARNOSI - MARNE - CALCAREE
CALCARI - CALCARI MARNOSI - MARNE - CALCAREE / C.S. CON ZONE A PICCOLA FRATTURAZIONE E/O GIUNTI DI DILATAZIONE
CALCARI BIANCASTRI A RODOLITI
CALCARI CON ZONE DI ALTERAZIONE E FRATTURAZIONE DI COLORE GRIGIO CHIARO
CALCARI DI COLORE GRIGIO SCURO
CALCARI DOLOMITICO A TRATTI DECIMETRICI INTENSAMENTE FRATTURATO CON TRACCE DI ARGILLA
CALCARI E CALCARI MARNOSI NERI FRATTURATI.
CALCARI FRATTURATI
CALCARI FRATTURATI CON STRATI DI ARGILLA ROSSA
CALCARI FRATTURATI DI COLORE GRIGIO - NERO, CON PASSAGGI IN CUI IL COLORE E' VERDE.
CALCARI FRATTURATI DI COLORE GRIGIO - NERO.
CALCARI FRATTURATI GRIGIASTRI MOLTO TENACI, CON VENE CALCITICHE BIANCHE E SOTTILI LIVELLI DI ARGILLOSCISTI (F. M.TE ANTOLA)
CALCARI FRATTURATI GRIGIO-BIANCASTRI (F. M.TE ANTOLA)
CALCARI FRATTURATI GRIGIO-BIANCASTRI, CON VENE CALCITICHE BIANCHE (F. M.TE ANTOLA)
CALCARI FRATTURATI GRIGIO-BIANCASTRI, MOLTO TENACI (F. M.TE ANTOLA)
CALCARI FRATTURATI GRIGIO-BIANCASTRI, MOLTO TENACI, CON VEN

In [ ]:
#all values containing a key word
keyW = 'CEPPO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'
    

CEPPO
CEPPO A TRATTI FRATTURATO CON OSSIDAZIONI E ALTERAZIONI.
CEPPO ARG.SO COMP. CON POCA SAB.
CEPPO ARGILLOSO
CEPPO ARGILLOSO CON GHIAIA
CEPPO ARGILLOSO CON GHIAINO
CEPPO ARGILLOSO GRIGIO COMPATTO
CEPPO BIANCASTRO E GRIGIO CON POCA GHIAIA
CEPPO BIANCO ARGILLOSO COMPATTO
CEPPO BIANCO ARGILLOSO CON GHIAIA
CEPPO BIANCO COMPATTO
CEPPO BIANCO COMPATTO CON GHIAIA
CEPPO BIANCO COMPATTO CON LENTI DI CONGLOMERATO E POCA GHIAIA
CEPPO BIANCO COMPATTO CON POCA GHIAIA
CEPPO BIANCO COMPATTO CON POCO GHIAIETTO
CEPPO BIANCO COMPATTO E POCA GHIAIA
CEPPO BIANCO COMPATTO LIMOSO CON GHIAIA
CEPPO BIANCO CON GH.TTO E ARG. GRIGIA
CEPPO BIANCO CON GHIAIA
CEPPO BIANCO CON GHIAIA COMPATTA
CEPPO BIANCO CON GHIAIA MOLTO COMPATTO
CEPPO BIANCO CON GHIAIETTO
CEPPO BIANCO CON POCA GHIAIA
CEPPO BIANCO CON POCO GHIAIETTO
CEPPO BIANCO E GRIGIO
CEPPO BIANCO E GRIGIO COMPATTO
CEPPO BIANCO E GRIGIO CON POCA GHIAIA
CEPPO BIANCO E NOCCIOLA COMPATTO
CEPPO BIANCO E NOCCIOLA CON LENTI DI CONGLOMERATO
CEPPO BIANCO GRIGIO COMPA

In [ ]:
#all values containing a key word
keyW = 'CONGLOMERATO (CEPPO)'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

CONGLOMERATO (CEPPO)
CONGLOMERATO (CEPPO) A CEMENTO ARENACEO CON SCARSO GRADO DI CEMENTAZIONE. PRESENZA DI LENTE DI SABBIA LIMOSA COLOR OCRA DA 24.70 / 24.80.
CONGLOMERATO (CEPPO) A CEMENTO ARENACEO, COMPATTO CON RARE OSSIDAZIONI.
CONGLOMERATO (CEPPO) A CEMENTO ARENACEO, DEBOLMENTE CEMENTATO CON ORIZZONTE COMPATTO DA MT. 31.40 A 31.65.
CONGLOMERATO (CEPPO) A TRATTI BEN CEMENTATO A TRATTI FRATTURATO CON OSSIDAZIONI E ALTERAZIONI.
CONGLOMERATO (CEPPO) A TRATTI BEN CEMENTATO CON VACUOLI. PRESENZA DI OSSIDAZIONI.
CONGLOMERATO (CEPPO) A TRATTI BEN CEMENTATO, A TRATTI CON SCARSO GRADO DI CEMENTAZIONI, CON OSSIDAZIONI.
CONGLOMERATO (CEPPO) A TRATTI BEN CEMENTATO, A TRATTI FRATTURATO CON OSSIDAZIONI E ALTERAZIONI.
CONGLOMERATO (CEPPO) A TRATTI FRATTURATO CON SCARSO GRADO DI CEMENTAZIONE. PRESENZA DI OSSIDAZIONI E OSSIDAZIONI. [SIC]
CONGLOMERATO (CEPPO) BEN CEMENTATO A TRATTI CON SCARSO GRADO DI CEMENTAZIONE. PRESENZA DI ALTERAZIONI E OSSIDAZIONI.
CONGLOMERATO (CEPPO) BEN CEMENTATO A TRATTI FRA

In [ ]:
#all values containing a key word
keyW = 'argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

argilla
argilla azzurra
argilla cenere
argilla compatta
argilla con sabbia
argilla gialla grigia
argilla grigia compatta
argilla grigia con toorba
argilla grigia con torba
argilla grigia scura e torba
argilla limosa gialla
argilla torbosa
argilla verde


In [ ]:
#all values containing a key word
keyW = 'TROVANTI,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

TROVANTI, CIOTTOLI E GHIAIA GROSSOLANA
TROVANTI, CIOTTOLI E GHIAIETTO
TROVANTI, CIOTTOLI IN GHIAIA DA MEDIA A PICCOLA
TROVANTI, CIOTTOLI, GHIAIA E SABBIA COMPATTA
TROVANTI, GHIAIA E CIOTTOLI
TROVANTI, GHIAIA E CONGLOMERATO
TROVANTI, GHIAIONE, GHIAIA E SABBIA


In [ ]:
#all values containing a key word
keyW = 'TORBA,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

TORBA, ARGILLA
TORBA, FOSSILI E LIGNITE
TORBA, LIGNITE E GHIAIETTO
TORBA, LIGNITE, ARGILLA TORBOSA


In [ ]:
#all values containing a key word
keyW = 'TERRICCIO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

TERRICCIO
TERRICCIO - GHIAIA E SABBIA
TERRICCIO - GHIAIETTA E SABBIA
TERRICCIO AGRICOLO
TERRICCIO ARGILLOSO
TERRICCIO ARGILLOSO CON GHIAIA E CIOTTOLI.
TERRICCIO ARGILLOSO CON GHIAIONE
TERRICCIO CON TROVANTI
TERRICCIO DI COLTURA (HUMUS)
TERRICCIO E BRUTTO (?)
TERRICCIO E CIOTTOLI
TERRICCIO E GHIAIA
TERRICCIO E GHIAIETTO
TERRICCIO E SABBIETTA
TERRICCIO GHIAIETTA
TERRICCIO MISTO
TERRICCIO ROSSICCIO
TERRICCIO ROSSO ARGILLOSO
TERRICCIO SABBIOSO E PIETROSO
TERRICCIO SUPERFICIE


In [ ]:
#all values containing a key word
keyW = 'TERRENO VEGETALE:'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

TERRENO VEGETALE: ARGILLA LIMOSA GRIGIA CON OSSIDAZIONI OCRACEE E TORBA NERASTRA
TERRENO VEGETALE: LIMO ARGILLOSO GRIGIASTRO
TERRENO VEGETALE: LIMO ARGILLOSO NOCCIOLA,
TERRENO VEGETALE: LIMO ARGILLOSO TORBOSO GRIGIASTRO
TERRENO VEGETALE: LIMO-SABBIOSO
TERRENO VEGETALE: SABBIA FINE, LIMOSA, INGLOBANTE GHIAIA MEDIO-FINE E RESTI VEGETALI. COLORE BRUNO.


In [ ]:
#all values containing a key word
keyW = 'TERRENO VEGETALE,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

TERRENO VEGETALE, ALLUVIONI FINI DI FONDOVALLE FRAMMISTI A DETRITI DI VERSANTE
TERRENO VEGETALE, ARGILLA LIMOSA
TERRENO VEGETALE, CIOTTOLI E GHIAIE GROSSOLANE
TERRENO VEGETALE, CIOTTOLI, GHIAIE E SABBIE GROSSOLANE
TERRENO VEGETALE, COLTRE AGRARIA LIMOSO-ARGILLOSA E SUBORDINATAMENTE SABBIOSA DI COLORE BRUNO NOCCIOLA, DESTRUTTURATA SUPERFICIALMENTE DALLE INTENSE PRATICHE AGRICOLE E DALLE ALTERAZIONI INDOTTE DALLE VARIAZIONI CLIMATICHE E DAGLI ATMOSFERILI.
TERRENO VEGETALE, GHIAIE E SABBIE GROSSOLANE
TERRENO VEGETALE, SABBIA BLU-GRIGIASTRA FINE, TALORA DEBOLMENTE LIMOSA
TERRENO VEGETALE, SABBIOSO
TERRENO VEGETALE, TERRICCIO


In [ ]:
#all values containing a key word
keyW = 'TERRENO COLTIVO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

TERRENO COLTIVO
TERRENO COLTIVO A MEDIO IMPASTO
TERRENO COLTIVO ARGILLOSO
TERRENO COLTIVO ARGILLOSO.
TERRENO COLTIVO COLOR MARRONE
TERRENO COLTIVO CON GHIAIA E SABBIA ARGILLOSA
TERRENO COLTIVO CON GHIAIETTO
TERRENO COLTIVO E ARGILLA
TERRENO COLTIVO E GHIAIA
TERRENO COLTIVO LEGGERMENTE LIMOSO
TERRENO COLTIVO LIMOSO
TERRENO COLTIVO SABBIOSO
TERRENO COLTIVO SABBIOSO HUMACEO


In [ ]:
#all values containing a key word
keyW = 'TERRENO AGRICOLO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

TERRENO AGRICOLO
TERRENO AGRICOLO / GHIAIA CON CIOTTOLI E ARGILLA
TERRENO AGRICOLO COMPOSTA DA LIMO E RESTI VEGETALI. UMIDO E POCO ADDENSATO. DI COLORE MARRONE SCURO.
TERRENO AGRICOLO DI RIPORTO COSTITUITO DA ARGILLA E CIOTTOLI DI COLORE MARRONE NOCCIOLA
TERRENO AGRICOLO E INIZIO ACQUA
TERRENO AGRICOLO LIM-ARGILLOSO
TERRENO AGRICOLO LIMO-SABBIOSO GRIGIASTRO
TERRENO AGRICOLO LIMOSO GRIGIO VERDE CON VENATURE MARRONI
TERRENO AGRICOLO LIMOSO SABB.
TERRENO AGRICOLO LIMOSO-SABB.
TERRENO AGRICOLO LIMOSO-SABB. MARRONE VERDE-SCURO E NERASTRO
TERRENO AGRICOLO LIMOSO-SABBIOSO GRIGIO MARRONE-SCURO
TERRENO AGRICOLO LIMOSO-SABBIOSO MARRONE SCURO
TERRENO AGRICOLO PALUDOSO
TERRENO AGRICOLO SABB-LIMOSO
TERRENO AGRICOLO SABB. LIMOSO


In [ ]:
#all values containing a key word
keyW = 'TERRENO AGRARIO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

TERRENO AGRARIO
TERRENO AGRARIO (ELUVIALE COPERTURA LOESSICA)
TERRENO AGRARIO ARGILLOSO
TERRENO AGRARIO CON CIOTOLI
TERRENO AGRARIO CON CIOTTOLI
TERRENO AGRARIO CON RESTI VEGETALI
TERRENO AGRARIO CON SABBIA LIMOSA
TERRENO AGRARIO COSTITUITO DA DEPOSITI LIMOSO-ARGILLOSI.
TERRENO AGRARIO COSTITUITO DA LIMO SABBIOSO DI COLORE MARRONE, CON GHIAIETTO SPARSO E FRAMMENTI DI LATERIZI.
TERRENO AGRARIO DI RIPORTO
TERRENO AGRARIO E ARGILLA GIALLA
TERRENO AGRARIO E CIOTTOLI
TERRENO AGRARIO E LIMI ARGILLOSI
TERRENO AGRARIO E LIMI ARGILLOSI OCRACEI;
TERRENO AGRARIO E LIMI ARGILLOSI OCRACEO
TERRENO AGRARIO E LIMI ARGILLOSI;
TERRENO AGRARIO E LIMO
TERRENO AGRARIO E SUOLO
TERRENO AGRARIO E VEGETALE
TERRENO AGRARIO E/O RIPORTO
TERRENO AGRARIO ESSENZIALMENTE LIMOSO
TERRENO AGRARIO LIMO - ARGILLOSO / COLTRE DI ORIGINE LOESSICA
TERRENO AGRARIO LIMO - ARGILLOSO DI COLORE NOCCIOLA
TERRENO AGRARIO LIMOSO
TERRENO AGRARIO LIMOSO-ARGILLOSO
TERRENO AGRARIO LIMOSO-SABBIOSO
TERRENO AGRARIO O RIPORTO
TERRENO AGRARIO

In [ ]:
#all values containing a key word
keyW = 'SUOLO VEGETALE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

SUOLO VEGETALE
SUOLO VEGETALE COSTITUITO DA LIMI SABBIOSO ARGILLOSI BRUNI CON RESTI VEGETALI E NUMEROSI CLASTI ETEROMETRICI
SUOLO VEGETALE LIMOSO, BRUNASTRO, CON RESTI VEGETALI.
SUOLO VEGETALE MARRONE SCURO.


In [ ]:
#all values containing a key word
keyW = 'SUOLO AGRARIO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

SUOLO AGRARIO
SUOLO AGRARIO ARGILLOSO
SUOLO AGRARIO ARGILLOSO BRUNO CON CIOTTOLI
SUOLO AGRARIO ARGILLOSO BRUNO.
SUOLO AGRARIO ARGILLOSO ROSSICCIO
SUOLO AGRARIO DI COPERTURA
SUOLO AGRARIO E SUBSTRATO ARGILLOSO


In [ ]:
#all values containing a key word
keyW = 'SUBSTRATO ROCCIOSO,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

SUBSTRATO ROCCIOSO, ATTARVERSATO DA LIVELLI MOLTO ALTERATI
SUBSTRATO ROCCIOSO, COSTITUITO DALLA FORMAZIONE DEL FLYSCH DI M.TE ANTOLA: INTERCALAZIONI DI ARENARIE, ARGILLOSCISTI, CALCARI MARNOSI E CALCARI GRIGI, TENACI
SUBSTRATO ROCCIOSO, FRATTURATO E ALTERATO, ATTRAVERSATO DA SUBORDINATI LIVELLI MOLTO ALTERATI


In [ ]:
#all values containing a key word
keyW = 'SUBSTRATO ROCCIOSO:'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

SUBSTRATO ROCCIOSO: BRECCIA POCO CEMENTATA COSTITUITA DA ELEMENTI LAPIDEI DI VARIA PEZZATURA (DIAMETRO MAX = 4 - 5 CM), SPIGOLOSI, DI PREVALENTI SERPENTINITI ED OFIOLITI, COLOR GRIGIO SCURO GRIGIO VERDASTRO, QUALCHE TROVANTE DIAMETRO = 10 - 15 CM, DA MATRICE A CLASTO SOSTENUTI. MATRICE LEGANTE ARENACEA, SILTOSO ARGILLOSA COLOR GRIGIO VERDE.
SUBSTRATO ROCCIOSO: CALCARI MARNOSI (MEDOLO)
SUBSTRATO ROCCIOSO: CALCARI MARNOSI CON INTERCALAZIONI DI MARNA ARGILLOSA (FORMAZIONE DI CONCESIO)


In [ ]:
#all values containing a key word
keyW = 'SUBSTRATO ROCCIOSO.'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

SUBSTRATO ROCCIOSO. SI TRATTA DI GNEISS A PIGMENTO CARBONIOSO PROFONDAMENTE ALTERATO E FRATTURATO IN SUPERFICIE.


In [ ]:
#all values containing a key word
keyW = 'STRATO GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

STRATO GHIAIA
STRATO GHIAIA CON ACQUA
STRATO GHIAIA E SABBIA
STRATO GHIAIA MARNE GRIGIO-TERRA E GHIAIA


In [ ]:
#all values containing a key word
keyW = 'sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

sabbia
sabbia da media a grossa
sabbia e ghiaia
sabbia fine
sabbia fine grigia
sabbia fine leggermente limosa
sabbia fine limosa
sabbia fine limosa con ghiaietto
sabbia ghiaia e ciottoli
sabbia grigia
sabbia leggermente limosa
sabbia limosa con ghiaia
sabbia meda
sabbia media
sabbia mista ghiaia
sabbia rossiccia


In [ ]:
#all values containing a key word
keyW = 'STRATO ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

STRATO ARGILLA BLU
STRATO ARGILLA DURA
STRATO ARGILLA GIALLA
STRATO ARGILLA GRIGIA
STRATO ARGILLA SABBIOSA GIALLA


In [ ]:
#all values containing a key word
keyW = 'STRATI DI ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

STRATI DI ARGILLA CON GHIAIA CEMENTATA
STRATI DI ARGILLA CON LIMO GIALLO
STRATI DI ARGILLA E ARENARIA
STRATI DI ARGILLA E GHIAIA
STRATI DI ARGILLA E LIMO
STRATI DI ARGILLA E ROCCIA
STRATI DI ARGILLA E SABBIA
STRATI DI ARGILLA E SABBIA CON ARENARIA
STRATI DI ARGILLA E SABBIA FINE
STRATI DI ARGILLA GIALLA (SPESSORE CM. 40 / 60) ALTERNATI A SABBIA FINISSIMA.
STRATI DI ARGILLA GRIGIA E SABBIA
STRATI DI ARGILLA GRIGIA INTERVALLATA DA STRATI DI SABBIA FINE
STRATI DI ARGILLA PLASTICA CON ALTERNANZE DI LIVELLI SABBIOSI
STRATI DI ARGILLA SABBIA E TORBA


In [ ]:
#all values containing a key word
keyW = 'STRATI ALTERNATI DI GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

STRATI ALTERNATI DI GHIAIA 10-30 CM CON LIMO COMPATTO
STRATI ALTERNATI DI GHIAIA COMPATTA, CON SABBIA GROSSA.
STRATI ALTERNATI DI GHIAIA E ARGILLA
STRATI ALTERNATI DI GHIAIA E ARGILLA GIALLA
STRATI ALTERNATI DI GHIAIA E TERRA


In [ ]:
#all values containing a key word
keyW = 'SD) ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

SD) ARGILLA COMPATTA CON GHIAIA
SD) ARGILLA GIALLA MOLTO COMPATTA
SD) ARGILLA GIALLA MOLTO COMPATTA MISTA GHIAIA.
SD) ARGILLA MOLTO COMPRESSA MISTA GHIAIA.


In [ ]:
#all values containing a key word
keyW = 'SBBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SBBIA FINE
SBBIA FINE CON LENTI DI ARGILLA
SBBIA FINE CON LIVELLI D'ARGILLA
SBBIA FINE GRIGIA
SBBIA FINE LIMOSA
SBBIA LIMOSA
SBBIA MEDIA GRANA
SBBIA MEDIA, LA QUOTA FINISCE A M. 48
SBBIA MEDIO FINE CON LIMO
SBBIA MEDIO FINE LIMOSA GRIGIA,LIVELLETTI DI LIMO DEBOLMENTE ARGILLOSO GRIGIO
SBBIA MEDIO-FINE GRIGIA
SBBIA MEDIO-FINE LIMOSA


In [ ]:
#all values containing a key word
keyW = 'SABIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABIA CON GHIAIETTO
SABIA CON RESIDUI VEGETALI
SABIA FINE
SABIA FINE A TRATTI LIMOSA CIOTTOLI
SABIA LIMACCIOSA CON SASSOLINI
SABIA MEDIA GRIGIA INGL. GHIAIA E CIOTTOLI. LOC. PRES. TROVANTI GNEISSICI
SABIA MEDIO FINE GRIGIA
SABIA MISTA A GHIAIA


In [ ]:
#all values containing a key word
keyW = 'SABBIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIOSO


In [ ]:
#all values containing a key word
keyW = 'SABBIOSI,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIOSI, SABBIOSI LIMOSI


In [ ]:
#all values containing a key word
keyW = 'SABBIOSA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIOSA
SABBIOSA ARGILLOSA GIALLA COMPATTA E ARENARIA
SABBIOSA FINE COMPATTA E ARENARIA
SABBIOSA LIMOSA


In [ ]:
#all values containing a key word
keyW = 'SABBIONE,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIONE, ARGILLA, LEGNO


In [ ]:
#all values containing a key word
keyW = 'SABBIONCINO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIONCINO SILICEO


In [ ]:
#all values containing a key word
keyW = 'SABBIOA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIOA GROSSA


In [ ]:
#all values containing a key word
keyW = 'SABBIOA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

In [ ]:
#all values containing a key word
keyW = 'SABBIETTO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIETTO SOTTILE


In [ ]:
#all values containing a key word
keyW = 'SABBIASABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIASABBIA


In [ ]:
#all values containing a key word
keyW = 'SABBIAI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIAI FINE
SABBIAI GROSSA GHIAIOSA
SABBIAI MEDIA


In [ ]:
#all values containing a key word
keyW = 'SABBIAGROSSA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIAGROSSA


In [ ]:
#all values containing a key word
keyW = 'SABBIAE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIAE GHIAIA


In [ ]:
#all values containing a key word
keyW = 'SABBIACON'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIACON GHIAIETTO GHIAIAN E ARGILLA
SABBIACON LIMO


In [ ]:
#all values containing a key word
keyW = 'SABBIA;'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIA;


In [ ]:
#all values containing a key word
keyW = 'SABBIA:'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIA: GROSSOLANA MARRONE OCRA
SABBIA: MEDIO-GROSSOLANA DI COLORE MARRONE OCRA


In [ ]:
#all values containing a key word
keyW = 'SABBIA/TERRENO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIA/TERRENO


In [ ]:
#all values containing a key word
keyW = 'SABBIA/ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIA/ARGILLA


In [ ]:
#all values containing a key word
keyW = 'SABBIA.'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIA.


In [ ]:
#all values containing a key word
keyW = 'SABBIA-MEDIO-GROSSA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIA-MEDIO-GROSSA


In [ ]:
#all values containing a key word
keyW = 'SABBIA-MEDIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIA-MEDIA


In [ ]:
#all values containing a key word
keyW = 'SABBIA-LIMO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIA-LIMO


In [ ]:
#all values containing a key word
keyW = 'SABBIA-GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIA-GHIAIA CON CIOTTOLI E TROV.


In [ ]:
#all values containing a key word
keyW = 'SABBIA-ARGILLOSA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIA-ARGILLOSA


In [ ]:
#all values containing a key word
keyW = 'SABBIA-ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBIA-ARGILLA E LIMO MARRONI INGLOBANTI LIVELLI DECIMETRICI DI GHIAIA FINE. PRESENZA DI SOTTILI ORIZZONTI TORBOSI


In [ ]:
#all values containing a key word
keyW = 'SABBBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBBIA FINE
SABBBIA FINE CON ARGILLA COMPATTA
SABBBIA FINE GIALLOGNOLA


In [ ]:
#all values containing a key word
keyW = 'SABBA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SABBA ARGILLOSA ROSSASTRA
SABBA MEDIO-FINE SCIOLTA CON GHIAIA MEDIO GROSSOLANA. C. GRIGIO.


In [ ]:
#all values containing a key word
keyW = 'SAB.'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SAB. FINE / LIMO
SAB. GIAL. F. CON LENTI ARG. E TRACCE GH.
SAB. GIAL. MEDIA F. CON TRACCE DI GH.


In [ ]:
#all values containing a key word
keyW = 'SAABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

SAABBIA FINE


In [ ]:
#all values containing a key word
keyW = 'S.'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

S. FINE CON LENTI DI ARGILLA
S. FINE CON PIRITE E LIMO ARGILLOSO
S. FINE LIMOSA
S. GROSSA CON CIOTTOLI E LIMO
S. MEDIO FINE


In [ ]:
#all values containing a key word
keyW = 'ROCCIA CALCAREA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

In [ ]:
#all values containing a key word
keyW = 'ROCCIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

ROCCIA
ROCCIA (GNEISS)
ROCCIA (GONFOLITE)
ROCCIA (NON TEREBRATA)
ROCCIA (S.D. M. 35)
ROCCIA (VERDE)
ROCCIA (VULCANITI) ALTERATA
ROCCIA A STRATI
ROCCIA A STRATI COLORE BLU
ROCCIA A STRUTTURA CRISTALLINA SACCAROIDE DI COLORE GRIGIO CHIARO - BIANCO. A 115.10 MT. FRATTURA INCLINATA DI CIRCA 45°.
ROCCIA A TRATTI FESSURATA FACILMENTE LAVORABILE
ROCCIA ALTERATA
ROCCIA ALTERATA ASSIMILABILE A SABBIA COMPATTA
ROCCIA ALTERATA INGLOBANTE ELEMENTI LAPIDEI INALTERATI
ROCCIA ALTERATA: BRECCIA CALCAREA IN ARGILLA.
ROCCIA ARENACEA COMPATTA DA VERDE A GRIGIO-VERDE
ROCCIA ARENACEA GRIGIO VERDE COMPATTA E DEBOLMENTE FRATTURATA
ROCCIA ARENACEA GRIGIO-VERDASTRA COMPATTA
ROCCIA ARENACEA GRIGIO-VERDE COMPATTA
ROCCIA ARENACEA VERDE SCURO COMPATTA
ROCCIA AVORIO FRATTURATA
ROCCIA BIANCA
ROCCIA BIANCA COMPATTA
ROCCIA BIANCA COMPATTA CON PICCOLE FESSURAZIONI
ROCCIA BIANCA CON ARGILLITI E PICCOLE FESSURE
ROCCIA BIANCA CON GRANDI FESSURAZIONI
ROCCIA BIANCA CON SOTTILI STRATI DI ROCCIA FRATTURATA COLOR ROSASTRO
ROCC

In [ ]:
#all values containing a key word
keyW = 'ROCCE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

ROCCE FRATTURATE DI NATURA CALCAREA O CALCAREO DOLOMITICA (CALCARE DI ZORZINO)
ROCCE GRANITICHE DEBOLMENTE ALTERATE / ROCCIA INTEGRA
ROCCE GRANITICHE MOLTO ALTERATE
ROCCE METAMORFICHE BIOTITICHE.
ROCCE METAMORFICHE BIOTITICHE;
ROCCE METAMORFICHE REGIONALI (MICASCISTI) DI CONSISTENZA LAPIDEA;
ROCCE METAMORFICHE REGIONALI DI CONSISTENZA BASSA;
ROCCE SCISTOSE PELITICHE;


In [ ]:
#all values containing a key word
keyW = 'RIPORTO: SABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

RIPORTO: SABBIA CON GHIAIA E RESTI LATERITICI
RIPORTO: SABBIA CON GHIAIA LIMOSA, SCIOLTA, COLORE MARRONE; PRESENZA DI ABBONDANTI RESTI DI LATERIZI
RIPORTO: SABBIA DA FINE A GROSSOLANA LIMOSA GRIGIA CON GHIAIA GHIAIETTO CIOTTOLI E RESTI LATERITICI
RIPORTO: SABBIA DEB. LIMOSA A PIGMENTAZIONE NERASTRA INGL. ED ELEMENTI DI RIPORTO (VETRO PLASTICA)
RIPORTO: SABBIA DEB. LIMOSA NOCCIOLA INGL. RARA GHIAIA E GHIAIETTO
RIPORTO: SABBIA FINE BRUNA CON RESTI DI LATERIZI
RIPORTO: SABBIA FINE CON LIMO CON CALCINACCI E CIOTTOLI
RIPORTO: SABBIA FINE DEB. LIMOSA MARRONE CON GHIAIA
RIPORTO: SABBIA FINE LIMOSA BRUNA INGL. GHIAIA ETEROMETRICA CALCESTRUZZO E RESTI LATERITICI
RIPORTO: SABBIA FINE LIMOSA MARRONE INGL. RARA GHIAIA MEDIO FINE
RIPORTO: SABBIA GRIGIASTRA CON GHIAIA
RIPORTO: SABBIA GRIGIASTRA CON GHIAIA CIOTTOLI E RESTI LATERITICI
RIPORTO: SABBIA GRIGIASTRA CON GHIAIA E RESTI LATERITICI
RIPORTO: SABBIA GRIGIASTRA DEB. CEMENTATA CON GHIAIA GHIAIETTO CIOTTOLI E RESTI LATERITICI
RIPORTO: SABBIA GRIGI

In [ ]:
#all values containing a key word
keyW = 'PREVALENZA DI SABBIE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

PREVALENZA DI SABBIE
PREVALENZA DI SABBIE FINI LIMOSE COLORE GRIGIO-NOCCIOLA E GRIGIO
PREVALENZA DI SABBIE MEDIE E FINI GRIGIE
PREVALENZA DI SABBIE MEDIE GRIGIE
PREVALENZA DI SABBIE MEDIO FINI GRIGIE NOCCIOLA CON GHIAIE E CIOTTOLI


In [ ]:
#all values containing a key word
keyW = 'PREVALENZA DI SABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

In [ ]:
#all values containing a key word
keyW = 'REVALENZA DI SABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

In [ ]:
#all values containing a key word
keyW = 'PREVALENZA DI ARGILLE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

PREVALENZA DI ARGILLE
PREVALENZA DI ARGILLE GRIGIE
PREVALENZA DI ARGILLE LIMOSE
PREVALENZA DI ARGILLE LIMOSE CON LIMITATI LIVELLI DI SABBIA


In [ ]:
#all values containing a key word
keyW = 'PREVALENZA DI ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

PREVALENZA DI ARGILLA
PREVALENZA DI ARGILLA DEBOLMENTE LIMOSA GRIGIA
PREVALENZA DI ARGILLA E ARGILLA LIMOSA GRIGIA
PREVALENZA DI ARGILLA E LIMO
PREVALENZA DI ARGILLA GRIGIA
PREVALENZA DI ARGILLA GRIGIA CON LIVELLI TORBOSI E LIVELLI SABBIOSI


In [ ]:
#all values containing a key word
keyW = 'PORFIDO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

PORFIDO
PORFIDO (SUBSTRATO ROCCIOSO)
PORFIDO COMPATTO ROSSO
PORFIDO COMPATTO ROSSO ALTERNATO A LENTI DI PORFIDO FRATTURATO
PORFIDO CON FRATTURE E FESSURE / VENUTE DI ACQUA
PORFIDO FRATTURATO
PORFIDO FRATTURATO / DEBOLI VENUTE DI ACQUA
PORFIDO ROSSO COMPATTO


In [ ]:
#all values containing a key word
keyW = 'MISTO GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

MISTO GHIAIA CON FINI ARGILLOSI
MISTO GHIAIA E PIETRE
MISTO GHIAIA GIALLA
MISTO GHIAIA GROSSA E PICCOLA CON SABBIA
MISTO GHIAIA SECCA


In [ ]:
#all values containing a key word
keyW = 'MATERIALE GHIAIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

MATERIALE GHIAIOSO
MATERIALE GHIAIOSO COMPATTO CON GROSSI TROVANTI
MATERIALE GHIAIOSO DURO CON GROSSI TROVANTI
MATERIALE GHIAIOSO GRASSO
MATERIALE GHIAIOSO GRASSO CON CONGLOMERATI
MATERIALE GHIAIOSO MAGRO
MATERIALE GHIAIOSO PIU' TENERO CON GROSSI TROVANTI


In [ ]:
#all values containing a key word
keyW = 'LIVELLO DI GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

LIVELLO DI GHIAIA A GRANULOMETRIA MEDIA, MAGGIORMENTE COMPATTA RISPETTO A QUELLO SUPERIORE, CON ALL'INTERNO ELEMENTI ETEROGENEI (DIAMETRO MEDIO DI CIRCA 10 - 12 CM), DI COLORE VARIABILE DA GRIGIO A MARRONCINO
LIVELLO DI GHIAIA MEDIO FINE, BEN COMPATTA, CON ALL'INTERNO ELEMENTI ARROTONDATI DI DIAMETRO MAX PARI A 5-7 CM IMMERSI IN UN'ABBONDANTE MATRICE FINE LIMOSO-SABBIOSA, DI COLORE DA MARRONCINO CHIARO A GIALLOGNOLO
LIVELLO DI GHIAIA PULITA
LIVELLO DI GHIAIA SCIOLTA CON CIOTTOLI, A ELEMENTI BEN ARROTONDATI MODERATAMENTE GROSSOLANI (DIAMETRO MEDIO 4-5 CM, FINO A MAX DI 10 - 12 CM), DI COLORE DA GRIGIO A BIANCASTRO, IN DEBOLE MATRICE FINE PER LO PIU' SABBIOSO-LIMOSA


In [ ]:
#all values containing a key word
keyW = 'LIVELLI LIMOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

LIVELLI LIMOSO ARGILLOSI DISCONTINUI
LIVELLI LIMOSO SABBIOSI
LIVELLI LIMOSO SABBIOSI E SABBIOSI A COLORAZIONE ROSSICCIA;
LIVELLI LIMOSO SABBIOSI E SABBIOSO GHIAIOSI A COLORAZIONE ROSSICCIA;


In [ ]:
#all values containing a key word
keyW = 'LIMO-ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

LIMO-ARGILLOSO
LIMO-ARGILLOSO CON PASSATE DI SILTITE FINE DI COLORE DA AZZURRO FINO A NERASTRO (DEPOSITI VARVATI?);
LIMO-ARGILLOSO NOCCIOLA
LIMO-ARGILLOSO ROSSO E GIALLO


In [ ]:
#all values containing a key word
keyW = 'LENTI DI ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

LENTI DI ARGILLA
LENTI DI ARGILLA CON GHIAIA E SABBIA
LENTI DI ARGILLA CON SABBIA FINE
LENTI DI ARGILLA E GHIAIA
LENTI DI ARGILLA E SABBIA
LENTI DI ARGILLA E TORBA
LENTI DI ARGILLA GIALLA


In [ ]:
#all values containing a key word
keyW = 'LENTI ARGILLOSE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

LENTI ARGILLOSE
LENTI ARGILLOSE - SABBIOSE E GHIAIA
LENTI ARGILLOSE CON GHIAIA E CONGLOMERATO
LENTI ARGILLOSE CON POCA GHIAIA
LENTI ARGILLOSE CON SABBIA LIMOSA SCURA


In [ ]:
#all values containing a key word
keyW = 'LENTE DI SABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

LENTE DI SABBIA
LENTE DI SABBIA FINE
LENTE DI SABBIA LIMOSA


In [ ]:
#all values containing a key word
keyW = 'IMPASTO DI ARGILLA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

IMPASTO DI ARGILLA E GHIAIA
IMPASTO DI ARGILLA E GHIAIA ROSSASTRA
IMPASTO DI ARGILLA ROSSASTRA, GHIAIA E TROVANTI


In [ ]:
#all values containing a key word
keyW = 'GROSSI BLOCCHI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

GROSSI BLOCCHI (DIAMETRO 300-400 MM)
GROSSI BLOCCHI (TROVANTI)
GROSSI BLOCCHI (TROVANTI), IN MATRICE PARZIALMENTE CEMENTATA (CONGLOMERATO)
GROSSI BLOCCHI E TROVANTI PARZIALMENTE ALTERATI


In [ ]:
#all values containing a key word
keyW = 'GROSSI BLOCCHI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

In [ ]:
#all values containing a key word
keyW = 'GRANITI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

GRANITI ALTERATI (CAPPELLACCIO)
GRANITI COMPATTI
GRANITI NOTEVOLMENTE FRATTURATI


In [ ]:
#all values containing a key word
keyW = 'GHIAIONE,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIONE, CIOTTOLI E SABBIA
GHIAIONE, CON MOLTA ARGILLA
GHIAIONE, GHIAIA
GHIAIONE, GHIAIA CON POCA SABBIA
GHIAIONE, GHIAIA E SABBIA
GHIAIONE, GHIAIA, ARGILLA
GHIAIONE, GHIAIA, LIMO CON ELEMENTI POLIGENICI
GHIAIONE, GHIAIETTO E CIOTTOLI
GHIAIONE, GHIAIETTO E SABBIONE
GHIAIONE, GHIAIETTO MISTO A SABBIA CON LENTI DI ARENARIA
GHIAIONE, GHIAIETTO, PIETRA NERA CON ARGILLA BLU SABBIOSA (TRACCE)
GHIAIONE, SABBIA CIOTTOLI


In [ ]:
#all values containing a key word
keyW = 'GHIAINO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAINO
GHIAINO COMPATTO CON ARGILLA
GHIAINO COMPATTO CON CEPPO BIANCO
GHIAINO COMPATTO CON LIMO GRIGIO
GHIAINO CON ARGILLA
GHIAINO CON CEPPO BIANCO
GHIAINO CON CEPPO BIANCO E GRIGIO
GHIAINO CON CEPPO NOCCIOLA E TROVANTI
GHIAINO CON CONGLOMERATO
GHIAINO CON LIMO BIANCO GRIGIO
GHIAINO CON LIMO GRIGIO
GHIAINO CON POCO CEPPO NOCCIOLA
GHIAINO CON SABBIA
GHIAINO CON SABBIA CEMENTATA
GHIAINO CON SABBIA E ARGILLA
GHIAINO CON SABBIA GROSSA
GHIAINO CON SABBIA MEDIA-GROSSA
GHIAINO CON STRATI DI ARGILLA
GHIAINO CON STRATI DI CONGLOMERATO
GHIAINO E POCA LITTA CENERE
GHIAINO E SABBIA
GHIAINO E SABBIA CON PRESENZA DI ARGILLA E CONCHIGLIE
GHIAINO E SABBIA PRESSATA
GHIAINO FINE
GHIAINO GRIGIO - ACQUIFERO
GHIAINO MISTO CON SABBIA
GHIAINO MISTO SABBIA
GHIAINO SABBIOSO
GHIAINO SPORCO


In [ ]:
#all values containing a key word
keyW = 'GHIAIETTINO,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIETTINO, SABBIA GROSSA, LENTI D'ARGILLA
GHIAIETTINO, SABBIA MEDIO-SCURA


In [ ]:
#all values containing a key word
keyW = 'GHIAIETTINO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIETTINO COMPATTO
GHIAIETTINO COMPATTO MOLTO ARGILLOSO
GHIAIETTINO CON GHIAIA E SABBIA CHIARA
GHIAIETTINO CON SABBIA
GHIAIETTINO CON SABBIA E TRACCE DI LIGNITE
GHIAIETTINO CON SABBIA GRIGIA
GHIAIETTINO CON SABBIA GROSSA
GHIAIETTINO CON SABBIA GROSSA GRIGIA
GHIAIETTINO CON SABBIA MEDIA GRIGIA
GHIAIETTINO E SABBIA GROSSA
GHIAIETTINO E SABBIA MEDIA GIALLASTRA
GHIAIETTINO GIALLASTRO PULITO
GHIAIETTINO GRIGIASTRO (PROVA)
GHIAIETTINO GRIGIASTRO POI GIALLASTRO
GHIAIETTINO GRIGIO E GIALLO
GHIAIETTINO VIVO CON POCA SABBIA GRIGIA


In [ ]:
#all values containing a key word
keyW = 'GHIAIETTI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIETTI E GHIAIE CON INTERCALAZIONI SABBIOSO-ARENACEE.
GHIAIETTI E GHIAIE MINUTE (MAX 2 - 3 CM) CON SABBIA.
GHIAIETTI E SABBIE
GHIAIETTI E SABBIE CON ACQUA


In [ ]:
#all values containing a key word
keyW = 'GHIAIAI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIAI CON ARGILLA GIALLA
GHIAIAI E GHIAIETTO
GHIAIAI E SABBIA


In [ ]:
#all values containing a key word
keyW = 'GHIAIAETTO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIAETTO CON PICCOLI STRATI DI ARGILLA GIALLA
GHIAIAETTO CON SABBIA
GHIAIAETTO CON SABBIA FINE LIMOSA
GHIAIAETTO PULITO


In [ ]:
#all values containing a key word
keyW = 'GHIAIA:'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIA: CON CIOTTOLI DA MILLIMETRICI A DECIMETRICI DI COLORE GRIGIASTRO.
GHIAIA: DI COLORE GRIGIO CON POCA MATRICE SABBIOSA E CIOTTOLI CENTIMETRICI


In [ ]:
#all values containing a key word
keyW = 'GHIAIA.'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAIA.
GHIAIA. PERDITE DI CIRCOLAZIONE


In [ ]:
#all values containing a key word
keyW = 'GHIAAI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHIAAI
GHIAAI ARGILLOSA
GHIAAI COMPATTA
GHIAAI CON ARGILLA
GHIAAI CON ARGILLA SABBIOSA DEBOLMENTE CIOTTOLOSA
GHIAAI CON CIOTOLI
GHIAAI CON CONGLOMERATO
GHIAAI CON LENTI DI ARGILLA
GHIAAI CON SABBAI
GHIAAI E ARGILLA
GHIAAI E CIOTTOLI
GHIAAI E SABBIA ARGILLOSA GIALLASTRA
GHIAAI E SABBIA ASCIUTTA
GHIAAI E SABBIA CON ACQUA
GHIAAI E SABBIA GIALLO ROSSA MOLTO ARGILLOSA, PICCOLE TRACCE DI LIMO
GHIAAI GROSSA CON ARGILLA E SABBIA
GHIAAI GROSSA E CIOTTOLI
GHIAAI GROSSOLANA
GHIAAI IDRICA CON ARGILLA
GHIAAI IDRICA CON STRATI DI ARGILLA
GHIAAI MISTA CON TERRA
GHIAAI SABBIA LEGGERMENTE ARGILLOSA
GHIAAI SABBIOSO LIMOSA
GHIAAI SPORCA


In [ ]:
#all values containing a key word
keyW = 'GHAIIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHAIIA ARGILLOSA ADDENSATA
GHAIIA E CIOTTOLI
GHAIIA E CIOTTOLI CON SABBIA GROSSA GRIGIA (LS. 57)


In [ ]:
#all values containing a key word
keyW = 'GHAIAI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GHAIAI
GHAIAI COMPATTA
GHAIAI CON SABBIA
GHAIAI SABBIA E CIOTTOLI


In [ ]:
#all values containing a key word
keyW = 'GH.'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

GH. COMP. CON MOLTO LIO GR. E C. B.
GH. COMP. CON POCO CEP. BIANCO GR.
GH. COMPATTA CON SAB. MARRONE-GRIGIA
GH. E ARG. LIMOSA GRIGIA A STRATI
GH. E CIOTTOLI CON P. SAB. GRI. COMP.
GH. E CIOTTOLI CON SAB. COMP. IN MATR. ARGILL.
GH. GROSSA MOLTO COMPATTA


In [ ]:
#all values containing a key word
keyW = 'FORMAZIONE MARNOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

FORMAZIONE MARNOSO ARENACEA FLYSH MARNOSO ARENACEO STRATI ARENACEI ALTERNATI A SILTITI E MARNE
FORMAZIONE MARNOSO ARENACEA ROMAGNOLA: ARENARIE E SILTITI GRADATE QUARZOSO FELDSPATICHE, MICACEE E DOLOMITICHE, ALTERNATE IN MODO ABBASTANZA REGOLARE, A STRATI DI MARNE SILTOSE. LO SPESSORE DEGLI...
FORMAZIONE MARNOSO ARENACEA ROMAGNOLA: FORMAZIONE TORBIDITICA COSTITUITA DA ARENARIE E SILTITI GRADATE QUARZOSO FELDSPATICHE, MICACCEE E DOLOMITICHE ALTERNATE A STRATI DI MARNE SILTOSE. QUEST'ULTIME..
FORMAZIONE MARNOSO ARENACEA ROMAGNOLA: SI HANNO ARENARIE E SILTITI, GRADATE, QUARZOSO FELDSPATICHE, MICACEE E DOLOMITICHE, ALTERNATE IN MODO ASSAI REGOLARE A STRATI DI MARNE SILTOSE. GLI STRATI...
FORMAZIONE MARNOSO ARENACEA: ALTERNANZE RITMICHE DI ARENARIE E SILTITI RISEDIMENTATE DI MARNE E DI ARGILLITI; LE SILTITI, DI COLORE GRIGIO, RISULTANO PREVALENTI. TRA GLI 33-36 METRI E I 65-70 M...
FORMAZIONE MARNOSO ROMAGNOLA: SI HANNO ARENARIE E SILTITI GRADATE QUARZOSO FELDSPATICHE, MICACEE E DOLOMITICHE,

In [ ]:
#all values containing a key word
keyW = 'DESCRIZIONE LITOLOGICA NON PRESENTE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=900
    key_list[position]='Altro'

DESCRIZIONE LITOLOGICA NON PRESENTE
DESCRIZIONE LITOLOGICA NON PRESENTE (APPROFONDIMENTO POZZO PREESISTENTE)
DESCRIZIONE LITOLOGICA NON PRESENTE (ARGILLA BLU?)
DESCRIZIONE LITOLOGICA NON PRESENTE (ARGILLA COMPATTA?)
DESCRIZIONE LITOLOGICA NON PRESENTE (ARGILLA?)
DESCRIZIONE LITOLOGICA NON PRESENTE (FONDO POZZO)
DESCRIZIONE LITOLOGICA NON PRESENTE (GHIAIETTO E SABBIA?)
DESCRIZIONE LITOLOGICA NON PRESENTE (POZZO PREESISTENTE)
DESCRIZIONE LITOLOGICA NON PRESENTE (SECONDO PIANO INTERRATO)
DESCRIZIONE LITOLOGICA NON PRESENTE - APPROFONDIMENTO POZZO PREESISTENTE
DESCRIZIONE LITOLOGICA NON PRESENTE - AVAMPOZZO ESISTENTE
DESCRIZIONE LITOLOGICA NON PRESENTE - AVANPOZZO ESISTENTE (SIGILLATO CON ARGILLA)
DESCRIZIONE LITOLOGICA NON PRESENTE - AVANPOZZO PREESISTENTE
DESCRIZIONE LITOLOGICA NON PRESENTE - POZZO A CAMICIA PRE-ESISTENTE
DESCRIZIONE LITOLOGICA NON PRESENTE - POZZO ESISTENTE
DESCRIZIONE LITOLOGICA NON PRESENTE - POZZO ESISTENTE - STRATIGRAFIA SCONOSCIUTA
DESCRIZIONE LITOLOGICA NON PRESEN

In [ ]:
#all values containing a key word
keyW = 'DEPOSITO LIMOSO-ARGILLOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

DEPOSITO LIMOSO-ARGILLOSO
DEPOSITO LIMOSO-ARGILLOSO CON ABBONDANTI CIOTTOLI DI DIMENSIONE MILLIMETRICA E NATURA POLIGENICA.
DEPOSITO LIMOSO-ARGILLOSO CON DEBOLE FRAZIONE SABBIOSA; PRESENZA DI SPORADICI CIOTTOLI DI DIMENSIONE MILLIMETRICA.
DEPOSITO LIMOSO-ARGILLOSO CON PRESENZA DI SPORADICI CIOTTOLI DI DIMENSIONE MILLIMETRICA.
DEPOSITO LIMOSO-ARGILLOSO PLASTICO. TRA 2 E 8 M PRESENZA DI FRAZIONE SABBIOSA GROSSOLANA E SPORADICI CIOTTOLI CENTIMETRICI. TRA 8 E 9 M DIMINUISCE LA GRANULOMETRIA DELLA FRAZIONE SABBIOSA
DEPOSITO LIMOSO-ARGILLOSO POCO PLASTICO.


In [ ]:
#all values containing a key word
keyW = 'DEPOSITO ARGILLOSO-LIMOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

DEPOSITO ARGILLOSO-LIMOSO
DEPOSITO ARGILLOSO-LIMOSO COMPATTO DI COLORE 10 YR 6/8.
DEPOSITO ARGILLOSO-LIMOSO COMPATTO DI COLORE 10YR 3/4-3 CON PRESENZA DI ELEMENTI CARBONIOSI
DEPOSITO ARGILLOSO-LIMOSO COMPATTO E PLASTICO CON FRAZIONE SABBIOSA E SPORA DICI CIOTTOLI CALCAREO-MARNOSI DI DIMENSIONI CENTIMETRICHE.
DEPOSITO ARGILLOSO-LIMOSO COMPATTO, PLASTICO DI COLORE 2.5 YR 3/2 MARRONE SCURO. TRA 28-30 M E TRA 34-36 M PRESENZA DI FRAZIONE SABBIOSA E SPORADICI CIOTTOLI DI NATURA PER LO PIU' CALCAREA, DI DIM....
DEPOSITO ARGILLOSO-LIMOSO CON FRAZIONE SABBIOSA CON SPORADICI CIOTTOLI ETEROMETICI DI DIMENSIONI VARIBILI DAL MILLIMETRO AL CENTIMETRO. COLORE 10YR 4/4.
DEPOSITO ARGILLOSO-LIMOSO CON FRAZIONE SABBIOSA DI COLORE 10YR 5/4 CON SPORADICI CIOTTOLI CALCAREI DI DIMENSIONI VARIABILI DAL MILLIMETRO AL CENTIMETRO LOCALIZZATI AL TETTO DEL LIVELLO
DEPOSITO ARGILLOSO-LIMOSO CON FRAZIONE SABBIOSA; COLORE 10YR 3/3. TRA 2 E 5 M. DI PROFONDITA' IL DEPOSITO RISULTA PREVALENTEMENTE LIMOSO. OLTRE 5 M AUM

In [ ]:
#all values containing a key word
keyW = 'DEPOSITO ALLUVIONALE SABBIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

DEPOSITO ALLUVIONALE SABBIOSO
DEPOSITO ALLUVIONALE SABBIOSO CON ABBONDANTE MATRICE FINE DI COLORE GRIGIO GIALLASTRO CON PRESENZA DI CIOTTOLI TRA 35.5 E 36.5 M.
DEPOSITO ALLUVIONALE SABBIOSO DA MEDIO A FINE, CONTENENTE RARI CIOTTOLI MILLIMETRICI E CENTIMETRICI
DEPOSITO ALLUVIONALE SABBIOSO E GHIAIOSO MOLTO ADDENSATO, INGLOBANTE SUL FONDO INTERCALAZIONI SABBIOSO LIMOSE
DEPOSITO ALLUVIONALE SABBIOSO FINE E LIMOSO CONTENENTE CIOTTOLI CENTIMETRICI
DEPOSITO ALLUVIONALE SABBIOSO GHIAIOSO (GHIAIE A GRANULOMETRIA MEDIA)
DEPOSITO ALLUVIONALE SABBIOSO INTASATO CON SPORADICA PRESENZA DI GHIAIETTO.
DEPOSITO ALLUVIONALE SABBIOSO INTERCALATO DA FREQUENTI CORPI SABBIOSO LIMOSI GIALLASTRI
DEPOSITO ALLUVIONALE SABBIOSO INTERCALATO DA FREQUENTI CORPI SABBIOSO LIMOSI GIALLASTRI E GRIGIASTRI


In [ ]:
#all values containing a key word
keyW = 'DEPOSITO ALLUVIONALE GHIAIOSO,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

DEPOSITO ALLUVIONALE GHIAIOSO, IN MATRICE DA ARGILLOSA AD AERGILLOSA-LIMOSA, LOCALMENTE ABBONDANTE (TOP)
DEPOSITO ALLUVIONALE GHIAIOSO, INGLOBANTE FREQUENTI INTERCALAZIONI SABBIOSO LIMOSE
DEPOSITO ALLUVIONALE GHIAIOSO, INGLOBANTE FREQUENTI INTERCALAZIONI SABBIOSO LIMOSE, CON FREQUENTI BLOCCHI IN PROFONDITA'
DEPOSITO ALLUVIONALE GHIAIOSO, SABBIOSO / GHIAIONE NATURALE A FASCE PIU' O MENO DURE E PIU' O MENO GRASSE CON TROVANTI A DIVERSE QUOTE
DEPOSITO ALLUVIONALE GHIAIOSO, SABBIOSO, CON TROVANTI


In [ ]:
#all values containing a key word
keyW = 'DEPOSITI SABBIOSI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

DEPOSITI SABBIOSI A GRANULOMETRIA FINE CON GHIAIA, DEBOLMENTE LIMOSI;
DEPOSITI SABBIOSI A GRANULOMETRIA FINE CON INTERCALAZIONI LIMOSO-ARGILLOSE;
DEPOSITI SABBIOSI A GRANULOMETRIA FINE, CEMENTATI, CON GHIAIETTO;
DEPOSITI SABBIOSI A GRANULOMETRIA FINE, CEMENTATI;
DEPOSITI SABBIOSI A GRANULOMETRIA FINE;
DEPOSITI SABBIOSI A GRANULOMETRIA GROSSOLANA
DEPOSITI SABBIOSI A GRANULOMETRIA GROSSOLANA CON LIVELLI GHIAIOSI FINI INTERCALATI;
DEPOSITI SABBIOSI A GRANULOMETRIA GROSSOLANA;
DEPOSITI SABBIOSI A GRANULOMETRIA MEDIA CON LOCALI INTERCALAZIONI LIMOSE;
DEPOSITI SABBIOSI A GRANULOMETRIA MEDIA DEBOLMENTE GHIAIOSO-CIOTTOLOSI;
DEPOSITI SABBIOSI A GRANULOMETRIA MEDIA, DEBOLMENTE GHIAIOSI;
DEPOSITI SABBIOSI A GRANULOMETRIA MEDIA;
DEPOSITI SABBIOSI A GRANULOMETRIA MEDIO-GROSSOLANA CON INTERCALAZIONI DI LIVELLI GHIAIOSI.
DEPOSITI SABBIOSI A MEDIA GRANULOMETRIA;
DEPOSITI SABBIOSI ARGILLOSI
DEPOSITI SABBIOSI CEMENTATI;
DEPOSITI SABBIOSI CON GHIAIA
DEPOSITI SABBIOSI CON GHIAIA A GRANULOMETRIA FINE
DEPOS

In [ ]:
#all values containing a key word
keyW = 'DEPOSITI LIMOSO-ARGILLOSI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

DEPOSITI LIMOSO-ARGILLOSI
DEPOSITI LIMOSO-ARGILLOSI COMPATTI, INGLOBANTI LIVELLI GHIAIOSI FINI E CIOTTOLI DISPERSI.
DEPOSITI LIMOSO-ARGILLOSI CON INTERCALAZIONI GHIAIOSE E TROVANTI.
DEPOSITI LIMOSO-ARGILLOSI CON INTERCALAZIONI GHIAIOSE E TROVANTI;
DEPOSITI LIMOSO-ARGILLOSI DEBOLMENTE GHIAIOSI
DEPOSITI LIMOSO-ARGILLOSI DEBOLMENTE SABBIOSI;
DEPOSITI LIMOSO-ARGILLOSI DI COLORE BRUNO-GIALLASTRO;
DEPOSITI LIMOSO-ARGILLOSI DI COLORE BRUNO-GRIGIO;
DEPOSITI LIMOSO-ARGILLOSI DI COLORE GRIGIO-BLU;
DEPOSITI LIMOSO-ARGILLOSI DI COLORE MARRONCINO.
DEPOSITI LIMOSO-ARGILLOSI RICCHI DI MATERIALE ORGANICO DI COLORE BLU-NERASTRO;


In [ ]:
#all values containing a key word
keyW = 'DEPOSITI LIMOSI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

DEPOSITI LIMOSI CON CIOTTOLI DISPERSI E RESTI VEGETALI BEN CONSERVATI;
DEPOSITI LIMOSI CON RESTI VEGETALI;
DEPOSITI LIMOSI DEBOLMENTE GHIAIOSI CON CIOTTOLI DISPERSI DI DIMENSIONI ANCHE PLURIDECIMETRICHE
DEPOSITI LIMOSI DEBOLMENTE SABBIOSI CON CIOTTOLI DISPERSI
DEPOSITI LIMOSI DEBOLMENTE SABBIOSI CON GHIAIETTO;
DEPOSITI LIMOSI DEBOLMENTE SABBIOSI;


In [ ]:
#all values containing a key word
keyW = 'DEPOSITI LIMOSI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

In [ ]:
#all values containing a key word
keyW = 'DEPOSITI GHIAIOSI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

DEPOSITI GHIAIOSI
DEPOSITI GHIAIOSI A GRANULOMETRIA FINE
DEPOSITI GHIAIOSI A GRANULOMETRIA FINE IN SCARSA MATRICE SABBIOSA;
DEPOSITI GHIAIOSI A GRANULOMETRIA GROSSOLANA AD ELEMENTI CLASTICI DI DIMENSIONI CENTIMETRICHE;
DEPOSITI GHIAIOSI A GRANULOMETRIA GROSSOLANA CON PRESENZA DI BLOCCHI DISPERSI;
DEPOSITI GHIAIOSI A GRANULOMETRIA GROSSOLANA;
DEPOSITI GHIAIOSI A GRANULOMETRIA MEDIA, DI COLORE GRIGIO, CON CIOTTOLI DISPERSI DI DIMENSIONI DECIMETRICHE
DEPOSITI GHIAIOSI A GRANULOMETRIA MEDIO-FINE, ALTERNATI A LIVELLI SABBIOSI GROSSOLANI.
DEPOSITI GHIAIOSI CON CIOTTOLI (DIMENSIONI MAX 15 CM) IN SCARSA MATRICE SABBIOSA;
DEPOSITI GHIAIOSI ETEROMETRICI
DEPOSITI GHIAIOSI GROSSOLANI IN SCARSA MATRICE SABBIOSA.
DEPOSITI GHIAIOSI GROSSOLANI;
DEPOSITI GHIAIOSI IN SCARSA MATRICE SABBIOSA;
DEPOSITI GHIAIOSI MEDIO-GROSSOLANI IN SCARSA MATRICE SABBIOSA.
DEPOSITI GHIAIOSI MEDIO-GROSSOLANI, CON CIOTTOLI DI DIMENSIONI DECIMETRICHE.
DEPOSITI GHIAIOSI SEDE DI FALDA CAPTATA


In [ ]:
#all values containing a key word
keyW = 'DEPOSITI ARGILLOSI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

DEPOSITI ARGILLOSI
DEPOSITI ARGILLOSI COMPATTI
DEPOSITI ARGILLOSI COMPATTI.
DEPOSITI ARGILLOSI DEBOLMENTE SABBIOSI DI COLORE GIALLASTRO;
DEPOSITI ARGILLOSI DEBOLMENTE SABBIOSI DI COLORE GRIGIO-BLU CON GHIAIETTO DISPERSO NELLA MATRICE FINE;
DEPOSITI ARGILLOSI DEBOLMENTE SABBIOSI DI COLORE GRIGIO-BLU;
DEPOSITI ARGILLOSI DI COLORE BLUASTRO;
DEPOSITI ARGILLOSI DI COLORE GIALLASTRO.
DEPOSITI ARGILLOSI DI COLORE GRIGIO
DEPOSITI ARGILLOSI DI COLORE GRIGIO-AZZURRO
DEPOSITI ARGILLOSI DI COLORE MARRONE CHIARO.
DEPOSITI ARGILLOSI DI COLORE MARRONE CHIARO;


In [ ]:
#all values containing a key word
keyW = 'CORPO SABBIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

CORPO SABBIOSO
CORPO SABBIOSO GHIAIOSO
CORPO SABBIOSO GHIAIOSO ROSSICCI;
CORPO SABBIOSO LIMOSO
CORPO SABBIOSO LIMOSO GIALLASTRO


In [ ]:
#all values containing a key word
keyW = 'CORPO LIMOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

CORPO LIMOSO ARGILLOSO A COLORAZIONE GIALLASTRA
CORPO LIMOSO ARGILLOSO GIALLASTRO
CORPO LIMOSO ARGILLOSO GRIGIASTRO CONSISTENTE
CORPO LIMOSO SABBIOSO GIALLASTRO


In [ ]:
#all values containing a key word
keyW = 'CORPI GHIAIOSO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

CORPI GHIAIOSO CIOTTOLOSI A GRADO DI CEMENTAZIONE VARIABILE
CORPI GHIAIOSO CIOTTOLOSI ADDENSATI, INGLOBANTI GROSSI BLOCCHI (TROVANTI), E VERSO IL BASSO, LIVELLI SABBIOSO LIMOSI
CORPI GHIAIOSO CIOTTOLOSI ADDENSATI, TALORA PARZIALMENTE CEMENTATI (CONGLOMERATO)
CORPI GHIAIOSO SABBIOSI
CORPI GHIAIOSO SABBIOSI, A GRANULOMETRIA GROSSOLANA, ADDENSATI E DI COLORE GRIGIO


In [ ]:
#all values containing a key word
keyW = 'CORPI GHIAIOSI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

CORPI GHIAIOSI ALTERNATI A LIVELLI PARZIALMENTE CEMENTATI (CONGLOMERATO)
CORPI GHIAIOSI CON FREQUENTI INTERCALAZIONI SABBIOSO LIMOSE


In [ ]:
#all values containing a key word
keyW = 'CONGLOMERATO, GHIAIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

CONGLOMERATO, GHIAIA E ARGILLA
CONGLOMERATO, GHIAIA E ARGILLA GIALLA
CONGLOMERATO, GHIAIA E SABBIA


In [ ]:
#all values containing a key word
keyW = 'CONGLOMERATO POLIGENICO (CEPPO)'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

CONGLOMERATO POLIGENICO (CEPPO)
CONGLOMERATO POLIGENICO (CEPPO) A TRATTI BEN CEMENTATO A TRATTI CON SCARSO GRADO DI CEMENTAZIONE, CON OSSIDAZIONI.
CONGLOMERATO POLIGENICO (CEPPO) A TRATTI BEN CEMENTATO A TRATTI CON SCARSO GRADO DI CEMENTAZIONE. PRESENZA DI ALTERAZIONI E OSSIDAZIONI.
CONGLOMERATO POLIGENICO (CEPPO) A TRATTI BEN CEMENTATO A TRATTI FRATTURATO CON ALTERAZIONI E OSSIDAZIONI.
CONGLOMERATO POLIGENICO (CEPPO) A TRATTI BEN CEMENTATO CON OSSIDAZIONI E ALTERAZIONI.
CONGLOMERATO POLIGENICO (CEPPO) BEN CEMENTATO CON ARENARIA DA MT 39.10 - 39.30.
CONGLOMERATO POLIGENICO (CEPPO) BEN CEMENTATO CON LENTE LIMOSO ARGILLOSA A 47.00 - 47.20.
CONGLOMERATO POLIGENICO (CEPPO) BEN CEMENTATO CON OSSIDAZIONI.
CONGLOMERATO POLIGENICO (CEPPO) BEN CEMENTATO E A TRATTI CON SCARSO GRADO DI CEMENTAZIONE.
CONGLOMERATO POLIGENICO (CEPPO) BEN CEMENTATO.
CONGLOMERATO POLIGENICO (CEPPO) BLU CEMENTATO CON OSSIDAZIONI E ALTERAZIONI.
CONGLOMERATO POLIGENICO (CEPPO) BLU CEMENTATO CON OSSIDAZIONI.
CONGLOMERATO 

In [ ]:
#all values containing a key word
keyW = 'COLTIVO: SABBIA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

COLTIVO: SABBIA DEB. LIMOSA MARRONE INGL. RARO GHIAIETTO E CIOTTOLI
COLTIVO: SABBIA FINE DEB. LIMOSA BRUNO NOCCIOLA
COLTIVO: SABBIA FINE LIMOSA BRUNA. RES. RESTI VEGETALI
COLTIVO: SABBIA LIMOSA BRUNA CON BRECCIA E TROVANTI CALCAREI
COLTIVO: SABBIA LIMOSA BRUNA CON RESTI VEGETALI
COLTIVO: SABBIA LIMOSA BRUNA CON RESTI VEGETALI E RARA GHIAIA
COLTIVO: SABBIA LIMOSA BRUNA INGL. GHIAIA E GHIAIETTO
COLTIVO: SABBIA LIMOSA GRIGIO VERDE CON GHIAIETTO E RARA GHIAIA


In [ ]:
#all values containing a key word
keyW = 'CIOTTOLONI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

CIOTTOLONI
CIOTTOLONI ARGILLOSI
CIOTTOLONI ASCIUTTI
CIOTTOLONI COMPATTI
CIOTTOLONI CON ALTER. TROVANTI
CIOTTOLONI CON ARGILLA COMPATTA GIALLA
CIOTTOLONI CON POCA SABBIA
CIOTTOLONI CON SASSI E ARGILLA
CIOTTOLONI E GHIAIA
CIOTTOLONI E POCA SABBIA ARGILLOSA
CIOTTOLONI PULITI
CIOTTOLONI SCIOLTI SPORCHI


In [ ]:
#all values containing a key word
keyW = 'CIOTOLI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

CIOTOLI CEMENTATI
CIOTOLI CON ARGILLA
CIOTOLI CON CONGLOMERATO
CIOTOLI CON SABBIA CEMENTATI
CIOTOLI E SABBIA
CIOTOLI GHIAIA E SABBIA
CIOTOLI GHIAIETTO E SABBIA
CIOTOLI GRANDI
CIOTOLI GROSSOLANI
CIOTOLI PIU' GROSSI CEMENTATI CON GHIAIA
CIOTOLI SABBIE DI COLORE ROSSO E ACQUA
CIOTOLI TERRA


In [ ]:
#all values containing a key word
keyW = 'CALACRE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

CALACRE DOLOMITICO BIANCASTRO DEBOLMENTE FESSURATO E ALTERATO
CALACRE DOLOMITICO BIANCO-GRIGIASTRO DEBOLMENTE ALTERATO IN ARGILLA CON LOCALMENTE PASSAGGI IN ROCCIA A MAGGIORE FRATTURAZIONE
CALACRE DOLOMITICO FRATTURATO, FRIABILE E DEBOLMENTE RICEMENTATO
CALACRE DOLOMITICO MOLTO FRATTURATO A TRATTI MICROFRATTURATO E FESSURATO DA BIANCO A GRIGIO


In [ ]:
#all values containing a key word
keyW = 'ARGILLE,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argille'

ARGILLE, ARGILLE LIMOSE
ARGILLE, ARGILLE LIMOSE CON INTER. SOTTILI LENTI SABBIOSE
ARGILLE, ARGILLE LIMOSE CON INTERCALAZIONI DI SOTTILI LENTI SABBIOSE
ARGILLE, DI COLORE GRIGIO, TALORA BLUASTRE
ARGILLE, LIMI ARGILLOSI CON CIOTOLETTI SPARSI


In [ ]:
#all values containing a key word
keyW = 'ARGILL'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argille'

ARGILL - SABBIA DA FIUME MEDIA E GROSSA
ARGILL A GRIGIA CON GHIAIETTO
ARGILL A PLASTICA
ARGILL APLASTICA GRIGIO VERDE
ARGILL AZZURRA
ARGILL MISTA A SABBIA
ARGILL VEGETALE


In [ ]:
#all values containing a key word
keyW = 'ARGILA'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argille'

ARGILA BLU E SASSI
ARGILA CON ELEMENTI GHIAIOSI
ARGILA CON GHIAIETTO
ARGILA DEBOLMENTE LIMOSA MEDIAMENTE PLASTICA DA GIALLO AVANA A GRIGIO.
ARGILA GIALLA
ARGILA GIALLASTRA
ARGILA GIALLO OCRA
ARGILA GRIGIA
ARGILA GRIGIA CON POCA GHIAIA
ARGILA LIMOSA NOCCIOLA
ARGILA MARRONE
ARGILA ROSSASTRA CON RARI FRAMMENTI
ARGILA SABBIOSA
ARGILA SABBIOSA GRIGIA CON SCREZIATURE NOCCIOLA


In [ ]:
#all values containing a key word
keyW = 'ALTERNANZE DI LIMI'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

ALTERNANZE DI LIMI ARGILLOSI E ARGILLE LIMOSE
ALTERNANZE DI LIMI ARGILLOSI ED ARGILLE LIMOSE
ALTERNANZE DI LIMI ARGILLOSI ED ARGILLE LIMOSE DI COLORE GRIGIO
ALTERNANZE DI LIMI ARGILLOSI ED ARGILLE LIMOSE DI COLORE NOCCIOLA
ALTERNANZE DI LIMI ARGILLOSI GRIGI NOCC.
ALTERNANZE DI LIMI SABBIOSI E GHIAIE


In [ ]:
#all values containing a key word
keyW = 'ALTERNANZE DI GHIAIE,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ALTERNANZE DI GHIAIE, ARGILLE E CONGLOMERATO
ALTERNANZE DI GHIAIE, GHIAIE CEMENTATE E, SUBORDINATAMENTE, ARGILLE
ALTERNANZE DI GHIAIE, SABBIE E CIOTTOLI CON CROSTONI DI CONGLOMERATO. LIMITATE ED IRREGOLARI LENTI ARGILLOSE DI COLORE OCRA


In [ ]:
#all values containing a key word
keyW = 'ALTERNANZE DI GHIAIE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ALTERNANZE DI GHIAIE E ARGILLA ROSSA
ALTERNANZE DI GHIAIE E SABBIE
ALTERNANZE DI GHIAIE E SABBIE GROSSOLANE DI COLORE OCRA
ALTERNANZE DI GHIAIE ETEROMETRICHE CON SABBIE E SABBIE CON GHIAIE; COLORE DA GRIGIASTRO A GIALLASTRO
ALTERNANZE DI GHIAIE GROSSOLANE E CIOTTOLI SUBARROTONDATI CON LIVELLI SABBIOSI PULITI. LOCALMENTE RISULTANO INTERCALATI DEI TROVANTI
ALTERNANZE DI GHIAIE POLIGENICHE, CIOTTOLI E ARGILLA
ALTERNANZE DI GHIAIE SABBIOSE CON CIOTTOLI E SABBIE GHIAIOSE
ALTERNANZE DI GHIAIE SABBIOSO-LIMOSE E DI SABBIE GHIAIOSE ENTRAMBE A GROSSI CIOTTOLI


In [ ]:
#all values containing a key word
keyW = 'ALTERNANZE DI ARGILLE'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

ALTERNANZE DI ARGILLE CON LIVELLI LIMOSI SUBORDINATI
ALTERNANZE DI ARGILLE E ARGILLE GHIAIOSE
ALTERNANZE DI ARGILLE E GHIAIE
ALTERNANZE DI ARGILLE E LIMI ARGILLOSI
ALTERNANZE DI ARGILLE E SABBIE
ALTERNANZE DI ARGILLE LIMOSE CON LIMI DI COLOR NOCCIOLA
ALTERNANZE DI ARGILLE LIMOSE E LIMI ARGILLOSI DI COLORE GRIGIO
ALTERNANZE DI ARGILLE LIMOSE LIMI ARGILLOSI DI COLORE GRIGIO


In [ ]:
a=np.unique(key_list)
for i in range(len(a)):
  print(a[i])

" FILTRO" FORSE ARGILLA; PERFORO FINO A 116 MT.
"COME SOPRA", DI COLORE BRUNO-ROSSICCIO, TALORA DEBOLMENTE ALTERATI
"DESCRIZIONE LITOLOGICA NON PRESENTE"
"MARNOSO ARENACEA ROMAGNOLA" ARENARIE QUARZOSO FELDSPATICHE IN GROSSI BANCHI CON INTERCALAZIONI DI SILTITI GRIGIE, MARNE ED ARGILLITICI. TRA 27 E 28 E TRA 36 E 38 M DAL P.C. SI HANNO LIVELLI...
"MARNOSO-ARENACEE ROMAGNOLA" ALTERNANZE RITMICHE DI ARENARIE E SILTITI RISEDIMENTATE E DI MARNE ED ARGILLITI. GENERALMENTE LE SILTITI SONO PREVALENTI ED HANNO COMPOSIZIONE ANALOGA ALLE ARENARIE
(CONTINUA) GHIAIA MISTA ARGILLA
(FALDA LEGGERMENTE ACQUIFERA) ARGILLA SABBIOSA GIALLA
(INCLINAZIONE SONDA: 45 GRADI, DIREZIONE 240 GRADI) CONGLOMERATO CEMENTIZIO
(MATERIALE NON RECUPERATO) PROBABILMENTE SABBIA FINE LIMOSA
+ SD) = STRATI DURI. M. 32
,,ILLEGGIBILE
- ALLUVIONI - GHIAIA - SABBIA E CIOTTOLI.
- DESCRIZIONE LITOLOGICA NON PRESENTE -
- DESCRIZIONE NON PRESENTE -
- M. 16 - SECONDA FALDA (POVERA)
- MARNA CON INTERLIVELLI ARENACEI. / FORMAZIONE DI 

In [ ]:
#all values containing a key word
keyW = 'CONGLOMERATO'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

CONGLOMERATO
CONGLOMERATO (CIOTTOLI POLIGENICI CON CRISTALLINI QUARZOSI, PORFIDI E CARBONATICI)
CONGLOMERATO (FALDA ACQUIFERA)
CONGLOMERATO (FORMAZIONE DELLE ARENARIE DI SERRAVALLE) (ACQUIFERO DI POTENZIALE CAPTAZIONE)
CONGLOMERATO (L.S. 30.0)
CONGLOMERATO (MOLTO DURO)
CONGLOMERATO (PERDITA PARZ. DI CIRCOLAZ.)
CONGLOMERATO (PERDITA TOTALE DI CIRCOLAZIONE)
CONGLOMERATO (STRATO DURO M 5,5)
CONGLOMERATO (STRATO DURO M. 1)
CONGLOMERATO (STRATO DURO M. 14)
CONGLOMERATO (STRATO DURO M. 3)
CONGLOMERATO (STRATO DURO M. 4)
CONGLOMERATO (TERRENO COMPATTO M. 5)
CONGLOMERATO - ACQUIFERO
CONGLOMERATO - STRATO DURO
CONGLOMERATO / CONGLOMERATO FESSURATO
CONGLOMERATO / POZZO PREESISTENTE
CONGLOMERATO A CEMENTO ARENACEO CON UN PO' DI LIMO ARGILLOSO
CONGLOMERATO A CEMENTO ARENACEO, BEN CEMENTATO, COMPATTO, CON RARE OSSIDAZIONI.
CONGLOMERATO A CEMENTO CARBONATICO CON TROVANTI.
CONGLOMERATO A CEMENTO CARBONATICO, CON PRESENZA DI LITICI CARBONATICI, ATTRIBUIBILE AL "CEPPO".
CONGLOMERATO A CROSTE
CONGLOMERA

In [ ]:
#all values containing a key word
keyW = 'Argille'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille
Argille


complete supervised categorization

In [ ]:

c = 100002
i =  27543


while c != 0:
  if (val_list[i]==100 or val_list[i]==200 or val_list[i]==201 or val_list[i]==202 or
      val_list[i]==300 or val_list[i]==306 or val_list[i]==400 or val_list[i]==602 or
      val_list[i]==700 or val_list[i]==1001):
    i=i+1
  else:
    print(i)
    print(key_list[i])
    print('1=Argilla  2=Materiale roccioso  3=calcare 4=gesso 5=ghiaia 6=Blocchi,trovanti 7=Limo 8=materiale organico 9=sabba s=skip')
    inp=input('lito:')
    if inp=='s':
        i=i+1
    else:
      c=int(inp)
      if c==1:
        val_list[i]=100
        key_list[i]='Argilla'
        print(key_list[i])
        i=i+1
      elif c==2:
        val_list[i]=200
        key_list[i]='Materiale roccioso del substrato (escluso rocce clastiche)'
        print(key_list[i])
        i=i+1
      elif c==3:
        val_list[i]=201
        key_list[i]='Calcare'
        print(key_list[i])
        i=i+1
      elif c==4:
        val_list[i]=202
        key_list[i]='Gesso, anidrite'
        print(key_list[i])
        i=i+1
      elif c==5:
        val_list[i]=300
        key_list[i]='Ghiaia'
        print(key_list[i])
        i=i+1
      elif c==6:
        val_list[i]=306
        key_list[i]='Blocchi, trovanti, erratici'
        print(key_list[i])
        i=i+1
      elif c==7:
        val_list[i]=400
        key_list[i]='Limo'
        print(key_list[i])
        i=i+1
      elif c==8:
        val_list[i]=602
        key_list[i]='Materiale organico'
        print(key_list[i])
        i=i+1        
      elif c==9:
        val_list[i]=700
        key_list[i]='Sabbia'
        print(key_list[i])
        i=i+1
      else:
        print('ERRORE!!')

27543
UNITA' PERMEABILE DISCONTINUA A SPESSORE RIDOTTO QUANDO E' PRESENTE IN GENERE E' COSTITUITA DA SABBIA FINE
1=Argilla  2=Materiale roccioso  3=calcare 4=gesso 5=ghiaia 6=Blocchi,trovanti 7=Limo 8=materiale organico 9=sabba s=skip
lito:9
Sabbia
27544
UNITA' SUPERFICIALI DI COPERTURA.
1=Argilla  2=Materiale roccioso  3=calcare 4=gesso 5=ghiaia 6=Blocchi,trovanti 7=Limo 8=materiale organico 9=sabba s=skip
lito:s
27545
UNITA0' IMPERMEABILE FORMATA DA LITOTIPI ARGILLOSI - LIMOSI;
1=Argilla  2=Materiale roccioso  3=calcare 4=gesso 5=ghiaia 6=Blocchi,trovanti 7=Limo 8=materiale organico 9=sabba s=skip
lito:1
Argilla
27546
VANO INTERRATO.
1=Argilla  2=Materiale roccioso  3=calcare 4=gesso 5=ghiaia 6=Blocchi,trovanti 7=Limo 8=materiale organico 9=sabba s=skip
lito:s
27547
VEGETALE
1=Argilla  2=Materiale roccioso  3=calcare 4=gesso 5=ghiaia 6=Blocchi,trovanti 7=Limo 8=materiale organico 9=sabba s=skip
lito:8
Materiale organico
27548
VEGETALE CON ARG.
1=Argilla  2=Materiale roccioso  3=calca

IndexError: ignored

In [ ]:
pd.DataFrame(val_list).to_excel('val_list.xlsx')
pd.DataFrame(key_list).to_excel('key_list.xlsx')

In [ ]:
len(np.unique(key_list))

12

In [ ]:
i=0
while i < 27606:
  if (val_list[i]==100 or val_list[i]==200 or val_list[i]==201 or val_list[i]==202 or
      val_list[i]==300 or val_list[i]==306 or val_list[i]==400 or val_list[i]==602 or
      val_list[i]==700 or val_list[i]==900):
    i=i+1
  else:
    print(i)
    print(key_list[i])
    val_list[i]=900
    key_list[i]='Altro'
    i=i+1

1
"COME SOPRA", DI COLORE BRUNO-ROSSICCIO, TALORA DEBOLMENTE ALTERATI


In [ ]:
print(np.unique(key_list))
print(np.unique(val_list))
print(len(np.unique(key_list)))
print(len(np.unique(val_list)))

['Altro' 'Argilla' 'Blocchi, trovanti, erratici' 'Calcare'
 'Gesso, anidrite' 'Ghiaia' 'Limo' 'Materiale organico'
 'Materiale roccioso del substrato (escluso rocce clastiche)' 'Sabbia']
[100 200 201 202 300 306 400 602 700 900]
10
10


In [ ]:
#creo e stampo il dizionario
litoDict =  {}

i=0

litotype = pd.DataFrame(np.zeros(len(pd.DataFrame(key_list)[0].unique())))
litotype['name'] = pd.DataFrame(np.zeros(len(pd.DataFrame(key_list)[0].unique())))
litotype = litotype.iloc[: , 1:]
litotype['name'] = pd.DataFrame(key_list)[0].unique()
litotype.name.astype(str)
#litotype.name[742] ='zero'


for lito in litotype.name.unique():
  litoDict[lito] = pd.DataFrame(val_list)[0].unique()[i]
  i += 1
litoDict

{'Altro': 900,
 'Argilla': 100,
 'Blocchi, trovanti, erratici': 306,
 'Calcare': 201,
 'Gesso, anidrite': 202,
 'Ghiaia': 300,
 'Limo': 400,
 'Materiale organico': 602,
 'Materiale roccioso del substrato (escluso rocce clastiche)': 200,
 'Sabbia': 700}

In [ ]:
Dict =  {}

i=0

litotype = pd.DataFrame(np.zeros(len(wellLoc.l1_descr.unique())))
litotype['name'] = pd.DataFrame(np.zeros(len(wellLoc.l1_descr.unique())))
litotype = litotype.iloc[: , 1:]
litotype['name'] = wellLoc.l1_descr.unique()
litotype.name.astype(str)
litotype.name[742] ='zero'
litotype = litotype.sort_values(by=['name'],ignore_index=1)


for lito in litotype.name.unique():
  Dict[lito] = key_list[i]
  i += 1
Dict

{'" FILTRO" FORSE ARGILLA; PERFORO FINO A 116 MT.': 'Altro',
 '"COME SOPRA", DI COLORE BRUNO-ROSSICCIO, TALORA DEBOLMENTE ALTERATI': 'Altro',
 '"DESCRIZIONE LITOLOGICA NON PRESENTE"': 'Altro',
 '"MARNOSO ARENACEA ROMAGNOLA" ARENARIE QUARZOSO FELDSPATICHE IN GROSSI BANCHI CON INTERCALAZIONI DI SILTITI GRIGIE, MARNE ED ARGILLITICI. TRA 27 E 28 E TRA 36 E 38 M DAL P.C. SI HANNO LIVELLI...': 'Calcare',
 '"MARNOSO-ARENACEE ROMAGNOLA" ALTERNANZE RITMICHE DI ARENARIE E SILTITI RISEDIMENTATE E DI MARNE ED ARGILLITI. GENERALMENTE LE SILTITI SONO PREVALENTI ED HANNO COMPOSIZIONE ANALOGA ALLE ARENARIE': 'Calcare',
 '(CONTINUA) GHIAIA MISTA ARGILLA': 'Ghiaia',
 '(FALDA LEGGERMENTE ACQUIFERA) ARGILLA SABBIOSA GIALLA': 'Argilla',
 '(INCLINAZIONE SONDA: 45 GRADI, DIREZIONE 240 GRADI) CONGLOMERATO CEMENTIZIO': 'Altro',
 '(MATERIALE NON RECUPERATO) PROBABILMENTE SABBIA FINE LIMOSA': 'Altro',
 '+ SD) = STRATI DURI. M. 32': 'Altro',
 ',,ILLEGGIBILE': 'Altro',
 '- ALLUVIONI - GHIAIA - SABBIA E CIOTTOLI.':

In [ ]:
wellLoc['category_name'] = wellLoc['l1_descr']
wellLoc = wellLoc.replace({"category_name": Dict})

In [ ]:
wellLoc['category_name'].unique()

array(['Ghiaia', 'Argilla', 'Altro', 'Materiale organico', 'Sabbia',
       'Materiale roccioso del substrato (escluso rocce clastiche)',
       'Limo', 'Calcare', 'Blocchi, trovanti, erratici',
       'ARENARIA CON TRACCE DI ARGILLA', 'Gesso, anidrite'], dtype=object)

In [ ]:
 wellLoc['category'] = wellLoc['category_name']
 wellLoc = wellLoc.replace({"category": litoDict})

In [ ]:
litoDict

{'Altro': 900,
 'Argilla': 100,
 'Blocchi, trovanti, erratici': 306,
 'Calcare': 201,
 'Gesso, anidrite': 202,
 'Ghiaia': 300,
 'Limo': 400,
 'Materiale organico': 602,
 'Materiale roccioso del substrato (escluso rocce clastiche)': 200,
 'Sabbia': 700}

#Data export

In [ ]:
wellLoc.to_csv('wellLito_Po.csv')

In [ ]:
# creo un file di visualizzazione dei risultati
grid = pv.UnstructuredGrid(offsetArray, linArray, cellType, vertArray)
grid.cell_arrays["values"] = wellLoc.category.values
grid.save('wellLito_Po.vtu',binary=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: VTK 9 no longer accepts an offset array
  
/usr/local/lib/python3.7/dist-packages/pyvista/core/dataset.py:1335: PyvistaDeprecationWarning: Use of `cell_arrays` is deprecated. Use `cell_data` instead.
  PyvistaDeprecationWarning
